# Metabolic Harmony
## A Harmonic Model of Human Gut Microbiome Activity

This notebook maps ~573 curated metabolic pathways to unique harmonic ratios based on:
- **Category character**: Energy (consonant) → Biosynthesis (complex) → Degradation (subharmonic)
- **Prevalence/Abundance**: Common pathways receive simpler, more consonant ratios
- **Polarity**: Anabolic processes (harmonic) vs Catabolic processes (subharmonic)
- **Paired pathways**: Biosynthesis/Degradation pairs receive inverse ratios (5/1 ↔ 1/5)

Each section generates a ratio pool suited to its metabolic character, then assigns ratios
to pathways while tracking global usage to prevent duplicates.

---

### PART 1: INFRASTRUCTURE 
---
Core classes and functions for ratio analysis, generation, and tracking.

In [1]:
# ============================================================
# 1.1 IMPORTS & CONFIGURATION
# ============================================================

from math import gcd, log, log2, sqrt, floor, ceil
from collections import defaultdict, OrderedDict
from typing import Dict, List, Tuple, Optional, Set, Union
from IPython.display import display, HTML, clear_output
from functools import reduce
import pandas as pd
import numpy as np
import json
import csv
import os

# Configuration
MIN_HARMONIC = 8         # Lower boundary (1/8 = 0.125)
MAX_HARMONIC = 16        # Upper boundary (16/1 = 16.0)
FUNDAMENTAL_HZ = 660     # A5 - matches your visualization
SAVE_PATH = "harmonic_map.json"

# Derived
RANGE_LOW = 1.0 / MIN_HARMONIC   # 0.125
RANGE_HIGH = float(MAX_HARMONIC)  # 16.0
NUM_BANDS_HARMONIC = int(log2(MAX_HARMONIC))      # 4 bands above unison
NUM_BANDS_SUBHARMONIC = int(log2(MIN_HARMONIC))   # 3 bands below unison

print("✓ Infrastructure imports loaded")
print(f"  Fundamental: {FUNDAMENTAL_HZ} Hz")
print(f"  Range: 1/{MIN_HARMONIC} to {MAX_HARMONIC}/1  ({RANGE_LOW:.4f} to {RANGE_HIGH})")
print(f"  Bands: {NUM_BANDS_SUBHARMONIC} below + {NUM_BANDS_HARMONIC} above = {NUM_BANDS_SUBHARMONIC + NUM_BANDS_HARMONIC} total")
print(f"  Frequency range: {FUNDAMENTAL_HZ * RANGE_LOW:.1f} Hz to {FUNDAMENTAL_HZ * RANGE_HIGH:.1f} Hz")

✓ Infrastructure imports loaded
  Fundamental: 660 Hz
  Range: 1/8 to 16/1  (0.1250 to 16.0)
  Bands: 3 below + 4 above = 7 total
  Frequency range: 82.5 Hz to 10560.0 Hz


In [2]:
# ============================================================
# 1.2 PRIME FACTORIZATION UTILITIES
# ============================================================

def prime_factors(n: int) -> Dict[int, int]:
    """Return prime factorization as {prime: exponent}."""
    if n < 1:
        return {}
    factors = {}
    d = 2
    while d * d <= n:
        while n % d == 0:
            factors[d] = factors.get(d, 0) + 1
            n //= d
        d += 1
    if n > 1:
        factors[n] = factors.get(n, 0) + 1
    return factors


def prime_limit(n: int, d: int) -> int:
    """Return the highest prime in the ratio's factorization."""
    if n == 0 or d == 0:
        return 0
    all_factors = set(prime_factors(n).keys()) | set(prime_factors(d).keys())
    return max(all_factors) if all_factors else 1


def prime_power_vector(n: int, d: int, primes: List[int] = [2, 3, 5, 7, 11, 13]) -> List[int]:
    """
    Return the exponent vector for the ratio.
    Positive = in numerator, Negative = in denominator.
    E.g., 5/4 = 5^1 * 2^-2 → [−2, 0, 1, 0, 0, 0]
    """
    n_factors = prime_factors(n)
    d_factors = prime_factors(d)
    return [n_factors.get(p, 0) - d_factors.get(p, 0) for p in primes]


def format_prime_vector(n: int, d: int) -> str:
    """Human-readable prime factorization."""
    vec = prime_power_vector(n, d)
    primes = [2, 3, 5, 7, 11, 13]
    parts = []
    for p, exp in zip(primes, vec):
        if exp != 0:
            parts.append(f"{p}^{exp}" if exp != 1 else str(p))
    return " · ".join(parts) if parts else "1"


# Test
print("Prime factorization tests:")
for n, d in [(3, 2), (5, 4), (7, 6), (9, 8), (1, 5)]:
    print(f"  {n}/{d} = {format_prime_vector(n, d):<20} prime_limit={prime_limit(n, d)}")

Prime factorization tests:
  3/2 = 2^-1 · 3             prime_limit=3
  5/4 = 2^-2 · 5             prime_limit=5
  7/6 = 2^-1 · 3^-1 · 7      prime_limit=7
  9/8 = 2^-3 · 3^2           prime_limit=3
  1/5 = 5^-1                 prime_limit=5


In [3]:
# ============================================================
# 1.3 RATIO CLASS
# ============================================================

class Ratio:
    """
    A musical ratio with harmonic analysis and fold tracking.
    
    Features:
    - Prime factorization and limit analysis
    - Continuous polarity (-1 to +1) based on harmonic/subharmonic character
    - Core extraction (octave-independent identity)
    - Fold tracking (preserves original ratio)
    - Consonance calculation
    """
    
    def __init__(self, n, d, original_n=None, original_d=None):
        if d == 0:
            raise ValueError("Denominator cannot be zero")
        g = gcd(n, d) if n > 0 else d
        self.n = n // g if n > 0 else 0
        self.d = d // g
        
        # Track original (pre-fold) ratio
        self.original_n = original_n if original_n is not None else self.n
        self.original_d = original_d if original_d is not None else self.d
    
    def __repr__(self):
        return f"{self.n}/{self.d}"
    
    def __str__(self):
        return f"{self.n}/{self.d}"
    
    def __format__(self, format_spec):
        return format(str(self), format_spec)
    
    def __eq__(self, other):
        if isinstance(other, Ratio):
            return self.n == other.n and self.d == other.d
        if isinstance(other, tuple) and len(other) == 2:
            return self.n == other[0] and self.d == other[1]
        return False
    
    def __hash__(self):
        return hash((self.n, self.d))
    
    def __lt__(self, other):
        return self.value() < other.value()
    
    # ─────────────────────────────────────────────────────────────
    # BASIC PROPERTIES
    # ─────────────────────────────────────────────────────────────
    
    def value(self):
        """Decimal value of the ratio."""
        return self.n / self.d if self.d != 0 else float('inf')
    
    def cents(self):
        """Distance from 1/1 in cents."""
        if self.n <= 0:
            return float('-inf')
        return 1200 * log2(self.value())
    
    def hz(self, fundamental=660):
        """Frequency in Hz at given fundamental."""
        return fundamental * self.value()
    
    def consonance(self):
        """Current consonance (n × d). Lower = more consonant."""
        return self.n * self.d
    
    def original_consonance(self):
        """Consonance of original (pre-fold) ratio."""
        return self.original_n * self.original_d
    
    def original_str(self):
        """String representation of original ratio."""
        return f"{self.original_n}/{self.original_d}"
    
    def was_folded(self):
        """True if this ratio was folded from a different ratio."""
        return self.n != self.original_n or self.d != self.original_d
    
    def to_tuple(self):
        """Return (n, d) tuple."""
        return (self.n, self.d)
    
    def inverse(self):
        """Return d/n."""
        return Ratio(self.d, self.n)
    
    # ─────────────────────────────────────────────────────────────
    # PRIME ANALYSIS
    # ─────────────────────────────────────────────────────────────
    
    def _get_prime_factors(self, num):
        """Get set of prime factors."""
        if num <= 1:
            return set()
        factors = set()
        d = 2
        temp = num
        while d * d <= temp:
            while temp % d == 0:
                factors.add(d)
                temp //= d
            d += 1
        if temp > 1:
            factors.add(temp)
        return factors
    
    def all_primes(self):
        """All primes in both numerator and denominator."""
        return sorted(self._get_prime_factors(self.n) | self._get_prime_factors(self.d))
    
    def prime_factors(self):
        """Tuple of odd primes involved. (2,) for octaves-only, (1,) for unison."""
        all_p = self.all_primes()
        odd_p = [p for p in all_p if p > 2]
        if odd_p:
            return tuple(odd_p)
        elif 2 in all_p:
            return (2,)
        else:
            return (1,)
    
    def prime_factors_str(self):
        """Human-readable prime factors."""
        pf = self.prime_factors()
        if pf == (1,):
            return "1"
        elif pf == (2,):
            return "2"
        return "×".join(str(p) for p in pf)
    
    def prime_limit(self):
        """Highest prime factor (the 'limit' of the ratio)."""
        pf = self.prime_factors()
        return max(pf)
    
    # ─────────────────────────────────────────────────────────────
    # CORE (octave-independent identity)
    # ─────────────────────────────────────────────────────────────
    
    def core(self):
        """Strip powers of 2 — the octave-independent identity."""
        cn, cd = self.n, self.d
        while cn % 2 == 0 and cn > 0:
            cn //= 2
        while cd % 2 == 0 and cd > 0:
            cd //= 2
        return (cn, cd)
    
    # ─────────────────────────────────────────────────────────────
    # POLARITY (continuous -1 to +1)
    # ─────────────────────────────────────────────────────────────
    
    def _prime_weight(self, x):
        """Sum of log(prime) for odd primes in factorization."""
        if x <= 1:
            return 0
        total = 0
        temp = x
        d = 2
        while d * d <= temp:
            while temp % d == 0:
                if d > 2:
                    total += log(d)
                temp //= d
            d += 1
        if temp > 2:
            total += log(temp)
        return total
    
    def polarity(self):
        """
        Harmonic (+1) vs subharmonic (-1) character.
        
        Returns a continuous value from -1 to +1:
        - +1: Pure harmonic (odd/1)
        - -1: Pure subharmonic (1/odd)
        - 0: Unison or balanced
        - Values decay toward 0 with octave displacement
        
        Examples:
            3/1 → +1.0 (pure harmonic)
            1/3 → -1.0 (pure subharmonic)
            3/2 → +1.0 (perfect fifth, harmonic)
            5/4 → +0.58 (major third, harmonic but decayed by octave)
            6/5 → -0.13 (minor third, slightly subharmonic)
        """
        cn, cd = self.core()
        
        # Count powers of 2 in original ratio
        n_twos = d_twos = 0
        temp_n, temp_d = self.n, self.d
        while temp_n % 2 == 0 and temp_n > 0:
            temp_n //= 2
            n_twos += 1
        while temp_d % 2 == 0 and temp_d > 0:
            temp_d //= 2
            d_twos += 1
        
        # Pure octaves (core is 1/1)
        if cn == 1 and cd == 1:
            if self.n > self.d:
                return 1.0
            elif self.n < self.d:
                return -1.0
            return 0.0
        
        # Harmonic: odd/1 (decayed by denominator's octaves)
        if cd == 1:
            return 1.0 / sqrt(1 + d_twos)
        
        # Subharmonic: 1/odd (decayed by numerator's octaves)
        if cn == 1:
            return -1.0 / sqrt(1 + n_twos)
        
        # Mixed: weight by prime content
        n_weight = self._prime_weight(cn)
        d_weight = self._prime_weight(cd)
        if n_weight + d_weight == 0:
            return 0.0
        base = (n_weight - d_weight) / (n_weight + d_weight)
        decay = 1 / sqrt(1 + n_twos + d_twos)
        return base * decay
    
    def polarity_label(self):
        """Categorical label based on polarity value."""
        p = self.polarity()
        if p > 0.3:
            return "harmonic"
        elif p < -0.3:
            return "subharmonic"
        else:
            return "neutral"
    
    # ─────────────────────────────────────────────────────────────
    # SPECIAL PROPERTIES
    # ─────────────────────────────────────────────────────────────
    
    def is_superparticular(self):
        """True if n = d + 1 (e.g., 3/2, 4/3, 5/4)."""
        return self.n == self.d + 1
    
    def is_harmonic(self):
        """True if core denominator is 1 (pure harmonic series member)."""
        return self.core()[1] == 1
    
    def is_subharmonic(self):
        """True if core numerator is 1 (pure subharmonic series member)."""
        return self.core()[0] == 1
    
    # ─────────────────────────────────────────────────────────────
    # FOLDING
    # ─────────────────────────────────────────────────────────────
    
    def fold(self, max_harmonic=16):
        """
        Fold ratio into range [1/max, max] preserving polarity.
        Returns new Ratio with original tracked.
        """
        n, d = self.n, self.d
        
        if n > d:  # harmonic
            while n > max_harmonic * d:
                d *= 2
            while n < d:
                n *= 2
        else:  # subharmonic
            while d > max_harmonic * n:
                n *= 2
            while d < n:
                d *= 2
        
        return Ratio(n, d, self.original_n, self.original_d)
    
    # ─────────────────────────────────────────────────────────────
    # INFO
    # ─────────────────────────────────────────────────────────────
    
    def info(self):
        """Full analysis dictionary."""
        return {
            'ratio': str(self),
            'n': self.n,
            'd': self.d,
            'decimal': round(self.value(), 6),
            'cents': round(self.cents(), 2),
            'hz': round(self.hz(), 2),
            'consonance': self.consonance(),
            'prime_limit': self.prime_limit(),
            'prime_factors': self.prime_factors_str(),
            'core': f"{self.core()[0]}/{self.core()[1]}",
            'polarity': round(self.polarity(), 3),
            'polarity_label': self.polarity_label(),
            'is_harmonic': self.is_harmonic(),
            'is_subharmonic': self.is_subharmonic(),
            'is_superparticular': self.is_superparticular(),
        }


# ─────────────────────────────────────────────────────────────
# Test the Ratio class
# ─────────────────────────────────────────────────────────────

print("✓ Ratio class loaded")
print()
print("Polarity examples:")
test_ratios = [(3, 1), (1, 3), (3, 2), (2, 3), (5, 4), (4, 5), (6, 5), (5, 6), (9, 8), (15, 8)]
for n, d in test_ratios:
    r = Ratio(n, d)
    print(f"  {str(r):<6} polarity={r.polarity():>+6.3f}  {r.polarity_label():<12} core={r.core()[0]}/{r.core()[1]}")


✓ Ratio class loaded

Polarity examples:
  3/1    polarity=+1.000  harmonic     core=3/1
  1/3    polarity=-1.000  subharmonic  core=1/3
  3/2    polarity=+0.707  harmonic     core=3/1
  2/3    polarity=-0.707  subharmonic  core=1/3
  5/4    polarity=+0.577  harmonic     core=5/1
  4/5    polarity=-0.577  subharmonic  core=1/5
  6/5    polarity=-0.133  neutral      core=3/5
  5/6    polarity=+0.133  neutral      core=5/3
  9/8    polarity=+0.500  harmonic     core=9/1
  15/8   polarity=+0.500  harmonic     core=15/1


In [4]:
# ============================================================
# 1.4 RATIO GENERATION FUNCTIONS
# ============================================================

def fold_ratio(n, d, max_harmonic=MAX_HARMONIC, min_harmonic=MIN_HARMONIC, mode="wrap"):
    """
    Fold a ratio into range [1/min_harmonic, max_harmonic].
    
    Supports ASYMMETRIC ranges like [1/8, 16].
    
    Preserves polarity:
    - Ratios > 1 stay > 1 (folded into [1, max_harmonic])
    - Ratios < 1 stay < 1 (folded into [1/min_harmonic, 1])
    
    Only adjusts by exact powers of 2 (octave shifts).
    Uses band-based logic for correct wrap/reflect behavior.
    
    Args:
        n, d: Numerator and denominator
        max_harmonic: Upper boundary (e.g., 16)
        min_harmonic: Lower boundary denominator (e.g., 8 means 1/8)
        mode: "wrap" (saw) or "reflect" (tri)
    
    Returns:
        (new_n, new_d) tuple
    """
    if n == d:
        return (n, d)
    
    value = n / d
    
    # Determine polarity
    is_harmonic = value > 1
    
    if is_harmonic:
        # Work in harmonic space (value > 1)
        num_bands = int(log2(max_harmonic))  # e.g., 4 for max=16
        current_band = floor(log2(value))     # Which band is the value in?
        
        # Already in range? (bands 0 to num_bands-1)
        if 0 <= current_band < num_bands:
            return (n, d)
        
        # Calculate target band
        if mode == "wrap":
            target_band = current_band % num_bands
        elif mode == "reflect":
            down = list(range(num_bands - 1, -1, -1))
            up = list(range(1, num_bands - 1))
            pattern = down + up if up else down
            excess = current_band - num_bands
            target_band = pattern[excess % len(pattern)]
        else:
            raise ValueError("mode must be 'wrap' or 'reflect'")
        
        # Calculate octave shift
        band_shift = current_band - target_band
        new_d = d * (2 ** band_shift)
        
        g = gcd(n, new_d)
        return (n // g, new_d // g)
    
    else:
        # Work in subharmonic space (value < 1)
        # Flip to work with value > 1, then flip back
        inv_value = d / n
        num_bands = int(log2(min_harmonic))  # e.g., 3 for min=8
        current_band = floor(log2(inv_value))
        
        # Already in range?
        if 0 <= current_band < num_bands:
            return (n, d)
        
        # Calculate target band
        if mode == "wrap":
            target_band = current_band % num_bands
        elif mode == "reflect":
            down = list(range(num_bands - 1, -1, -1))
            up = list(range(1, num_bands - 1))
            pattern = down + up if up else down
            excess = current_band - num_bands
            target_band = pattern[excess % len(pattern)]
        else:
            raise ValueError("mode must be 'wrap' or 'reflect'")
        
        # Calculate octave shift (applied to numerator for subharmonic)
        band_shift = current_band - target_band
        new_n = n * (2 ** band_shift)
        
        g = gcd(new_n, d)
        return (new_n // g, d // g)


def generate_folded(count, max_harmonic=MAX_HARMONIC, min_harmonic=MIN_HARMONIC,
                    fold_mode="wrap", max_prime_limit=None, exclude_ratios=None):
    """
    Generate exactly `count` ratios by consonance, folding into range.
    
    Key insight: We fold DURING generation, so we can skip duplicates
    and keep generating until we have exactly `count` unique ratios.
    
    Args:
        count: Exactly how many unique folded ratios to generate
        max_harmonic: Upper range boundary
        min_harmonic: Lower range boundary (as denominator, e.g., 8 for 1/8)
        fold_mode: "wrap" (saw) or "reflect" (tri)
        max_prime_limit: Optional prime limit filter
        exclude_ratios: Set of (n, d) tuples to skip
    
    Returns:
        List of Ratio objects (with original tracking)
    """
    result = []
    seen_folded = set()
    consonance = 1
    
    if exclude_ratios is None:
        exclude_ratios = set()
    
    while len(result) < count and consonance < 1000000:
        for n in range(1, consonance + 1):
            if consonance % n == 0:
                d = consonance // n
                if gcd(n, d) == 1:
                    # Prime limit check (on original)
                    if max_prime_limit:
                        temp_r = Ratio(n, d)
                        if temp_r.prime_limit() > max_prime_limit:
                            continue
                    
                    # Fold the ratio
                    folded_n, folded_d = fold_ratio(n, d, max_harmonic, min_harmonic, fold_mode)
                    
                    # Skip if in exclusion set
                    if (folded_n, folded_d) in exclude_ratios:
                        continue
                    
                    # Skip if we've seen this folded ratio before
                    folded_key = (folded_n, folded_d)
                    if folded_key in seen_folded:
                        continue
                    
                    seen_folded.add(folded_key)
                    
                    # Create ratio with original tracking
                    r = Ratio(folded_n, folded_d, original_n=n, original_d=d)
                    result.append(r)
                    
                    if len(result) >= count:
                        break
            if len(result) >= count:
                break
        consonance += 1
    
    return result


def generate_natural_range(count, min_value=RANGE_LOW, max_value=RANGE_HIGH, 
                           max_prime_limit=None, exclude_ratios=None):
    """
    Generate exactly `count` ratios that naturally fall within range.
    Ordered by consonance (n × d).
    
    No folding - these ratios genuinely exist in the specified range.
    """
    result = []
    seen = set()
    consonance = 1
    
    if exclude_ratios is None:
        exclude_ratios = set()
    
    while len(result) < count and consonance < 500000:
        for n in range(1, consonance + 1):
            if consonance % n == 0:
                d = consonance // n
                if gcd(n, d) == 1:
                    value = n / d
                    
                    # Must be in range
                    if not (min_value <= value <= max_value):
                        continue
                    
                    # Prime limit check
                    if max_prime_limit:
                        r = Ratio(n, d)
                        if r.prime_limit() > max_prime_limit:
                            continue
                    
                    # Exclusion check
                    if (n, d) in exclude_ratios:
                        continue
                    
                    key = (n, d)
                    if key not in seen:
                        seen.add(key)
                        result.append(Ratio(n, d))
                    
                    if len(result) >= count:
                        break
            if len(result) >= count:
                break
        consonance += 1
    
    return result


# ─────────────────────────────────────────────────────────────
# ADDITIONAL GENERATION HELPERS
# ─────────────────────────────────────────────────────────────

def generate_harmonic_series(start=1, end=16, denominator=1):
    """Generate n/d for n in [start, end]. Classic harmonic series."""
    return [Ratio(n, denominator) for n in range(start, end + 1) if gcd(n, denominator) == 1]


def generate_subharmonic_series(start=1, end=8, numerator=1):
    """Generate n/d for d in [start, end]. Subharmonic series."""
    return [Ratio(numerator, d) for d in range(start, end + 1) if gcd(numerator, d) == 1]


def generate_by_prime_limit(limit, count=None, max_consonance=10000, 
                            polarity=None, exclude_ratios=None):
    """
    Generate ratios up to a prime limit within the configured range.
    
    Args:
        limit: Maximum prime (2, 3, 5, 7, 11, 13...)
        count: How many ratios (None = all up to max_consonance)
        polarity: "harmonic" (>1), "subharmonic" (<1), or None for both
        exclude_ratios: Set of (n, d) tuples to skip
    """
    result = []
    seen = set()
    consonance = 1
    
    if exclude_ratios is None:
        exclude_ratios = set()
    
    while consonance <= max_consonance:
        if count is not None and len(result) >= count:
            break
        
        for n in range(1, consonance + 1):
            if consonance % n == 0:
                d = consonance // n
                if gcd(n, d) == 1:
                    value = n / d
                    
                    # Must be in configured range
                    if not (RANGE_LOW <= value <= RANGE_HIGH):
                        continue
                    
                    # Polarity filter
                    if polarity == "harmonic" and n <= d:
                        continue
                    if polarity == "subharmonic" and n >= d:
                        continue
                    
                    r = Ratio(n, d)
                    
                    # Prime limit check
                    if r.prime_limit() > limit:
                        continue
                    
                    # Exclusion check
                    if (n, d) in exclude_ratios:
                        continue
                    
                    if (n, d) not in seen:
                        seen.add((n, d))
                        result.append(r)
                    
                    if count is not None and len(result) >= count:
                        break
            
            if count is not None and len(result) >= count:
                break
        
        consonance += 1
    
    return result


# ─────────────────────────────────────────────────────────────
# TEST
# ─────────────────────────────────────────────────────────────

print("✓ Generation functions loaded")
print(f"  Range: [1/{MIN_HARMONIC}, {MAX_HARMONIC}] = [{RANGE_LOW}, {RANGE_HIGH}]")
print()

# Test asymmetric folding
print("FOLD TEST (asymmetric range [1/8, 16]):")
print("-" * 60)
print(f"{'Original':<12} {'Value':<10} {'Folded':<12} {'New Value':<10} {'OK?'}")
print("-" * 60)

test_cases = [
    (3, 1),    # 3.0 - in range
    (17, 1),   # 17.0 - needs fold down
    (32, 1),   # 32.0 - needs fold down
    (64, 1),   # 64.0 - needs fold down
    (1, 3),    # 0.333 - in range
    (1, 9),    # 0.111 - needs fold up
    (1, 16),   # 0.0625 - needs fold up
    (1, 32),   # 0.03125 - needs fold up
]

for n, d in test_cases:
    val = n / d
    fn, fd = fold_ratio(n, d)
    new_val = fn / fd
    in_range = RANGE_LOW <= new_val <= RANGE_HIGH
    ok = "✓" if in_range else "✗"
    print(f"{n}/{d:<10} {val:<10.4f} {fn}/{fd:<10} {new_val:<10.4f} {ok}")


✓ Generation functions loaded
  Range: [1/8, 16] = [0.125, 16.0]

FOLD TEST (asymmetric range [1/8, 16]):
------------------------------------------------------------
Original     Value      Folded       New Value  OK?
------------------------------------------------------------
3/1          3.0000     3/1          3.0000     ✓
17/1          17.0000    17/16         1.0625     ✓
32/1          32.0000    2/1          2.0000     ✓
64/1          64.0000    4/1          4.0000     ✓
1/3          0.3333     1/3          0.3333     ✓
1/9          0.1111     8/9          0.8889     ✓
1/16         0.0625     1/2          0.5000     ✓
1/32         0.0312     1/4          0.2500     ✓


In [5]:
# ============================================================
# 1.5 RATIO FILTERING & SORTING
# ============================================================

def filter_ratios(ratios: List[Ratio], 
                  min_consonance: int = None,
                  max_consonance: int = None,
                  prime_limit_max: int = None,
                  polarity: str = None,
                  min_cents: float = None,
                  max_cents: float = None,
                  exclude: Set[Tuple[int, int]] = None) -> List[Ratio]:
    """
    Filter ratio list by multiple criteria.
    """
    result = ratios[:]
    
    if min_consonance is not None:
        result = [r for r in result if r.consonance() >= min_consonance]
    if max_consonance is not None:
        result = [r for r in result if r.consonance() <= max_consonance]
    if prime_limit_max is not None:
        result = [r for r in result if r.prime_limit() <= prime_limit_max]
    if polarity is not None:
        result = [r for r in result if r.polarity() == polarity]
    if min_cents is not None:
        result = [r for r in result if r.cents() >= min_cents]
    if max_cents is not None:
        result = [r for r in result if r.cents() <= max_cents]
    if exclude is not None:
        result = [r for r in result if r.to_tuple() not in exclude]
    
    return result


def sort_ratios(ratios: List[Ratio], by: str = 'consonance', reverse: bool = False) -> List[Ratio]:
    """
    Sort ratios by: consonance, cents, decimal, prime_limit, n, d
    """
    key_funcs = {
        'consonance': lambda r: r.consonance(),
        'cents': lambda r: r.cents(),
        'decimal': lambda r: r.decimal,
        'prime_limit': lambda r: (r.prime_limit(), r.consonance()),
        'n': lambda r: (r.n, r.d),
        'd': lambda r: (r.d, r.n),
    }
    return sorted(ratios, key=key_funcs.get(by, key_funcs['consonance']), reverse=reverse)


def ratio_stats(ratios: List[Ratio]) -> dict:
    """Summary statistics for a ratio list."""
    if not ratios:
        return {'count': 0}
    
    consonances = [r.consonance() for r in ratios]
    limits = [r.prime_limit() for r in ratios]
    
    return {
        'count': len(ratios),
        'consonance_range': (min(consonances), max(consonances)),
        'prime_limit_range': (min(limits), max(limits)),
        'harmonic': sum(1 for r in ratios if r.polarity() == 'harmonic'),
        'subharmonic': sum(1 for r in ratios if r.polarity() == 'subharmonic'),
        'unison': sum(1 for r in ratios if r.polarity() == 'unison'),
        'by_prime': {p: sum(1 for r in ratios if r.prime_limit() == p) for p in sorted(set(limits))},
    }


print("Filter/sort functions loaded ✓")

Filter/sort functions loaded ✓


In [6]:
# ============================================================
# 1.6 HARMONIC MAP - Global Assignment Registry
# ============================================================

class HarmonicMap:
    """
    Central registry for all pathway-to-ratio assignments.
    
    Features:
    - Tracks assignments, prevents duplicates
    - Rich metadata per assignment
    - Save/load for persistence across sessions
    - Analysis: by category, by prime limit, by consonance
    - Export: JS, JSON, CSV
    """
    
    def __init__(self):
        self.assignments: Dict[str, Ratio] = {}  # pathway_id → Ratio
        self.metadata: Dict[str, dict] = {}      # pathway_id → metadata
        self.used_ratios: Set[Tuple[int, int]] = set()
        self.log: List[dict] = []                # Ordered assignment history
        self._pending: Dict[str, Ratio] = {}     # Staged but not committed
        self._pending_meta: Dict[str, dict] = {}
    
    # ─────────────────────────────────────────────────────────
    # ASSIGNMENT
    # ─────────────────────────────────────────────────────────
    
    def assign(self, pathway_id: str, ratio: Union[Ratio, Tuple[int, int]], 
               meta: dict = None, force: bool = False) -> bool:
        """
        Assign a ratio to a pathway.
        
        Returns True if successful, False if blocked (unless force=True).
        """
        if isinstance(ratio, tuple):
            ratio = Ratio(ratio[0], ratio[1])
        
        # Check for conflicts
        if ratio.to_tuple() in self.used_ratios and not force:
            existing = [pid for pid, r in self.assignments.items() if r == ratio]
            print(f"  ⚠️ Ratio {ratio} already assigned to {existing[0]}")
            return False
        
        if pathway_id in self.assignments and not force:
            print(f"  ⚠️ Pathway {pathway_id} already has ratio {self.assignments[pathway_id]}")
            return False
        
        # Make assignment
        self.assignments[pathway_id] = ratio
        self.used_ratios.add(ratio.to_tuple())
        
        # Store metadata
        entry = {
            'pathway_id': pathway_id,
            'n': ratio.n,
            'd': ratio.d,
            'ratio': str(ratio),
            'consonance': ratio.consonance(),
            'prime_limit': ratio.prime_limit(),
            'polarity': round(ratio.polarity(), 3),
            'polarity_label': ratio.polarity_label(),
            'cents': round(ratio.cents(), 2),
        }
        if meta:
            entry.update(meta)
            self.metadata[pathway_id] = meta
        self.log.append(entry)
        
        return True
    
    def stage(self, pathway_id: str, ratio: Union[Ratio, Tuple[int, int]], 
              meta: dict = None) -> bool:
        """Stage an assignment for review before committing."""
        if isinstance(ratio, tuple):
            ratio = Ratio(ratio[0], ratio[1])
        
        # Check against both committed and staged
        if ratio.to_tuple() in self.used_ratios:
            return False
        staged_tuples = {r.to_tuple() for r in self._pending.values()}
        if ratio.to_tuple() in staged_tuples:
            return False
        if pathway_id in self.assignments or pathway_id in self._pending:
            return False
        
        self._pending[pathway_id] = ratio
        if meta:
            self._pending_meta[pathway_id] = meta
        return True
    
    def commit_staged(self) -> int:
        """Commit all staged assignments. Returns count committed."""
        count = 0
        for pid, ratio in self._pending.items():
            meta = self._pending_meta.get(pid, {})
            if self.assign(pid, ratio, meta):
                count += 1
        self._pending.clear()
        self._pending_meta.clear()
        return count
    
    def clear_staged(self):
        """Discard all staged assignments."""
        self._pending.clear()
        self._pending_meta.clear()
    
    def unassign(self, pathway_id: str) -> bool:
        """Remove an assignment."""
        if pathway_id not in self.assignments:
            return False
        ratio = self.assignments.pop(pathway_id)
        self.used_ratios.discard(ratio.to_tuple())
        self.metadata.pop(pathway_id, None)
        self.log = [e for e in self.log if e['pathway_id'] != pathway_id]
        return True
    
    # ─────────────────────────────────────────────────────────
    # QUERIES
    # ─────────────────────────────────────────────────────────
    
    def is_ratio_used(self, n: int, d: int) -> bool:
        return (n, d) in self.used_ratios
    
    def is_assigned(self, pathway_id: str) -> bool:
        return pathway_id in self.assignments
    
    def get_ratio(self, pathway_id: str) -> Optional[Ratio]:
        return self.assignments.get(pathway_id)
    
    def get_available(self, ratios: List[Ratio]) -> List[Ratio]:
        """Filter list to only unused ratios."""
        return [r for r in ratios if r.to_tuple() not in self.used_ratios]
    
    def get_unassigned(self, pathway_ids: List[str]) -> List[str]:
        """Return pathway IDs that don't have assignments."""
        return [pid for pid in pathway_ids if pid not in self.assignments]
    
    # ─────────────────────────────────────────────────────────
    # STATUS & ANALYSIS
    # ─────────────────────────────────────────────────────────
    
    def status(self) -> str:
        staged = len(self._pending)
        staged_str = f" (+{staged} staged)" if staged else ""
        return f"{len(self.assignments)} pathways | {len(self.used_ratios)} ratios{staged_str}"
    
    def stats(self) -> dict:
        """Comprehensive statistics."""
        ratios = list(self.assignments.values())
        if not ratios:
            return {'assigned': 0}
        
        consonances = [r.consonance() for r in ratios]
        limits = [r.prime_limit() for r in ratios]
        
        return {
            'assigned': len(ratios),
            'consonance': {
                'min': min(consonances),
                'max': max(consonances),
                'mean': sum(consonances) / len(consonances),
            },
            'prime_limit': {
                'min': min(limits),
                'max': max(limits),
                'distribution': {p: sum(1 for l in limits if l == p) for p in sorted(set(limits))},
            },
            'polarity': {
                'harmonic': sum(1 for r in ratios if r.polarity() > 0.3),
                'subharmonic': sum(1 for r in ratios if r.polarity() < -0.3),
                'unison': sum(1 for r in ratios if abs(r.polarity()) <= 0.3),
            },
        }
    
    def coverage(self, df: pd.DataFrame) -> pd.DataFrame:
        """Show assignment coverage by category."""
        assigned_ids = set(self.assignments.keys())
        
        rows = []
        for cat in sorted(df['category'].unique()):
            cat_ids = set(df[df['category'] == cat]['pathway_id'])
            done = len(cat_ids & assigned_ids)
            total = len(cat_ids)
            rows.append({
                'category': cat,
                'assigned': done,
                'total': total,
                'pct': round(done / total * 100, 1) if total > 0 else 0,
            })
        
        # Total row
        rows.append({
            'category': 'TOTAL',
            'assigned': len(assigned_ids & set(df['pathway_id'])),
            'total': len(df),
            'pct': round(len(assigned_ids & set(df['pathway_id'])) / len(df) * 100, 1),
        })
        
        return pd.DataFrame(rows)
    
    # ─────────────────────────────────────────────────────────
    # DATAFRAME CONVERSION
    # ─────────────────────────────────────────────────────────
    
    def to_df(self) -> pd.DataFrame:
        """Convert assignments to DataFrame."""
        return pd.DataFrame(self.log)
    
    # ─────────────────────────────────────────────────────────
    # SAVE / LOAD
    # ─────────────────────────────────────────────────────────
    
    def save(self, filepath: str = SAVE_PATH):
        """Save to JSON."""
        data = {
            'assignments': {pid: [r.n, r.d] for pid, r in self.assignments.items()},
            'metadata': self.metadata,
            'log': self.log,
        }
        with open(filepath, 'w') as f:
            json.dump(data, f, indent=2)
        print(f"✓ Saved {len(self.assignments)} assignments to {filepath}")
    
    def load(self, filepath: str = SAVE_PATH):
        """Load from JSON (additive)."""
        if not os.path.exists(filepath):
            print(f"  File not found: {filepath}")
            return
        
        with open(filepath, 'r') as f:
            data = json.load(f)
        
        loaded = 0
        for pid, (n, d) in data['assignments'].items():
            if pid not in self.assignments:
                meta = data.get('metadata', {}).get(pid, {})
                if self.assign(pid, Ratio(n, d), meta):
                    loaded += 1
        print(f"✓ Loaded {loaded} assignments from {filepath}")
    
    def clear(self):
        """Reset everything."""
        self.assignments.clear()
        self.metadata.clear()
        self.used_ratios.clear()
        self.log.clear()
        self._pending.clear()
        self._pending_meta.clear()
        print("✓ HarmonicMap cleared")
    
    # ─────────────────────────────────────────────────────────
    # EXPORT
    # ─────────────────────────────────────────────────────────
    
    def export_js(self, var: str = "HARMONIC_MAP") -> str:
        """Export as JavaScript object."""
        lines = [f"const {var} = {{"]
        for pid in sorted(self.assignments.keys()):
            r = self.assignments[pid]
            lines.append(f'  "{pid}": [{r.n}, {r.d}],')
        lines.append("};")
        return "\n".join(lines)
    
    def export_json(self) -> str:
        """Export as JSON string."""
        return json.dumps(
            {pid: [r.n, r.d] for pid, r in self.assignments.items()},
            indent=2
        )
    
    def export_csv(self, filepath: str):
        """Export to CSV."""
        self.to_df().to_csv(filepath, index=False)
        print(f"✓ Exported {len(self.assignments)} assignments to {filepath}")


# Initialize global map
HARMONIC_MAP = HarmonicMap()
print("✓ HARMONIC_MAP initialized")
print(f"  {HARMONIC_MAP.status()}")

✓ HARMONIC_MAP initialized
  0 pathways | 0 ratios


In [7]:
# ============================================================
# 1.7 DISPLAY HELPERS
# ============================================================

def display_ratios(ratios: List[Ratio], title: str = "RATIOS", max_show: int = 30):
    """Display ratio list with analysis."""
    stats = ratio_stats(ratios)
    
    print(f"\n{'═' * 70}")
    print(f" {title}")
    print(f"{'═' * 70}")
    print(f" Count: {stats['count']}  |  Consonance: {stats.get('consonance_range', 'N/A')}  |  Prime limit: {stats.get('prime_limit_range', 'N/A')}")
    print(f" Polarity: {stats.get('harmonic', 0)} harmonic, {stats.get('subharmonic', 0)} subharmonic")
    print(f"{'─' * 70}")
    
    show = ratios[:max_show]
    for r in show:
        print(f"  {str(r):<8} {r.cents():>+8.1f}¢  {r.hz():>7.1f}Hz  n*d={r.consonance():<5} {r.prime_limit()}-limit  {r.polarity()}")
    
    if len(ratios) > max_show:
        print(f"  ... and {len(ratios) - max_show} more")
    print()


def display_pool_preview(pool: List[Ratio], title: str, used: Set[Tuple[int, int]] = None):
    """Show pool with availability highlighting."""
    if used is None:
        used = HARMONIC_MAP.used_ratios
    
    available = [r for r in pool if r.to_tuple() not in used]
    
    print(f"\n{'═' * 70}")
    print(f" {title}")
    print(f" Total: {len(pool)} | Available: {len(available)} | Used: {len(pool) - len(available)}")
    print(f"{'─' * 70}")
    
    for r in pool[:40]:
        status = "✓" if r.to_tuple() not in used else "×"
        print(f"  {status} {str(r):<8} n*d={r.consonance():<5} {r.prime_limit()}-limit")
    
    if len(pool) > 40:
        print(f"  ... and {len(pool) - 40} more")


def display_assignment_table(results: List[dict], title: str, color: str = "#2e7d32"):
    """Rich HTML display for assignments."""
    if not results:
        print(f"[{title}] No assignments")
        return
    
    df = pd.DataFrame(results)
    
    html = f"""
    <div style="border-left:4px solid {color}; padding-left:16px; margin:20px 0;">
        <div style="font-size:14px; font-weight:600; text-transform:uppercase; letter-spacing:1px;">
            {title}
        </div>
        <div style="font-size:12px; color:#666; margin-top:4px;">
            {len(results)} pathways assigned
        </div>
    </div>
    """
    
    display(HTML(html))
    display(df)


def dashboard(hmap: HarmonicMap, df: pd.DataFrame):
    """Show comprehensive status dashboard."""
    stats = hmap.stats()
    coverage = hmap.coverage(df)
    
    print("\n" + "═" * 70)
    print(" 📊 HARMONIC MAP DASHBOARD")
    print("═" * 70)
    
    # Coverage
    print("\n COVERAGE BY CATEGORY")
    print(" ─" * 35)
    for _, row in coverage.iterrows():
        bar_len = int(row['pct'] / 5)
        bar = "█" * bar_len + "░" * (20 - bar_len)
        status = "✓" if row['pct'] == 100 else "○"
        print(f"  {status} {row['category']:<15} {row['assigned']:>3}/{row['total']:<3} {bar} {row['pct']:>5.1f}%")
    
    if stats.get('assigned', 0) > 0:
        # Prime distribution
        print("\n PRIME LIMIT DISTRIBUTION")
        print(" ─" * 35)
        for p, count in sorted(stats['prime_limit']['distribution'].items()):
            pct = count / stats['assigned'] * 100
            bar = "█" * int(pct / 5)
            print(f"  {p:>2}-limit: {count:>4} ({pct:>5.1f}%) {bar}")
        
        # Polarity
        print("\n POLARITY")
        print(" ─" * 35)
        for pol in ['harmonic', 'subharmonic', 'unison']:
            count = stats['polarity'][pol]
            pct = count / stats['assigned'] * 100
            print(f"  {pol:<12}: {count:>4} ({pct:>5.1f}%)")
        
        # Consonance
        print("\n CONSONANCE")
        print(" ─" * 35)
        print(f"  Range: {stats['consonance']['min']} - {stats['consonance']['max']}")
        print(f"  Mean:  {stats['consonance']['mean']:.1f}")
    
    print("\n" + "═" * 70)


print("Display helpers loaded ✓")

Display helpers loaded ✓


## PART 2: PATHWAY DATA 
---
Load curated metabolic pathways, apply filters, create category views.

In [8]:
# ============================================================
# 2.1 LOAD & FILTER PATHWAY DATA
# ============================================================
df_full = pd.read_csv("pathways_reference.csv")
df_full.columns = (
    df_full.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
      .str.replace("%", "_pct")
)

# ─────────────────────────────────────────────────────────────
# CURATED FILTER - Load from pathways_curation_v3.csv
# ─────────────────────────────────────────────────────────────
USE_CURATED = True

if USE_CURATED:
    # Load curation decisions (skip comment lines)
    curation_df = pd.read_csv("pathways_curation_v5.csv", comment='#')
    
    # 1. Get pathways to REMOVE
    remove_ids = curation_df[curation_df['action'] == 'REMOVE']['pathway_id'].tolist()
    
    # 2. Get subcategory REASSIGNMENTS
    reassign_df = curation_df[curation_df['action'] == 'REASSIGN']
    reassign_map = dict(zip(reassign_df['pathway_id'], reassign_df['new_subcategory']))
    
    # 3. Filter out removed pathways
    df = df_full[~df_full['pathway_id'].isin(remove_ids)].copy()
    
    # 4. Apply subcategory reassignments
    df['subcategory'] = df.apply(
        lambda row: reassign_map.get(row['pathway_id'], row['subcategory']),
        axis=1
    )
    
    removed_count = len(df_full) - len(df)
    reassigned_count = len(reassign_map)
else:
    df = df_full.copy()
    removed_count = 0
    reassigned_count = 0

# ─────────────────────────────────────────────────────────────
# CREATE CATEGORY DATAFRAMES
# ─────────────────────────────────────────────────────────────
energy_df = df[df["category"].str.upper() == "ENERGY"].copy()
biosynthesis_df = df[df["category"].str.upper() == "BIOSYNTHESIS"].copy()
degradation_df = df[df["category"].str.upper() == "DEGRADATION"].copy()
salvage_df = df[df["category"].str.upper() == "SALVAGE"].copy()
other_df = df[df["category"].str.upper() == "OTHER"].copy()

# ─────────────────────────────────────────────────────────────
# Summary
# ─────────────────────────────────────────────────────────────
filter_status = f"CURATED ({len(df)})" if USE_CURATED else f"FULL ({len(df)})"

print("=" * 70)
print(f"📊 PATHWAY DATA LOADED  |  {filter_status}")
print("=" * 70)
print(f"""
  Removed:       {removed_count:>3} pathways (non-gut-relevant)
  Reassigned:    {reassigned_count:>3} pathways (new subcategories)
  ───────────────────────────────
  energy:        {len(energy_df):>3} pathways  ({len(energy_df['subcategory'].unique())} subcategories) ✓ CURATED
  biosynthesis:  {len(biosynthesis_df):>3} pathways  ({len(biosynthesis_df['subcategory'].unique())} subcategories) ✓ CURATED
  degradation:   {len(degradation_df):>3} pathways  ({len(degradation_df['subcategory'].unique())} subcategories) ⏳ pending
  salvage:       {len(salvage_df):>3} pathways  ({len(salvage_df['subcategory'].unique())} subcategories) ⏳ pending
  other:         {len(other_df):>3} pathways  ({len(other_df['subcategory'].unique())} subcategories) ⏳ pending
  ───────────────────────────────
  TOTAL:         {len(df):>3} pathways
""")

# Verification
if USE_CURATED:
    print("📋 CURATION PROGRESS:")
    print(f"   Energy:       51 → {len(energy_df)} (removed {51 - len(energy_df)}) ✓")
    print(f"   Biosynthesis: 288 → {len(biosynthesis_df)} (removed {288 - len(biosynthesis_df)}) ✓")
    print(f"   Degradation:  {len(degradation_df)} (pending)")
    print(f"   Salvage:      {len(salvage_df)} (pending)")
    print(f"   Other:        {len(other_df)} (pending)")
    print()
    print("🧬 BIOSYNTHESIS SUBCATEGORIES:")
    for subcat, count in biosynthesis_df.groupby('subcategory').size().sort_values(ascending=False).items():
        print(f"   • {subcat:<25} {count:>3}")


# ============================================================
# 2.2 SUBCATEGORY INVENTORY
# ============================================================
print()
print("=" * 80)
print("📋 SUBCATEGORY INVENTORY")
print("=" * 80)

for cat_name, cat_df, curated in [
    ("ENERGY", energy_df, True), 
    ("BIOSYNTHESIS", biosynthesis_df, True), 
    ("DEGRADATION", degradation_df, False), 
    ("SALVAGE", salvage_df, False), 
    ("OTHER", other_df, False)
]:
    status = "✓" if curated else "⏳"
    print()
    print(f"┌─ {cat_name} ({len(cat_df)} pathways) {status}")
    
    if len(cat_df) == 0:
        print("│ └── (empty)")
        continue
        
    subcat_stats = (
        cat_df.groupby("subcategory")
        .agg(
            count=("pathway_id", "size"),
            prev_mean=("prevalence", "mean"),
        )
        .sort_values("prev_mean", ascending=False)
    )
    
    for i, (subcat, row) in enumerate(subcat_stats.iterrows()):
        prefix = "└──" if i == len(subcat_stats) - 1 else "├──"
        print(f"│ {prefix} {subcat:<35} {int(row['count']):>3}  (prev: {row['prev_mean']:.4f})")

print()
print("=" * 80)

📊 PATHWAY DATA LOADED  |  CURATED (360)

  Removed:       237 pathways (non-gut-relevant)
  Reassigned:     67 pathways (new subcategories)
  ───────────────────────────────
  energy:         27 pathways  (6 subcategories) ✓ CURATED
  biosynthesis:  143 pathways  (9 subcategories) ✓ CURATED
  degradation:   115 pathways  (5 subcategories) ⏳ pending
  salvage:        15 pathways  (1 subcategories) ⏳ pending
  other:          50 pathways  (1 subcategories) ⏳ pending
  ───────────────────────────────
  TOTAL:         360 pathways

📋 CURATION PROGRESS:
   Energy:       51 → 27 (removed 24) ✓
   Biosynthesis: 288 → 143 (removed 145) ✓
   Degradation:  115 (pending)
   Salvage:      15 (pending)
   Other:        50 (pending)

🧬 BIOSYNTHESIS SUBCATEGORIES:
   • Amino Acids                28
   • Cofactors/Vitamins         28
   • Cell Envelope              24
   • Fatty Acids/Lipids         17
   • Specialized                14
   • Other                      12
   • Isoprenoids              

In [9]:
# ============================================================
# 2.2 SUBCATEGORY INVENTORY
# ============================================================
# Overview of all subcategories to plan ratio allocation

print("=" * 80)
print("📋 SUBCATEGORY INVENTORY")
print("=" * 80)

for cat_name, cat_df in [("ENERGY", energy_df), ("BIOSYNTHESIS", biosynthesis_df), 
                          ("DEGRADATION", degradation_df), ("SALVAGE", salvage_df), 
                          ("OTHER", other_df)]:
    print()
    print(f"┌─ {cat_name} ({len(cat_df)} pathways)")
    
    subcat_stats = (
        cat_df.groupby("subcategory")
        .agg(
            count=("pathway_id", "size"),
            prev_mean=("prevalence", "mean"),
            prev_max=("prevalence", "max"),
            abu_mean=("mean_abundance", "mean"),
        )
        .sort_values("prev_mean", ascending=False)
    )
    
    for i, (subcat, row) in enumerate(subcat_stats.iterrows()):
        prefix = "└──" if i == len(subcat_stats) - 1 else "├──"
        print(f"│ {prefix} {subcat:<40} {row['count']:>3}  (prev: {row['prev_mean']:.4f})")

print()
print("=" * 80)

📋 SUBCATEGORY INVENTORY

┌─ ENERGY (27 pathways)
│ ├── Glycolysis/Gluconeogenesis               4.0  (prev: 0.8087)
│ ├── TCA Cycle                                2.0  (prev: 0.7458)
│ ├── Pentose Phosphate                        3.0  (prev: 0.6673)
│ ├── Fermentation                             15.0  (prev: 0.6556)
│ ├── Respiration                              1.0  (prev: 0.5079)
│ └── Glyoxylate Cycle                         2.0  (prev: 0.4534)

┌─ BIOSYNTHESIS (143 pathways)
│ ├── Nucleotides                              7.0  (prev: 0.9981)
│ ├── Amino Acids                              28.0  (prev: 0.9444)
│ ├── Fatty Acids/Lipids                       17.0  (prev: 0.8421)
│ ├── Cofactors/Vitamins                       28.0  (prev: 0.7104)
│ ├── Cell Envelope                            24.0  (prev: 0.6924)
│ ├── Isoprenoids                              7.0  (prev: 0.6732)
│ ├── Polyamines                               6.0  (prev: 0.5017)
│ ├── Other                                

In [10]:
# ============================================================
# 2.3 LOAD EXISTING PROGRESS (if any)
# ============================================================
# Run this at the start of each session to restore previous work

import os

if os.path.exists(SAVE_PATH):
    HARMONIC_MAP.load(SAVE_PATH)
    print()
    dashboard(HARMONIC_MAP, df)
else:
    print("No saved progress found. Starting fresh.")
    print(f"  {HARMONIC_MAP.status()}")

No saved progress found. Starting fresh.
  0 pathways | 0 ratios


## PART 3: COMPOSITION
---
Each section generates a ratio pool suited to its metabolic character, then assigns ratios
to pathways based on prevalence/abundance. Higher prevalence = more consonant ratio.

**Workflow per section:**
1. Generate pool (prime limit, polarity, consonance range)
2. Review pathways to assign
3. Run assignment algorithm
4. Review & adjust if needed
5. Save progress

---

### ENERGY METABOLISM

**Character:** Core metabolism, foundational, highly prevalent → **most consonant ratios**

**Prime limits:** Primarily 2-limit and 3-limit (pure octaves, fifths)

**Polarity:** Mostly harmonic (above 1/1) — energy production is fundamentally generative

| Subcategory | Count | Prime Limit | Polarity | Notes |
|-------------|-------|-------------|----------|-------|
| Glycolysis/Gluconeogenesis | 4 | 3 | harmonic | Glucose processing |
| TCA Cycle | 2 | 2-3 | harmonic | Core energy cycle |
| Pentose Phosphate | 3 | 3 | harmonic | Nucleotide precursors |
| Glyoxylate | 2 | 3 | harmonic | Acetate assimilation |
| Respiration | 1 | 3 | harmonic | Aerobic metabolism |
| Fermentation | 15 | 3-5 | harmonic | SCFA production |

#### Glycolysis

In [11]:
# ============================================================
# 3.1 ENERGY — Glycolysis
# Exponential 2-limit growth (1 → 2 → 4 → 16)
# ============================================================

SUBCATEGORY = "Glycolysis"

# ----------------------------------------------------------------
#   1/1 | 2/1 | 2²/1 = 4/1 | 4²/1 = 16/1
# ----------------------------------------------------------------

glycolysis_ratios = [
    Ratio(1, 1),
    Ratio(2, 1),
    Ratio(4, 1),
    Ratio(16, 1),
]

glycolysis_pathways = [
    "PWY-1042",         # Glycolysis IV
    "GLUCONEO-PWY",     # Gluconeogenesis I
    "GLYCOLYSIS-E-D",   # Superpathway: glycolysis + ED
    "PWY-7446",         # Sulfoglycolysis
]

print("Glycolysis ratio derivation:")
for r in glycolysis_ratios:
    print(f"  {r}  polarity={r.polarity():+.2f}")

# ----------------------------------------------------------------
# Clean up any existing assignments for these pathways
# ----------------------------------------------------------------
for pid in glycolysis_pathways:
    if HARMONIC_MAP.is_assigned(pid):
        HARMONIC_MAP.unassign(pid)

# ----------------------------------------------------------------
# Assign deterministically
# ----------------------------------------------------------------
print("\nAssigning glycolysis pathways:")

for pid, ratio in zip(glycolysis_pathways, glycolysis_ratios):
    row = energy_df.loc[energy_df["pathway_id"] == pid].iloc[0]

    assigned = HARMONIC_MAP.assign(
        pid,
        ratio,
        {
            "pathway_name": row["pathway_name"],
            "subcategory": row["subcategory"],
            "category": "energy",
            "derivation": "exponential 2-limit growth (1→2→4→16)",
            "prevalence": row["prevalence"],
        }
    )

    status = "✓" if assigned else "○"
    print(f"{status} {pid:<20} → {ratio}")

print(f"\nMap status: {HARMONIC_MAP.status()}")

HARMONIC_MAP.save()

Glycolysis ratio derivation:
  1/1  polarity=+0.00
  2/1  polarity=+1.00
  4/1  polarity=+1.00
  16/1  polarity=+1.00

Assigning glycolysis pathways:
✓ PWY-1042             → 1/1
✓ GLUCONEO-PWY         → 2/1
✓ GLYCOLYSIS-E-D       → 4/1
✓ PWY-7446             → 16/1

Map status: 4 pathways | 4 ratios
✓ Saved 4 assignments to harmonic_map.json


#### TCA Cycle

In [12]:
# ============================================================
# 3.2 ENERGY — TCA Cycle
# Prime-3 energy source with cyclic modulation
# ============================================================

SUBCATEGORY = "TCA Cycle"

# ----------------------------------------------------------------
# Harmonic derivation:
#
# 1/1 and 2/1 are used by glycolysis.
# The next most consonant available ratio introduces prime 3:
#
#   3/1  (n×d = 3)  → new energy source
#
# Cyclic behavior is modeled by its octave-reduced form:
#
#   3/2  (n×d = 6)  → rotational modulation
#
# Assignment order follows increasing consonance.
# ----------------------------------------------------------------

tca_ratios = [
    Ratio(3, 1),   # prime-3 energy source
    Ratio(6, 1),   # cyclic modulation (perfect fifth)
]

print("TCA harmonic derivation (ordered by consonance):")
for r in tca_ratios:
    print(
        f"  {r:<5} "
        f"n×d={r.consonance():<3} "
        f"polarity={r.polarity():+.2f}"
    )

# ----------------------------------------------------------------
# Select TCA pathways
# ----------------------------------------------------------------
tca_df = (
    energy_df[
        energy_df["subcategory"]
        .str.contains("TCA", case=False, na=False)
    ]
    .sort_values("prevalence", ascending=False)
    .copy()
)

if len(tca_df) == 0:
    print("⚠ No TCA pathways found")
else:
    # Clean any existing assignments for TCA pathways
    for pid in tca_df["pathway_id"].tolist():
        if HARMONIC_MAP.is_assigned(pid):
            HARMONIC_MAP.unassign(pid)

    print("\nAssigning TCA pathways by consonance:")

    for (_, row), ratio in zip(tca_df.iterrows(), tca_ratios):
        assigned = HARMONIC_MAP.assign(
            row["pathway_id"],
            ratio,
            {
                "pathway_name": row["pathway_name"],
                "subcategory": row["subcategory"],
                "category": "energy",
                "derivation": "prime-3 introduction (3/1 → 3/2)",
                "prevalence": row["prevalence"],
            }
        )

        status = "✓" if assigned else "○"
        print(f"{status} {row['pathway_id']:<25} → {ratio}")

print(f"\nMap status: {HARMONIC_MAP.status()}")

HARMONIC_MAP.save()

TCA harmonic derivation (ordered by consonance):
  3/1   n×d=3   polarity=+1.00
  6/1   n×d=6   polarity=+1.00

Assigning TCA pathways by consonance:
✓ TCA                       → 3/1
✓ TCA-GLYOX-BYPASS          → 6/1

Map status: 6 pathways | 6 ratios
✓ Saved 6 assignments to harmonic_map.json


#### Pentose Phosphate

In [13]:
# ============================================================
# 3.3 ENERGY: PENTOSE PHOSPHATE PATHWAY
# ============================================================
# 3 pathways - nucleotide precursors, NADPH production
# 3-limit, harmonic

SECTION = "Pentose Phosphate"
section_df = energy_df[energy_df["subcategory"].str.contains("Pentose", case=False, na=False)].copy()
section_df = section_df.sort_values("prevalence", ascending=False)

print(f"{'═' * 70}")
print(f" {SECTION.upper()} ({len(section_df)} pathways)")
print(f"{'═' * 70}")

unassigned = HARMONIC_MAP.get_unassigned(section_df["pathway_id"].tolist())
print(f"\n Unassigned: {len(unassigned)} of {len(section_df)}")
print(f"{'─' * 70}")
for _, row in section_df.iterrows():
    status = "○" if row["pathway_id"] in unassigned else "✓"
    existing = HARMONIC_MAP.get_ratio(row["pathway_id"])
    ratio_str = str(existing) if existing else "—"
    print(f" {status} {row['prevalence']:>7.4f}  {row['pathway_id']:<30} {ratio_str}")

══════════════════════════════════════════════════════════════════════
 PENTOSE PHOSPHATE (3 pathways)
══════════════════════════════════════════════════════════════════════

 Unassigned: 3 of 3
──────────────────────────────────────────────────────────────────────
 ○  0.9990  NONOXIPENT-PWY                 —
 ○  0.9846  PENTOSE-P-PWY                  —
 ○  0.0183  PWY-2221                       —


In [14]:
# ============================================================
# 3.3 ENERGY — Pentose Phosphate Pathway
# Prime-5 introduction (biosynthetic primer)
# ============================================================

SUBCATEGORY = "Pentose Phosphate"


ppp_ratios = [
    Ratio(9, 1),
    Ratio(9, 2),
    Ratio(9, 4),
]

print("Pentose phosphate harmonic derivation:")
for r in ppp_ratios:
    print(
        f"  {r:<5} "
        f"n×d={r.consonance():<3} "
        f"polarity={r.polarity():+.2f}"
    )

# ----------------------------------------------------------------
# Select PPP pathways
# ----------------------------------------------------------------
ppp_df = (
    energy_df[
        energy_df["subcategory"]
        .str.contains("Pentose", case=False, na=False)
    ]
    .sort_values("prevalence", ascending=False)
    .copy()
)

if len(ppp_df) == 0:
    print("⚠ No pentose phosphate pathways found")
else:
    # Clean any existing assignments
    for pid in ppp_df["pathway_id"].tolist():
        if HARMONIC_MAP.is_assigned(pid):
            HARMONIC_MAP.unassign(pid)

    print("\nAssigning pentose phosphate pathways:")

    for (_, row), ratio in zip(ppp_df.iterrows(), ppp_ratios):
        assigned = HARMONIC_MAP.assign(
            row["pathway_id"],
            ratio,
            {
                "pathway_name": row["pathway_name"],
                "subcategory": row["subcategory"],
                "category": "energy",
                "derivation": "prime-5 biosynthetic primer (5/1 → 5/2 → 5/3)",
                "prevalence": row["prevalence"],
            }
        )

        status = "✓" if assigned else "○"
        print(f"{status} {row['pathway_id']:<25} → {ratio}")

print(f"\nMap status: {HARMONIC_MAP.status()}")

HARMONIC_MAP.save()

Pentose phosphate harmonic derivation:
  9/1   n×d=9   polarity=+1.00
  9/2   n×d=18  polarity=+0.71
  9/4   n×d=36  polarity=+0.58

Assigning pentose phosphate pathways:
✓ NONOXIPENT-PWY            → 9/1
✓ PENTOSE-P-PWY             → 9/2
✓ PWY-2221                  → 9/4

Map status: 9 pathways | 9 ratios
✓ Saved 9 assignments to harmonic_map.json


#### Glyoxylate Cycle

In [15]:
# ============================================================
# 3.5 ENERGY: Glyoxylate Cycle
# ============================================================

glyoxylate_ratios = [
    Ratio(8, 1),    # glycolysis-adjacent reference
    Ratio(81, 8),   # stressed / rare transformation
]

glyoxylate_pathways = [
    "GLYOXYLATE-BYPASS",  # canonical glyoxylate cycle
    "PWY-5705",
]

print("Glyoxylate harmonic derivation:")
for r in glyoxylate_ratios:
    print(
        f"  {r:<6} "
        f"n×d={r.consonance():<4} "
        f"polarity={r.polarity():+.2f}"
    )

# Clean reassignment
for pid in glyoxylate_pathways:
    if HARMONIC_MAP.is_assigned(pid):
        HARMONIC_MAP.unassign(pid)

print("\nAssigning glyoxylate pathways:")
for pid, ratio in zip(glyoxylate_pathways, glyoxylate_ratios):
    row = energy_df.loc[energy_df["pathway_id"] == pid].iloc[0]

    HARMONIC_MAP.assign(
        pid,
        ratio,
        {
            "pathway_name": row["pathway_name"],
            "subcategory": row["subcategory"],
            "category": "energy",
            "derivation": "glycolysis-derived stress harmonic (8/1 → 81/8)",
            "prevalence": row["prevalence"],
        }
    )

    print(f"  ✓ {pid:<22} → {ratio}")

print(f"\nMap status: {HARMONIC_MAP.status()}")
HARMONIC_MAP.save()

Glyoxylate harmonic derivation:
  8/1    n×d=8    polarity=+1.00
  81/8   n×d=648  polarity=+0.50

Assigning glyoxylate pathways:
  ✓ GLYOXYLATE-BYPASS      → 8/1
  ✓ PWY-5705               → 81/8

Map status: 11 pathways | 11 ratios
✓ Saved 11 assignments to harmonic_map.json


#### Respiration

In [16]:
# ============================================================
# ENERGY — Respiration
# 3-limit amplification of existing energy
# ============================================================

respiration_ratio = Ratio(12, 1)

print(
    f"Respiration ratio: {respiration_ratio} "
    f"n×d={respiration_ratio.consonance()} "
    f"polarity={respiration_ratio.polarity():+.2f}"
)

resp_df = (
    energy_df[
        energy_df["subcategory"]
        .str.contains("Respiration", case=False, na=False)
    ]
    .sort_values("prevalence", ascending=False)
)

for pid in resp_df["pathway_id"].tolist():
    if HARMONIC_MAP.is_assigned(pid):
        HARMONIC_MAP.unassign(pid)

for (_, row) in resp_df.iterrows():
    HARMONIC_MAP.assign(
        row["pathway_id"],
        respiration_ratio,
        {
            "pathway_name": row["pathway_name"],
            "subcategory": row["subcategory"],
            "category": "energy",
            "derivation": "3-limit amplification (12/1)",
            "prevalence": row["prevalence"],
        }
    )
    print(f"  ✓ {row['pathway_id']} → {respiration_ratio}")

HARMONIC_MAP.save()

Respiration ratio: 12/1 n×d=12 polarity=+1.00
  ✓ PWY-3781 → 12/1
✓ Saved 12 assignments to harmonic_map.json


#### Fermentation

In [17]:
# ============================================================
# ENERGY — Fermentation pathways (comprehensive audit)
# Safe, schema-aware, read-only
# ============================================================

import pandas as pd

# ------------------------------------------------------------
# 1. Extract fermentation pathways
# ------------------------------------------------------------
fermentation_df = (
    energy_df[
        energy_df["subcategory"]
        .astype(str)
        .str.contains("Fermentation", case=False, na=False)
    ]
    .sort_values("prevalence", ascending=False)
    .copy()
)

print(f"\nFermentation pathways found: {len(fermentation_df)}")

# ------------------------------------------------------------
# 2. Assignment status
# ------------------------------------------------------------
fermentation_df["assigned"] = fermentation_df["pathway_id"].apply(
    lambda pid: HARMONIC_MAP.is_assigned(pid)
)

assigned_ct = fermentation_df["assigned"].sum()
unassigned_ct = len(fermentation_df) - assigned_ct

print(f"Assigned: {assigned_ct} | Unassigned: {unassigned_ct}")

# ------------------------------------------------------------
# 3. Infer fermentation type from pathway name
# ------------------------------------------------------------
FERMENTATION_REGEX = (
    r"(butyrate|butanoate|lactate|acetate|ethanol|propanoate|"
    r"succinate|formate|isobutanol|isopropanol|mixed[- ]acid)"
)

fermentation_df["fermentation_type"] = (
    fermentation_df["pathway_name"]
    .astype(str)
    .str.extract(FERMENTATION_REGEX, expand=False)
    .str.lower()
    .fillna("other")
)

# ------------------------------------------------------------
# 4. Summary by fermentation type
# ------------------------------------------------------------
print("\nFermentation type summary:")
display(
    fermentation_df
    .groupby("fermentation_type")
    .agg(
        pathways=("pathway_id", "count"),
        mean_prevalence=("prevalence", "mean"),
        max_prevalence=("prevalence", "max"),
    )
    .sort_values("pathways", ascending=False)
)

# ------------------------------------------------------------
# 5. Build display columns dynamically (schema-safe)
# ------------------------------------------------------------
base_cols = [
    "pathway_id",
    "pathway_name",
    "fermentation_type",
    "prevalence",
    "mean_abundance",
    "tier",
    "assigned",
]

optional_cols = [
    "ms_direction",
    "ms_lda",
]

display_cols = [
    c for c in base_cols + optional_cols
    if c in fermentation_df.columns
]

# ------------------------------------------------------------
# 6. Full fermentation table (inspection view)
# ------------------------------------------------------------
print("\nFull fermentation pathway table:")
display(
    fermentation_df[display_cols]
    .reset_index(drop=True)
)

# ------------------------------------------------------------
# 7. Unassigned fermentation pathways only
# ------------------------------------------------------------
if unassigned_ct > 0:
    print("\nUnassigned fermentation pathways (sorted by prevalence):")
    display(
        fermentation_df[
            ~fermentation_df["assigned"]
        ][display_cols]
        .sort_values("prevalence", ascending=False)
        .reset_index(drop=True)
    )
else:
    print("\n✓ All fermentation pathways are assigned")


Fermentation pathways found: 15
Assigned: 0 | Unassigned: 15

Fermentation type summary:


,pathways,mean_prevalence,max_prevalence
fermentation_type,,,
other,6,0.553032,0.996917
butanoate,2,0.968551,0.976156
acetate,2,0.582425,0.998356
propanoate,2,0.620863,0.960329
lactate,1,0.968345,0.968345
mixed acid,1,0.967112,0.967112
succinate,1,0.235971,0.235971



Full fermentation pathway table:


,pathway_id,pathway_name,fermentation_type,prevalence,mean_abundance,tier,assigned
0,PWY-5100,pyruvate fermentation to acetate and lactate II,acetate,0.998356,2.261310e-04,UNIVERSAL,False
1,ANAEROFRUCAT-PWY,homolactic fermentation,other,0.996917,2.585110e-04,UNIVERSAL,False
2,PWY-5676,acetyl-CoA fermentation to butanoate II,butanoate,0.976156,4.620000e-05,VERY_COMMON,False
3,P461-PWY,"hexitol fermentation to lactate, formate, etha...",lactate,0.968345,4.250000e-05,VERY_COMMON,False
4,FERMENTATION-PWY,mixed acid fermentation,mixed acid,0.967112,6.690000e-05,VERY_COMMON,False
5,PWY-6590,superpathway of Clostridium acetobutylicum aci...,other,0.960946,3.960000e-05,VERY_COMMON,False
6,CENTFERM-PWY,pyruvate fermentation to butanoate,butanoate,0.960946,3.220000e-05,VERY_COMMON,False
7,P108-PWY,pyruvate fermentation to propanoate I,propanoate,0.960329,3.030000e-05,VERY_COMMON,False
8,PWY-6588,pyruvate fermentation to acetone,other,0.894553,1.120000e-05,MODERATE,False
9,P122-PWY,heterolactic fermentation,other,0.464954,4.330000e-06,UNCOMMON,False



Unassigned fermentation pathways (sorted by prevalence):


,pathway_id,pathway_name,fermentation_type,prevalence,mean_abundance,tier,assigned
0,PWY-5100,pyruvate fermentation to acetate and lactate II,acetate,0.998356,2.261310e-04,UNIVERSAL,False
1,ANAEROFRUCAT-PWY,homolactic fermentation,other,0.996917,2.585110e-04,UNIVERSAL,False
2,PWY-5676,acetyl-CoA fermentation to butanoate II,butanoate,0.976156,4.620000e-05,VERY_COMMON,False
3,P461-PWY,"hexitol fermentation to lactate, formate, etha...",lactate,0.968345,4.250000e-05,VERY_COMMON,False
4,FERMENTATION-PWY,mixed acid fermentation,mixed acid,0.967112,6.690000e-05,VERY_COMMON,False
5,PWY-6590,superpathway of Clostridium acetobutylicum aci...,other,0.960946,3.960000e-05,VERY_COMMON,False
6,CENTFERM-PWY,pyruvate fermentation to butanoate,butanoate,0.960946,3.220000e-05,VERY_COMMON,False
7,P108-PWY,pyruvate fermentation to propanoate I,propanoate,0.960329,3.030000e-05,VERY_COMMON,False
8,PWY-6588,pyruvate fermentation to acetone,other,0.894553,1.120000e-05,MODERATE,False
9,P122-PWY,heterolactic fermentation,other,0.464954,4.330000e-06,UNCOMMON,False


In [18]:
# ============================================================
# ENERGY — Fermentation
# Structured 3-limit assignment by biological function
# ============================================================
SECTION = "Fermentation"

# ------------------------------------------------------------
# 1. Define the composed mapping
# ------------------------------------------------------------
# Ratios organized by functional family:
#
# 3/2        → Butyrate (canonical) - THE goal
# /3 family  → Lactate/Acetate feeders (build toward butyrate)
# 27/ family → Mixed acid, Propionate (essential SCFAs)
# /9 family  → Complex fermentation (Clostridium, rare routes)
# 32/27      → Rare solvents

FERMENTATION_MAPPING = {
    # BUTYRATE - The Goal (3/2)
    'CENTFERM-PWY':     (3, 2),     # pyruvate → butyrate (canonical)
    
    # LACTATE/ACETATE FEEDERS (/3 family - leads to butyrate)
    'PWY-5100':         (4, 3),     # pyruvate → acetate + lactate (most prevalent)
    'ANAEROFRUCAT-PWY': (8, 3),     # homolactic (pure lactate)
    'PWY-5676':         (16, 3),    # acetyl-CoA → butyrate (cross-feeding)
    'P461-PWY':         (32, 3),    # hexitol fermentation
    
    # MIXED ACID & PROPIONATE (27/ family - essential SCFAs)
    'FERMENTATION-PWY': (27, 2),    # mixed acid (Enterobacteriaceae)
    'P108-PWY':         (27, 4),    # pyruvate → propionate (canonical)
    'PWY-5677':         (27, 8),    # succinate → butyrate
    'PROPFERM-PWY':     (27, 16),   # alanine → propionate
    
    # COMPLEX FERMENTATION (/9 family)
    'PWY-6590':         (9, 8),     # Clostridium acidogenic
    'PWY-6588':         (16, 9),    # pyruvate → acetone
    'P122-PWY':         (32, 9),    # heterolactic
    'P163-PWY':         (64, 9),    # lysine → acetate + butyrate
    'PWY-6863':         (128, 9),   # pyruvate → hexanol (rare, has MS data)
    
    # RARE SOLVENTS (Pythagorean comma zone)
    'PWY-6883':         (32, 27),   # pyruvate → butanol
}

# Derivation notes for documentation
FERMENTATION_DERIVATIONS = {
    'CENTFERM-PWY':     'butyrate canonical - THE healthy gut interval (3/2)',
    'PWY-5100':         '/3 family - acetate+lactate feeds cross-feeders',
    'ANAEROFRUCAT-PWY': '/3 family - pure lactate production',
    'PWY-5676':         '/3 family - cross-feeding route to butyrate',
    'P461-PWY':         '/3 family - hexitol substrate',
    'FERMENTATION-PWY': '27/ family - mixed acid (high consonance for importance)',
    'P108-PWY':         '27/ family - propionate canonical',
    'PWY-5677':         '27/ family - succinate to butyrate',
    'PROPFERM-PWY':     '27/ family - alanine to propionate',
    'PWY-6590':         '/9 family - Clostridium complex fermentation',
    'PWY-6588':         '/9 family - acetone (solvent)',
    'P122-PWY':         '/9 family - heterolactic',
    'P163-PWY':         '/9 family - lysine to butyrate',
    'PWY-6863':         '/9 family - rare hexanol (has MS data)',
    'PWY-6883':         'Pythagorean comma zone - rare butanol',
}

# ------------------------------------------------------------
# 2. Select fermentation pathways from dataframe
# ------------------------------------------------------------
fermentation_df = (
    energy_df[
        energy_df["subcategory"]
        .astype(str)
        .str.contains("Fermentation", case=False, na=False)
    ]
    .copy()
)

if len(fermentation_df) == 0:
    print("⚠ No fermentation pathways found")
else:
    print(f"Fermentation pathways: {len(fermentation_df)}")

# ------------------------------------------------------------
# 3. Get unassigned fermentation pathways
# ------------------------------------------------------------
unassigned = HARMONIC_MAP.get_unassigned(
    fermentation_df["pathway_id"].tolist()
)
fermentation_df = fermentation_df[
    fermentation_df["pathway_id"].isin(unassigned)
]
print(f"Unassigned fermentation pathways: {len(fermentation_df)}")

# ------------------------------------------------------------
# 4. Assign from composed mapping
# ------------------------------------------------------------
assigned_count = 0
skipped = []

for pathway_id, (n, d) in FERMENTATION_MAPPING.items():
    # Check if pathway exists in our data
    row_match = fermentation_df[fermentation_df["pathway_id"] == pathway_id]
    
    if len(row_match) == 0:
        # Pathway might already be assigned or not in dataset
        if not HARMONIC_MAP.is_assigned(pathway_id):
            skipped.append(pathway_id)
        continue
    
    row = row_match.iloc[0]
    ratio = Ratio(n, d)
    
    # Check if ratio is available
    if HARMONIC_MAP.is_ratio_used(n, d):
        print(f"⚠ Ratio {n}/{d} already used, skipping {pathway_id}")
        continue
    
    derivation = FERMENTATION_DERIVATIONS.get(pathway_id, "3-limit composed assignment")
    
    if HARMONIC_MAP.assign(
        pathway_id,
        ratio,
        {
            "pathway_name": row["pathway_name"],
            "subcategory": row["subcategory"],
            "category": "energy",
            "prevalence": row["prevalence"],
            "derivation": derivation,
        }
    ):
        assigned_count += 1
        print(
            f"✓ {pathway_id:<22} "
            f"→ {str(ratio):<8} "
            f"n×d={ratio.consonance():<5} "
            f"| {derivation[:45]}"
        )

# ------------------------------------------------------------
# 5. Report results
# ------------------------------------------------------------
print(f"\n{'='*60}")
print(f"Assigned: {assigned_count} fermentation pathways")
if skipped:
    print(f"Skipped (not in dataset): {skipped}")
print(f"Map status: {HARMONIC_MAP.status()}")

# ------------------------------------------------------------
# 6. Display family structure
# ------------------------------------------------------------
print(f"\n{'='*60}")
print("FERMENTATION HARMONIC FAMILIES:")
print("="*60)

families = {
    "3/2 (Butyrate Goal)": ['CENTFERM-PWY'],
    "/3 (Lactate → Butyrate)": ['PWY-5100', 'ANAEROFRUCAT-PWY', 'PWY-5676', 'P461-PWY'],
    "27/ (Mixed + Propionate)": ['FERMENTATION-PWY', 'P108-PWY', 'PWY-5677', 'PROPFERM-PWY'],
    "/9 (Complex)": ['PWY-6590', 'PWY-6588', 'P122-PWY', 'P163-PWY', 'PWY-6863'],
    "32/27 (Rare)": ['PWY-6883'],
}

for family_name, pathway_ids in families.items():
    print(f"\n{family_name}:")
    for pid in pathway_ids:
        if pid in FERMENTATION_MAPPING:
            n, d = FERMENTATION_MAPPING[pid]
            r = Ratio(n, d)
            print(f"  {n}/{d:<6} n×d={n*d:<5} {pid}")

# ------------------------------------------------------------
# 7. Save progress
# ------------------------------------------------------------
HARMONIC_MAP.save()
print(f"\n✓ Saved!")
print(f"\nFinal stats:")
print(HARMONIC_MAP.stats())

Fermentation pathways: 15
Unassigned fermentation pathways: 15
✓ CENTFERM-PWY           → 3/2      n×d=6     | butyrate canonical - THE healthy gut interval
✓ PWY-5100               → 4/3      n×d=12    | /3 family - acetate+lactate feeds cross-feede
✓ ANAEROFRUCAT-PWY       → 8/3      n×d=24    | /3 family - pure lactate production
✓ PWY-5676               → 16/3     n×d=48    | /3 family - cross-feeding route to butyrate
✓ P461-PWY               → 32/3     n×d=96    | /3 family - hexitol substrate
✓ FERMENTATION-PWY       → 27/2     n×d=54    | 27/ family - mixed acid (high consonance for 
✓ P108-PWY               → 27/4     n×d=108   | 27/ family - propionate canonical
✓ PWY-5677               → 27/8     n×d=216   | 27/ family - succinate to butyrate
✓ PROPFERM-PWY           → 27/16    n×d=432   | 27/ family - alanine to propionate
✓ PWY-6590               → 9/8      n×d=72    | /9 family - Clostridium complex fermentation
✓ PWY-6588               → 16/9     n×d=144   | /9 family - 

#### Energy Summary

In [19]:
# ============================================================
# 3.7 ENERGY: CHECKPOINT
# ============================================================
# Verify all Energy pathways are assigned before moving on

print("═" * 70)
print(" ⚡ ENERGY CATEGORY CHECKPOINT")
print("═" * 70)

energy_ids = set(energy_df["pathway_id"])
assigned_energy = {pid for pid in HARMONIC_MAP.assignments.keys() if pid in energy_ids}
unassigned_energy = energy_ids - assigned_energy

print(f"\n Assigned:   {len(assigned_energy)} / {len(energy_ids)}")
print(f" Unassigned: {len(unassigned_energy)}")

if unassigned_energy:
    print(f"\n ⚠️ Missing assignments:")
    for pid in unassigned_energy:
        row = energy_df[energy_df["pathway_id"] == pid].iloc[0]
        print(f"   {row['subcategory']:<30} {pid}")
else:
    print("\n ✓ All Energy pathways assigned!")

# Show Energy assignments summary
print(f"\n{'─' * 70}")
print(" ENERGY ASSIGNMENTS")
print(f"{'─' * 70}")
energy_assignments = [(pid, r) for pid, r in HARMONIC_MAP.assignments.items() if pid in energy_ids]
energy_assignments.sort(key=lambda x: x[1].consonance())

for pid, ratio in energy_assignments:
    row = energy_df[energy_df["pathway_id"] == pid].iloc[0]
    print(f"  {str(ratio):<8} n*d={ratio.consonance():<5} {row['subcategory']:<25} {pid}")

══════════════════════════════════════════════════════════════════════
 ⚡ ENERGY CATEGORY CHECKPOINT
══════════════════════════════════════════════════════════════════════

 Assigned:   27 / 27
 Unassigned: 0

 ✓ All Energy pathways assigned!

──────────────────────────────────────────────────────────────────────
 ENERGY ASSIGNMENTS
──────────────────────────────────────────────────────────────────────
  1/1      n*d=1     Glycolysis/Gluconeogenesis PWY-1042
  2/1      n*d=2     Glycolysis/Gluconeogenesis GLUCONEO-PWY
  3/1      n*d=3     TCA Cycle                 TCA
  4/1      n*d=4     Glycolysis/Gluconeogenesis GLYCOLYSIS-E-D
  6/1      n*d=6     TCA Cycle                 TCA-GLYOX-BYPASS
  3/2      n*d=6     Fermentation              CENTFERM-PWY
  8/1      n*d=8     Glyoxylate Cycle          GLYOXYLATE-BYPASS
  9/1      n*d=9     Pentose Phosphate         NONOXIPENT-PWY
  12/1     n*d=12    Respiration               PWY-3781
  4/3      n*d=12    Fermentation              PWY-5100

In [20]:
# ════════════════════════════════════════════════════════════
# SAVE PROGRESS
# ════════════════════════════════════════════════════════════

HARMONIC_MAP.save()
print()
dashboard(HARMONIC_MAP, df)

✓ Saved 27 assignments to harmonic_map.json


══════════════════════════════════════════════════════════════════════
 📊 HARMONIC MAP DASHBOARD
══════════════════════════════════════════════════════════════════════

 COVERAGE BY CATEGORY
 ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─
  ○ biosynthesis      0/143 ░░░░░░░░░░░░░░░░░░░░   0.0%
  ○ degradation       0/115 ░░░░░░░░░░░░░░░░░░░░   0.0%
  ✓ energy           27/27  ████████████████████ 100.0%
  ○ other             0/50  ░░░░░░░░░░░░░░░░░░░░   0.0%
  ○ salvage           0/15  ░░░░░░░░░░░░░░░░░░░░   0.0%
  ○ superpathways     0/10  ░░░░░░░░░░░░░░░░░░░░   0.0%
  ○ TOTAL            27/360 █░░░░░░░░░░░░░░░░░░░   7.5%

 PRIME LIMIT DISTRIBUTION
 ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─
   1-limit:    1 (  3.7%) 
   2-limit:    4 ( 14.8%) ██
   3-limit:   22 ( 81.5%) ████████████████

 POLARITY
 ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─
  harmonic    :   17 

---
### BIOSYNTHESIS

**Character:** Anabolic, building molecules → **harmonic ratios** (n > d)

**Prime limits:** 5-limit through 11-limit (increasing complexity for specialized pathways)

**Key principle:** Biosynthesis pathways will be **paired** with their corresponding
degradation pathways using **inverse ratios**:
- Biosynthesis: 5/4 (harmonic, ascending)
- Degradation: 4/5 (subharmonic, descending)

This creates a natural polarity in the sonification where building up and breaking down
are harmonically related but inverted.

| Subcategory | Count | Prime Limit | Notes |
|-------------|-------|-------------|-------|
| Amino Acid | ~70 | 5-7 | Essential + non-essential |
| Nucleotide | ~30 | 5-7 | Purines, pyrimidines |
| Cofactor | ~50 | 7-11 | Vitamins, coenzymes |
| Fatty Acid/Lipid | ~40 | 5-7 | Membrane components |
| Carbohydrate | ~25 | 5 | Sugars, polysaccharides |
| Secondary Metabolite | ~30 | 7-13 | Specialized compounds |
| ... | ... | ... | ... |

In [21]:
# ============================================================
# BIOSYNTHESIS: SUBCATEGORY OVERVIEW
# ============================================================

print("═" * 70)
print(" 🧬 BIOSYNTHESIS SUBCATEGORIES")
print("═" * 70)

biosyn_subcats = (
    biosynthesis_df.groupby("subcategory")
    .agg(
        count=("pathway_id", "size"),
        prev_mean=("prevalence", "mean"),
        prev_max=("prevalence", "max"),
    )
    .sort_values("prev_mean", ascending=False)
)

print(f"\n{len(biosyn_subcats)} subcategories, {len(biosynthesis_df)} pathways total\n")

for subcat, row in biosyn_subcats.iterrows():
    print(f"  {row['count']:>3}  {subcat:<50} (prev: {row['prev_mean']:.4f})")

══════════════════════════════════════════════════════════════════════
 🧬 BIOSYNTHESIS SUBCATEGORIES
══════════════════════════════════════════════════════════════════════

9 subcategories, 143 pathways total

  7.0  Nucleotides                                        (prev: 0.9981)
  28.0  Amino Acids                                        (prev: 0.9444)
  17.0  Fatty Acids/Lipids                                 (prev: 0.8421)
  28.0  Cofactors/Vitamins                                 (prev: 0.7104)
  24.0  Cell Envelope                                      (prev: 0.6924)
  7.0  Isoprenoids                                        (prev: 0.6732)
  6.0  Polyamines                                         (prev: 0.5017)
  12.0  Other                                              (prev: 0.4915)
  14.0  Specialized                                        (prev: 0.4315)


#### Nucleotides

In [22]:
# ============================================================
# BIOSYNTHESIS — NUCLEOTIDES
# Ground-level inspection (pre-harmonic)
# ============================================================

import pandas as pd

# ------------------------------------------------------------
# 1. Filter nucleotide biosynthesis pathways
# ------------------------------------------------------------
nucleotide_df = (
    biosynthesis_df[
        biosynthesis_df["subcategory"]
        .astype(str)
        .str.lower()
        .str.contains("nucleotide")
    ]
    .copy()
)

print("=" * 70)
print("🧬 BIOSYNTHESIS — NUCLEOTIDES")
print("=" * 70)
print(f"Nucleotide pathways total: {len(nucleotide_df)}\n")

if len(nucleotide_df) == 0:
    print("⚠️ No nucleotide pathways found. Check subcategory labels.")
    nucleotide_df
else:

    # --------------------------------------------------------
    # 2. Top nucleotide pathways by prevalence
    # --------------------------------------------------------
    print("🧬 TOP NUCLEOTIDE PATHWAYS BY PREVALENCE\n")

    top_prev = (
        nucleotide_df
        .sort_values("prevalence", ascending=False)
        .reset_index(drop=True)
    )

    display(
        top_prev[
            [c for c in [
                "pathway_id",
                "pathway_name",
                "subcategory",
                "prevalence",
                "mean_abundance",
                "tier",
            ] if c in top_prev.columns]
        ]
    )

    # --------------------------------------------------------
    # 3. Top nucleotide pathways by mean abundance
    # --------------------------------------------------------
    print("\n🧬 TOP NUCLEOTIDE PATHWAYS BY MEAN ABUNDANCE\n")

    top_abund = (
        nucleotide_df
        .sort_values("mean_abundance", ascending=False)
        .reset_index(drop=True)
    )

    display(
        top_abund[
            [c for c in [
                "pathway_id",
                "pathway_name",
                "subcategory",
                "prevalence",
                "mean_abundance",
                "tier",
            ] if c in top_abund.columns]
        ]
    )

    # --------------------------------------------------------
    # 4. Simple RNA vs DNA vs Precursor heuristic
    # --------------------------------------------------------
    print("\n🧬 NUCLEOTIDE FUNCTIONAL ROLES (heuristic)\n")

    role_df = nucleotide_df.copy()
    role_df["role"] = role_df["pathway_name"].str.lower().apply(
        lambda x: (
            "DNA" if "deoxy" in x else
            "RNA" if "ribo" in x else
            "shared / precursor"
        )
    )

    display(
        role_df.groupby("role")
        .agg(
            pathways=("pathway_id", "count"),
            mean_prevalence=("prevalence", "mean"),
            mean_abundance=("mean_abundance", "mean"),
        )
        .sort_values("mean_prevalence", ascending=False)
    )

🧬 BIOSYNTHESIS — NUCLEOTIDES
Nucleotide pathways total: 7

🧬 TOP NUCLEOTIDE PATHWAYS BY PREVALENCE



,pathway_id,pathway_name,subcategory,prevalence,mean_abundance,tier
0,PWY-7219,adenosine ribonucleotides de novo biosynthesis,Nucleotides,0.999178,0.000572,UNIVERSAL
1,PWY-6122,5-aminoimidazole ribonucleotide biosynthesis II,Nucleotides,0.999178,0.000484,UNIVERSAL
2,PWY-7221,guanosine ribonucleotides de novo biosynthesis,Nucleotides,0.998972,0.000549,UNIVERSAL
3,PWY-7220,adenosine deoxyribonucleotides de novo biosynt...,Nucleotides,0.998561,0.000254,UNIVERSAL
4,PWY-7222,guanosine deoxyribonucleotides de novo biosynt...,Nucleotides,0.998561,0.000254,UNIVERSAL
5,PWY-7187,pyrimidine deoxyribonucleotides de novo biosyn...,Nucleotides,0.997122,0.000194,UNIVERSAL
6,PWY0-162,superpathway of pyrimidine ribonucleotides de ...,Nucleotides,0.995272,0.000160,UNIVERSAL



🧬 TOP NUCLEOTIDE PATHWAYS BY MEAN ABUNDANCE



,pathway_id,pathway_name,subcategory,prevalence,mean_abundance,tier
0,PWY-7219,adenosine ribonucleotides de novo biosynthesis,Nucleotides,0.999178,0.000572,UNIVERSAL
1,PWY-7221,guanosine ribonucleotides de novo biosynthesis,Nucleotides,0.998972,0.000549,UNIVERSAL
2,PWY-6122,5-aminoimidazole ribonucleotide biosynthesis II,Nucleotides,0.999178,0.000484,UNIVERSAL
3,PWY-7220,adenosine deoxyribonucleotides de novo biosynt...,Nucleotides,0.998561,0.000254,UNIVERSAL
4,PWY-7222,guanosine deoxyribonucleotides de novo biosynt...,Nucleotides,0.998561,0.000254,UNIVERSAL
5,PWY-7187,pyrimidine deoxyribonucleotides de novo biosyn...,Nucleotides,0.997122,0.000194,UNIVERSAL
6,PWY0-162,superpathway of pyrimidine ribonucleotides de ...,Nucleotides,0.995272,0.000160,UNIVERSAL



🧬 NUCLEOTIDE FUNCTIONAL ROLES (heuristic)



,pathways,mean_prevalence,mean_abundance
role,,,
RNA,4,0.998150,0.000441
DNA,3,0.998082,0.000234


In [23]:
# ============================================================
# BIOSYNTHESIS — Nucleotides
# Structured 5-limit / 15-limit assignment
# ============================================================
SECTION = "Nucleotides"

# ------------------------------------------------------------
# 1. Define the composed mapping
# ------------------------------------------------------------
# Conceptual structure:
#
# 5-series  → RNA (continuous, present-time metabolism)
# 10/1      → Purine pool amplification (shared precursor)
# 15-series → DNA (replication commitment and execution)

NUCLEOTIDE_MAPPING = {
    # RNA — Material permission & expression (5-series)
    'PWY-7219': (5, 1),   # adenosine ribonucleotides (ATP, RNA root)
    'PWY-7221': (5, 2),   # guanosine ribonucleotides (RNA action)
    'PWY0-162': (5, 4),   # pyrimidine ribonucleotides (RNA expression)

    # PURINE PRECURSOR — Capacity scaling
    'PWY-6122': (10, 1),  # AIR / IMP precursor (purine pool capacity)

    # DNA — Replication commitment & completion (15-series)
    'PWY-7220': (15, 1),  # adenosine deoxyribonucleotides (DNA decision)
    'PWY-7222': (15, 2),  # guanosine deoxyribonucleotides (DNA execution)
    'PWY-7187': (15, 4),  # pyrimidine deoxyribonucleotides (DNA completion)
}

# Derivation notes for documentation
NUCLEOTIDE_DERIVATIONS = {
    'PWY-7219': 'RNA + ATP root — material permission, metabolic time',
    'PWY-7221': 'RNA guanosine — execution within continuous time',
    'PWY0-162': 'RNA pyrimidines — transcriptional expression/color',
    'PWY-6122': 'Purine precursor — nucleotide pool amplification',
    'PWY-7220': 'DNA adenosine — commitment to replication',
    'PWY-7222': 'DNA guanosine — replication execution',
    'PWY-7187': 'DNA pyrimidines — replication completion/fidelity',
}

# ------------------------------------------------------------
# 2. Select nucleotide pathways from biosynthesis dataframe
# ------------------------------------------------------------
nucleotide_df = (
    biosynthesis_df[
        biosynthesis_df["subcategory"]
        .astype(str)
        .str.lower()
        .eq("nucleotides")
    ]
    .copy()
)

if len(nucleotide_df) == 0:
    print("⚠ No nucleotide pathways found")
else:
    print(f"Nucleotide pathways: {len(nucleotide_df)}")

# ------------------------------------------------------------
# 3. Get unassigned nucleotide pathways
# ------------------------------------------------------------
unassigned = HARMONIC_MAP.get_unassigned(
    nucleotide_df["pathway_id"].tolist()
)

nucleotide_df = nucleotide_df[
    nucleotide_df["pathway_id"].isin(unassigned)
]

print(f"Unassigned nucleotide pathways: {len(nucleotide_df)}")

# ------------------------------------------------------------
# 4. Assign from composed mapping
# ------------------------------------------------------------
assigned_count = 0
skipped = []

for pathway_id, (n, d) in NUCLEOTIDE_MAPPING.items():

    row_match = nucleotide_df[nucleotide_df["pathway_id"] == pathway_id]

    if len(row_match) == 0:
        # Either already assigned or not present
        if not HARMONIC_MAP.is_assigned(pathway_id):
            skipped.append(pathway_id)
        continue

    row = row_match.iloc[0]
    ratio = Ratio(n, d)

    # Check ratio availability
    if HARMONIC_MAP.is_ratio_used(n, d):
        print(f"⚠ Ratio {n}/{d} already used — skipping {pathway_id}")
        continue

    derivation = NUCLEOTIDE_DERIVATIONS.get(
        pathway_id,
        "5-limit / 15-limit nucleotide assignment"
    )

    if HARMONIC_MAP.assign(
        pathway_id,
        ratio,
        {
            "pathway_name": row["pathway_name"],
            "subcategory": row["subcategory"],
            "category": "biosynthesis",
            "prevalence": row["prevalence"],
            "derivation": derivation,
        }
    ):
        assigned_count += 1
        print(
            f"✓ {pathway_id:<15} "
            f"→ {str(ratio):<6} "
            f"n×d={ratio.consonance():<4} "
            f"| {derivation}"
        )

# ------------------------------------------------------------
# 5. Report results
# ------------------------------------------------------------
print(f"\n{'='*60}")
print(f"Assigned: {assigned_count} nucleotide pathways")

if skipped:
    print(f"Skipped (not in dataset): {skipped}")

print(f"Map status: {HARMONIC_MAP.status()}")

# ------------------------------------------------------------
# 6. Display harmonic structure
# ------------------------------------------------------------
print(f"\n{'='*60}")
print("NUCLEOTIDE HARMONIC STRUCTURE")
print("="*60)

families = {
    "RNA (5-series)": ['PWY-7219', 'PWY-7221', 'PWY0-162'],
    "Purine precursor (10/1)": ['PWY-6122'],
    "DNA (15-series)": ['PWY-7220', 'PWY-7222', 'PWY-7187'],
}

for family_name, pathway_ids in families.items():
    print(f"\n{family_name}:")
    for pid in pathway_ids:
        if pid in NUCLEOTIDE_MAPPING:
            n, d = NUCLEOTIDE_MAPPING[pid]
            print(f"  {n}/{d:<4} n×d={n*d:<4} {pid}")

# ------------------------------------------------------------
# 7. Save progress
# ------------------------------------------------------------
HARMONIC_MAP.save()
print(f"\n✓ Saved!")
print(f"\nFinal stats:")
print(HARMONIC_MAP.stats())

Nucleotide pathways: 7
Unassigned nucleotide pathways: 7
✓ PWY-7219        → 5/1    n×d=5    | RNA + ATP root — material permission, metabolic time
✓ PWY-7221        → 5/2    n×d=10   | RNA guanosine — execution within continuous time
✓ PWY0-162        → 5/4    n×d=20   | RNA pyrimidines — transcriptional expression/color
✓ PWY-6122        → 10/1   n×d=10   | Purine precursor — nucleotide pool amplification
✓ PWY-7220        → 15/1   n×d=15   | DNA adenosine — commitment to replication
✓ PWY-7222        → 15/2   n×d=30   | DNA guanosine — replication execution
✓ PWY-7187        → 15/4   n×d=60   | DNA pyrimidines — replication completion/fidelity

Assigned: 7 nucleotide pathways
Map status: 34 pathways | 34 ratios

NUCLEOTIDE HARMONIC STRUCTURE

RNA (5-series):
  5/1    n×d=5    PWY-7219
  5/2    n×d=10   PWY-7221
  5/4    n×d=20   PWY0-162

Purine precursor (10/1):
  10/1    n×d=10   PWY-6122

DNA (15-series):
  15/1    n×d=15   PWY-7220
  15/2    n×d=30   PWY-7222
  15/4    n×d=60   

#### Amino Acids

In [24]:
# ============================================================
# BIOSYNTHESIS — Amino Acids
# ============================================================

import pandas as pd

# ------------------------------------------------------------
# 1. Filter amino acid biosynthesis pathways
# ------------------------------------------------------------
aa_df = (
    biosynthesis_df[
        biosynthesis_df["subcategory"]
        .astype(str)
        .str.lower()
        .str.contains("amino")
    ]
    .copy()
)

print("=" * 70)
print("🧬 BIOSYNTHESIS — AMINO ACIDS (OVERVIEW)")
print("=" * 70)
print(f"Amino acid pathways total: {len(aa_df)}\n")

if len(aa_df) == 0:
    print("⚠ No amino acid pathways found")
    aa_df
else:
    # --------------------------------------------------------
    # 2. Top amino acid pathways by prevalence
    # --------------------------------------------------------
    print("🧬 TOP AMINO ACID PATHWAYS BY PREVALENCE\n")

    top_prev = (
        aa_df
        .sort_values("prevalence", ascending=False)
        .reset_index(drop=True)
    )

    display(
        top_prev[
            [c for c in [
                "pathway_id",
                "pathway_name",
                "subcategory",
                "prevalence",
                "mean_abundance",
                "tier",
            ] if c in top_prev.columns]
        ]
    )

    # --------------------------------------------------------
    # 3. Top amino acid pathways by mean abundance
    # --------------------------------------------------------
    print("\n🧬 TOP AMINO ACID PATHWAYS BY MEAN ABUNDANCE\n")

    top_abund = (
        aa_df
        .sort_values("mean_abundance", ascending=False)
        .reset_index(drop=True)
    )

    display(
        top_abund[
            [c for c in [
                "pathway_id",
                "pathway_name",
                "subcategory",
                "prevalence",
                "mean_abundance",
                "tier",
            ] if c in top_abund.columns]
        ]
    )

    # --------------------------------------------------------
    # 4. Simple biological clustering by name (heuristic)
    # --------------------------------------------------------
    print("\n🧬 AMINO ACID FAMILIES (heuristic view)\n")

    def aa_family(name):
        n = name.lower()
        if any(x in n for x in ["valine", "leucine", "isoleucine"]):
            return "branched-chain (BCAA)"
        if any(x in n for x in ["glycine", "serine", "threonine"]):
            return "small / polar"
        if any(x in n for x in ["phenyl", "tyrosine", "tryptophan", "aromatic"]):
            return "aromatic"
        if any(x in n for x in ["arginine", "lysine", "histidine"]):
            return "basic / nitrogen-rich"
        if any(x in n for x in ["methionine", "cysteine", "selen"]):
            return "sulfur / redox-sensitive"
        if any(x in n for x in ["glutamate", "glutamine", "aspartate", "asparagine"]):
            return "acidic / central carbon"
        return "other"

    fam_df = aa_df.copy()
    fam_df["aa_family"] = fam_df["pathway_name"].apply(aa_family)

    display(
        fam_df.groupby("aa_family")
        .agg(
            pathways=("pathway_id", "count"),
            mean_prevalence=("prevalence", "mean"),
            mean_abundance=("mean_abundance", "mean"),
        )
        .sort_values("mean_prevalence", ascending=False)
    )

🧬 BIOSYNTHESIS — AMINO ACIDS (OVERVIEW)
Amino acid pathways total: 28

🧬 TOP AMINO ACID PATHWAYS BY PREVALENCE



,pathway_id,pathway_name,subcategory,prevalence,mean_abundance,tier
0,VALSYN-PWY,L-valine biosynthesis,Amino Acids,0.999178,0.000558,UNIVERSAL
1,ILEUSYN-PWY,L-isoleucine biosynthesis I (from threonine),Amino Acids,0.998972,0.000558,UNIVERSAL
2,BRANCHED-CHAIN-AA-SYN-PWY,superpathway of branched amino acid biosynthesis,Amino Acids,0.998972,0.000461,UNIVERSAL
3,PWY-5097,L-lysine biosynthesis VI,Amino Acids,0.998972,0.000448,UNIVERSAL
4,ARGSYNBSUB-PWY,L-arginine biosynthesis II (acetyl cycle),Amino Acids,0.998972,0.000418,UNIVERSAL
5,PWY-5103,L-isoleucine biosynthesis III,Amino Acids,0.998972,0.000441,UNIVERSAL
6,HISTSYN-PWY,L-histidine biosynthesis,Amino Acids,0.998972,0.000377,UNIVERSAL
7,ARGSYN-PWY,L-arginine biosynthesis I (via L-ornithine),Amino Acids,0.998767,0.000419,UNIVERSAL
8,PWY-2942,L-lysine biosynthesis III,Amino Acids,0.998767,0.000416,UNIVERSAL
9,THRESYN-PWY,superpathway of L-threonine biosynthesis,Amino Acids,0.998767,0.000295,UNIVERSAL



🧬 TOP AMINO ACID PATHWAYS BY MEAN ABUNDANCE



,pathway_id,pathway_name,subcategory,prevalence,mean_abundance,tier
0,VALSYN-PWY,L-valine biosynthesis,Amino Acids,0.999178,0.000558,UNIVERSAL
1,ILEUSYN-PWY,L-isoleucine biosynthesis I (from threonine),Amino Acids,0.998972,0.000558,UNIVERSAL
2,BRANCHED-CHAIN-AA-SYN-PWY,superpathway of branched amino acid biosynthesis,Amino Acids,0.998972,0.000461,UNIVERSAL
3,PWY-5097,L-lysine biosynthesis VI,Amino Acids,0.998972,0.000448,UNIVERSAL
4,PWY-5103,L-isoleucine biosynthesis III,Amino Acids,0.998972,0.000441,UNIVERSAL
5,ARGSYN-PWY,L-arginine biosynthesis I (via L-ornithine),Amino Acids,0.998767,0.000419,UNIVERSAL
6,ARGSYNBSUB-PWY,L-arginine biosynthesis II (acetyl cycle),Amino Acids,0.998972,0.000418,UNIVERSAL
7,PWY-2942,L-lysine biosynthesis III,Amino Acids,0.998767,0.000416,UNIVERSAL
8,HISTSYN-PWY,L-histidine biosynthesis,Amino Acids,0.998972,0.000377,UNIVERSAL
9,THRESYN-PWY,superpathway of L-threonine biosynthesis,Amino Acids,0.998767,0.000295,UNIVERSAL



🧬 AMINO ACID FAMILIES (heuristic view)



,pathways,mean_prevalence,mean_abundance
aa_family,,,
small / polar,2,0.998664,0.000278
basic / nitrogen-rich,9,0.995250,0.000286
other,2,0.989723,0.000272
acidic / central carbon,2,0.989414,0.000133
sulfur / redox-sensitive,5,0.972785,0.000143
branched-chain (BCAA),5,0.886701,0.000338
aromatic,3,0.743953,0.000099


In [25]:
# ============================================================
# BIOSYNTHESIS TOOL — 7-limit consonant ratios (unused)
# Above unison, within range
# ============================================================

def generate_7limit_consonant_unused(
    min_value=1.0,
    max_value=16.0,
    max_consonance=3000
):
    """
    Generate unused 7-limit ratios:
      - value > min_value
      - value <= max_value
      - sorted by consonance (n * d)
      - excludes all currently assigned ratios
    """
    ratios = generate_by_prime_limit(
        limit=7,
        count=None,
        max_consonance=max_consonance,
        polarity=None,
        exclude_ratios=HARMONIC_MAP.used_ratios
    )

    # Filter to range
    ratios = [
        r for r in ratios
        if r.value() > min_value and r.value() <= max_value
    ]

    # Sort by consonance, then by value
    ratios.sort(key=lambda r: (r.consonance(), r.value()))

    return ratios


# === RUN ===
aa_candidate_pool = generate_7limit_consonant_unused(
    min_value=1.0,
    max_value=16.0,
    max_consonance=2500
)

# Preview top candidates
print(f"Total available 7-limit ratios: {len(aa_candidate_pool)}\n")

for r in aa_candidate_pool[:30]:
    print(
        f"{str(r):<6} "
        f"value={r.value():<6.3f} "
        f"n×d={r.consonance():<4} "
        f"pol={r.polarity():+.2f} "
        f"{r.polarity_label()}"
    )

Total available 7-limit ratios: 205

7/1    value=7.000  n×d=7    pol=+1.00 harmonic
7/2    value=3.500  n×d=14   pol=+0.71 harmonic
14/1   value=14.000 n×d=14   pol=+1.00 harmonic
5/3    value=1.667  n×d=15   pol=+0.19 neutral
7/3    value=2.333  n×d=21   pol=+0.28 neutral
7/4    value=1.750  n×d=28   pol=+0.58 harmonic
6/5    value=1.200  n×d=30   pol=-0.13 neutral
10/3   value=3.333  n×d=30   pol=+0.13 neutral
7/5    value=1.400  n×d=35   pol=+0.09 neutral
8/5    value=1.600  n×d=40   pol=-0.50 subharmonic
7/6    value=1.167  n×d=42   pol=+0.20 neutral
14/3   value=4.667  n×d=42   pol=+0.20 neutral
21/2   value=10.500 n×d=42   pol=+0.71 harmonic
9/5    value=1.800  n×d=45   pol=+0.15 neutral
25/2   value=12.500 n×d=50   pol=+0.71 harmonic
8/7    value=1.143  n×d=56   pol=-0.50 subharmonic
12/5   value=2.400  n×d=60   pol=-0.11 neutral
20/3   value=6.667  n×d=60   pol=+0.11 neutral
9/7    value=1.286  n×d=63   pol=+0.06 neutral
10/7   value=1.429  n×d=70   pol=-0.07 neutral
14/5   va

In [26]:
# ============================================================
# BIOSYNTHESIS — Amino Acids
# Histidine (catalytic hinge) → Branched-chain / Leucine axis
# ============================================================

SECTION = "Amino Acids — Histidine + BCAA / Leucine Axis"

# ------------------------------------------------------------
# Catalytic hinge BEFORE identity
# 5/3 = 5 / 3 → catalytic material embedded in circulation
# ------------------------------------------------------------

# ------------------------------------------------------------
# Introduce prime‑7 (protein identity)
# 7/1   = 7                         → protein identity (branched‑chain core)
# 7/2   = 7 / 2                     → simplest structural unit (valine)
# 7/4   = 7 / 2²                    → differentiated structure (leucine)
#
# L‑leucine / isoleucine variants (circulation & specialization)
# 21/8  = (7 × 3) / 2³              → identity + circulation
# 63/16 = (7 × 3²) / 2⁴             → identity under strong metabolic cycling
# 189/32= (7 × 3³) / 2⁵             → specialized / high‑load variant
# ------------------------------------------------------------

AMINO_ACID_MAPPING = {
    # HISTIDINE — catalytic hinge
    'HISTSYN-PWY':              (5, 3),   # catalysis in motion (baseline)

    # PRIME-7 INTRODUCTION
    'BRANCHED-CHAIN-AA-SYN-PWY':(7, 1),   # protein identity / hydrophobic core

    # CORE BCAA
    'VALSYN-PWY':               (7, 2),   # valine — simplest BCAA

    # LEUCINE AXIS (circulation ladder)
    'ILEUSYN-PWY':              (7, 4),   # leucine / isoleucine — differentiated structure
    'PWY-5103':                 (21, 8),  # variant (circulation engaged)
    'PWY-5104':                 (63, 16), # variant (high metabolic cycling)
    'PWY-5101':                 (189, 32) # rare / conditionally active route
}

# Derivation notes for documentation
AMINO_ACID_DERIVATIONS = {
    'HISTSYN-PWY':              'histidine — catalytic hinge embedded in metabolic flow',
    'BRANCHED-CHAIN-AA-SYN-PWY':'prime-7 introduction — protein identity / hydrophobic core',
    'VALSYN-PWY':               'simplest structural BCAA',
    'ILEUSYN-PWY':              'differentiated BCAA structure',
    'PWY-5103':                 'leucine variant — structure engaged with circulation',
    'PWY-5104':                 'leucine variant — high metabolic turnover',
    'PWY-5101':                 'leucine variant — specialized / condition-dependent'
}

# ------------------------------------------------------------
# 2. Select amino-acid biosynthesis pathways
# ------------------------------------------------------------
aa_df = (
    biosynthesis_df[
        biosynthesis_df["subcategory"]
        .astype(str)
        .str.lower()
        .str.contains("amino")
    ]
    .copy()
)

if len(aa_df) == 0:
    print("⚠ No amino acid pathways found")
else:
    print(f"Amino acid pathways (candidate): {len(aa_df)}")

# ------------------------------------------------------------
# 3. Get unassigned amino-acid pathways
# ------------------------------------------------------------
unassigned = HARMONIC_MAP.get_unassigned(
    aa_df["pathway_id"].tolist()
)

aa_df = aa_df[
    aa_df["pathway_id"].isin(unassigned)
]

print(f"Unassigned amino-acid pathways: {len(aa_df)}")

# ------------------------------------------------------------
# 4. Assign from composed mapping
# ------------------------------------------------------------
assigned_count = 0
skipped = []

for pathway_id, (n, d) in AMINO_ACID_MAPPING.items():

    row_match = aa_df[aa_df["pathway_id"] == pathway_id]

    if len(row_match) == 0:
        if not HARMONIC_MAP.is_assigned(pathway_id):
            skipped.append(pathway_id)
        continue

    row = row_match.iloc[0]
    ratio = Ratio(n, d)

    # Safety: check ratio availability
    if HARMONIC_MAP.is_ratio_used(n, d):
        print(f"⚠ Ratio {n}/{d} already used — skipping {pathway_id}")
        continue

    derivation = AMINO_ACID_DERIVATIONS.get(
        pathway_id,
        "amino-acid harmonic assignment"
    )

    if HARMONIC_MAP.assign(
        pathway_id,
        ratio,
        {
            "pathway_name": row["pathway_name"],
            "subcategory": row["subcategory"],
            "category": "biosynthesis",
            "prevalence": row["prevalence"],
            "derivation": derivation,
        }
    ):
        assigned_count += 1
        print(
            f"✓ {pathway_id:<28} "
            f"→ {str(ratio):<7} "
            f"n×d={ratio.consonance():<5} "
            f"| {derivation}"
        )

# ------------------------------------------------------------
# 5. Report results
# ------------------------------------------------------------
print(f"\n{'='*60}")
print(f"Assigned: {assigned_count} amino-acid pathways")
if skipped:
    print(f"Skipped (not in dataset or already assigned): {skipped}")
print(f"Map status: {HARMONIC_MAP.status()}")

# ------------------------------------------------------------
# 6. Display ladder explicitly
# ------------------------------------------------------------
print(f"\n{'='*60}")
print("HISTIDINE → BCAA / LEUCINE AXIS")
print("="*60)

ladder = [(5,3), (7,1), (7,2), (7,4), (21,8), (63,16), (189,32)]
for n, d in ladder:
    r = Ratio(n, d)
    print(f"{n}/{d:<6} value={r.value():<6.3f} n×d={r.consonance():<6}")

# ------------------------------------------------------------
# 7. Save progress
# ------------------------------------------------------------
HARMONIC_MAP.save()
print("\n✓ Saved!")
print("\nFinal stats:")
print(HARMONIC_MAP.stats())

Amino acid pathways (candidate): 28
Unassigned amino-acid pathways: 28
✓ HISTSYN-PWY                  → 5/3     n×d=15    | histidine — catalytic hinge embedded in metabolic flow
✓ BRANCHED-CHAIN-AA-SYN-PWY    → 7/1     n×d=7     | prime-7 introduction — protein identity / hydrophobic core
✓ VALSYN-PWY                   → 7/2     n×d=14    | simplest structural BCAA
✓ ILEUSYN-PWY                  → 7/4     n×d=28    | differentiated BCAA structure
✓ PWY-5103                     → 21/8    n×d=168   | leucine variant — structure engaged with circulation
✓ PWY-5104                     → 63/16   n×d=1008  | leucine variant — high metabolic turnover
✓ PWY-5101                     → 189/32  n×d=6048  | leucine variant — specialized / condition-dependent

Assigned: 7 amino-acid pathways
Map status: 41 pathways | 41 ratios

HISTIDINE → BCAA / LEUCINE AXIS
5/3      value=1.667  n×d=15    
7/1      value=7.000  n×d=7     
7/2      value=3.500  n×d=14    
7/4      value=1.750  n×d=28    
21/8    

In [27]:
# ============================================================
# BIOSYNTHESIS — Amino Acids
# Lysine Axis (interaction / binding ladder)
# ============================================================

SECTION = "Amino Acids — Lysine Axis"

# ------------------------------------------------------------
# 1. Define the composed mapping
# ------------------------------------------------------------
# Conceptual structure:
#
# 7/3   = 7 / 3             # aminotransferase route
# 7/6   = 7 / (2 × 3)       # dehydrogenase route
# 14/3  = (7 × 2) / 3       # succinylase route (canonical)
# 28/3  = (7 × 2²) / 3      # acetylase route

LYSINE_MAPPING = {
    'PWY-5097':        (7, 3),   # aminotransferase route
    'PWY-2942':        (7, 6),   # dehydrogenase route
    'DAPLYSINESYN-PWY':(14, 3),  # succinylase route (canonical)
    'PWY-2941':        (28, 3),  # acetylase route
}

LYSINE_DERIVATIONS = {
    'PWY-5097':        'lysine — identity engaged in interaction',
    'PWY-2942':        'lysine — buffered / moderated binding',
    'DAPLYSINESYN-PWY':'lysine — amplified structural anchoring',
    'PWY-2941':        'lysine — high-affinity interaction',
}

# ------------------------------------------------------------
# 2. Select amino-acid biosynthesis pathways
# ------------------------------------------------------------
aa_df = (
    biosynthesis_df[
        biosynthesis_df["subcategory"]
        .astype(str)
        .str.lower()
        .str.contains("amino")
    ]
    .copy()
)

if len(aa_df) == 0:
    print("⚠ No amino acid pathways found")
else:
    print(f"Amino acid pathways (candidate): {len(aa_df)}")

# ------------------------------------------------------------
# 3. Get unassigned amino-acid pathways
# ------------------------------------------------------------
unassigned = HARMONIC_MAP.get_unassigned(
    aa_df["pathway_id"].tolist()
)

aa_df = aa_df[
    aa_df["pathway_id"].isin(unassigned)
]

print(f"Unassigned amino-acid pathways: {len(aa_df)}")

# ------------------------------------------------------------
# 4. Assign from composed mapping
# ------------------------------------------------------------
assigned_count = 0
skipped = []

for pathway_id, (n, d) in LYSINE_MAPPING.items():

    row_match = aa_df[aa_df["pathway_id"] == pathway_id]

    if len(row_match) == 0:
        if not HARMONIC_MAP.is_assigned(pathway_id):
            skipped.append(pathway_id)
        continue

    row = row_match.iloc[0]
    ratio = Ratio(n, d)

    # Safety check
    if HARMONIC_MAP.is_ratio_used(n, d):
        print(f"⚠ Ratio {n}/{d} already used — skipping {pathway_id}")
        continue

    derivation = LYSINE_DERIVATIONS.get(
        pathway_id,
        "lysine interaction ladder assignment"
    )

    if HARMONIC_MAP.assign(
        pathway_id,
        ratio,
        {
            "pathway_name": row["pathway_name"],
            "subcategory": row["subcategory"],
            "category": "biosynthesis",
            "prevalence": row["prevalence"],
            "derivation": derivation,
        }
    ):
        assigned_count += 1
        print(
            f"✓ {pathway_id:<24} "
            f"→ {str(ratio):<6} "
            f"n×d={ratio.consonance():<5} "
            f"| {derivation}"
        )

# ------------------------------------------------------------
# 5. Report results
# ------------------------------------------------------------
print(f"\n{'='*60}")
print(f"Assigned: {assigned_count} lysine pathways")
if skipped:
    print(f"Skipped (not in dataset or already assigned): {skipped}")
print(f"Map status: {HARMONIC_MAP.status()}")

# ------------------------------------------------------------
# 6. Display ladder explicitly
# ------------------------------------------------------------
print(f"\n{'='*60}")
print("LYSINE INTERACTION LADDER")
print("="*60)

ladder = [(7,3), (7,6), (14,3), (28,3)]
for n, d in ladder:
    r = Ratio(n, d)
    print(
        f"{n}/{d:<5} "
        f"value={r.value():<6.3f} "
        f"n×d={r.consonance():<6} "
        f"pol={r.polarity():+.2f}"
    )

# ------------------------------------------------------------
# 7. Save progress
# ------------------------------------------------------------
HARMONIC_MAP.save()
print("\n✓ Saved!")
print("\nFinal stats:")
print(HARMONIC_MAP.stats())

Amino acid pathways (candidate): 28
Unassigned amino-acid pathways: 21
✓ PWY-5097                 → 7/3    n×d=21    | lysine — identity engaged in interaction
✓ PWY-2942                 → 7/6    n×d=42    | lysine — buffered / moderated binding
✓ DAPLYSINESYN-PWY         → 14/3   n×d=42    | lysine — amplified structural anchoring
✓ PWY-2941                 → 28/3   n×d=84    | lysine — high-affinity interaction

Assigned: 4 lysine pathways
Map status: 45 pathways | 45 ratios

LYSINE INTERACTION LADDER
7/3     value=2.333  n×d=21     pol=+0.28
7/6     value=1.167  n×d=42     pol=+0.20
14/3     value=4.667  n×d=42     pol=+0.20
28/3     value=9.333  n×d=84     pol=+0.16
✓ Saved 45 assignments to harmonic_map.json

✓ Saved!

Final stats:
{'assigned': 45, 'consonance': {'min': 1, 'max': 6048, 'mean': 277.3777777777778}, 'prime_limit': {'min': 1, 'max': 7, 'distribution': {1: 1, 2: 4, 3: 22, 5: 8, 7: 10}}, 'polarity': {'harmonic': 30, 'subharmonic': 9, 'unison': 6}}


In [28]:
# ============================================================
# BIOSYNTHESIS — Amino Acids
# Arginine Axis (directional / nitrogen signaling)
# ============================================================

SECTION = "Amino Acids — Arginine Axis"

# ------------------------------------------------------------
# 1. Define the composed mapping
# ------------------------------------------------------------
# Conceptual structure:
#
# 21/2  = (7 × 3) / 2      → primary arginine signaling
# 21/4  = (7 × 3) / 2²     → moderated / regulated flow
# 21/16 = (7 × 3) / 2⁴     → background / contextual usage

ARGININE_MAPPING = {
    'ARGSYN-PWY':     (21, 2),   # canonical route (ornithine)
    'ARGSYNBSUB-PWY': (21, 4),   # acetyl cycle (Gram+)
    'PWY-5154':       (21, 16),  # N-acetyl-citrulline route
}

ARGININE_DERIVATIONS = {
    'ARGSYN-PWY':     'arginine — primary directional signaling',
    'ARGSYNBSUB-PWY': 'arginine — moderated nitrogen routing',
    'PWY-5154':       'arginine — contextual / conditional signaling',
}

# ------------------------------------------------------------
# 2. Select amino-acid biosynthesis pathways
# ------------------------------------------------------------
aa_df = (
    biosynthesis_df[
        biosynthesis_df["subcategory"]
        .astype(str)
        .str.lower()
        .str.contains("amino")
    ]
    .copy()
)

if len(aa_df) == 0:
    print("⚠ No amino acid pathways found")
else:
    print(f"Amino acid pathways (candidate): {len(aa_df)}")

# ------------------------------------------------------------
# 3. Get unassigned amino-acid pathways
# ------------------------------------------------------------
unassigned = HARMONIC_MAP.get_unassigned(
    aa_df["pathway_id"].tolist()
)

aa_df = aa_df[
    aa_df["pathway_id"].isin(unassigned)
]

print(f"Unassigned amino-acid pathways: {len(aa_df)}")

# ------------------------------------------------------------
# 4. Assign from composed mapping
# ------------------------------------------------------------
assigned_count = 0
skipped = []

for pathway_id, (n, d) in ARGININE_MAPPING.items():

    row_match = aa_df[aa_df["pathway_id"] == pathway_id]

    if len(row_match) == 0:
        if not HARMONIC_MAP.is_assigned(pathway_id):
            skipped.append(pathway_id)
        continue

    row = row_match.iloc[0]
    ratio = Ratio(n, d)

    # Safety check
    if HARMONIC_MAP.is_ratio_used(n, d):
        print(f"⚠ Ratio {n}/{d} already used — skipping {pathway_id}")
        continue

    derivation = ARGININE_DERIVATIONS.get(
        pathway_id,
        "arginine directional assignment"
    )

    if HARMONIC_MAP.assign(
        pathway_id,
        ratio,
        {
            "pathway_name": row["pathway_name"],
            "subcategory": row["subcategory"],
            "category": "biosynthesis",
            "prevalence": row["prevalence"],
            "derivation": derivation,
        }
    ):
        assigned_count += 1
        print(
            f"✓ {pathway_id:<20} "
            f"→ {str(ratio):<6} "
            f"n×d={ratio.consonance():<5} "
            f"| {derivation}"
        )

# ------------------------------------------------------------
# 5. Report results
# ------------------------------------------------------------
print(f"\n{'='*60}")
print(f"Assigned: {assigned_count} arginine pathways")
if skipped:
    print(f"Skipped (not in dataset or already assigned): {skipped}")
print(f"Map status: {HARMONIC_MAP.status()}")

# ------------------------------------------------------------
# 6. Display ladder explicitly
# ------------------------------------------------------------
print(f"\n{'='*60}")
print("ARGININE DIRECTIONAL LADDER")
print("="*60)

ladder = [(21,2), (21,4), (21,16)]
for n, d in ladder:
    r = Ratio(n, d)
    print(
        f"{n}/{d:<5} "
        f"value={r.value():<6.3f} "
        f"n×d={r.consonance():<6} "
        f"pol={r.polarity():+.2f}"
    )

# ------------------------------------------------------------
# 7. Save progress
# ------------------------------------------------------------
HARMONIC_MAP.save()
print("\n✓ Saved!")
print("\nFinal stats:")
print(HARMONIC_MAP.stats())

Amino acid pathways (candidate): 28
Unassigned amino-acid pathways: 17
✓ ARGSYN-PWY           → 21/2   n×d=42    | arginine — primary directional signaling
✓ ARGSYNBSUB-PWY       → 21/4   n×d=84    | arginine — moderated nitrogen routing
✓ PWY-5154             → 21/16  n×d=336   | arginine — contextual / conditional signaling

Assigned: 3 arginine pathways
Map status: 48 pathways | 48 ratios

ARGININE DIRECTIONAL LADDER
21/2     value=10.500 n×d=42     pol=+0.71
21/4     value=5.250  n×d=84     pol=+0.58
21/16    value=1.312  n×d=336    pol=+0.45
✓ Saved 48 assignments to harmonic_map.json

✓ Saved!

Final stats:
{'assigned': 48, 'consonance': {'min': 1, 'max': 6048, 'mean': 269.6666666666667}, 'prime_limit': {'min': 1, 'max': 7, 'distribution': {1: 1, 2: 4, 3: 22, 5: 8, 7: 13}}, 'polarity': {'harmonic': 33, 'subharmonic': 9, 'unison': 6}}


In [29]:
# ============================================================
# RATIO INVENTORY — 9-limit ratios (unused, above unison)
# ============================================================

def generate_9limit_unused(
    min_value=1.0,
    max_value=16.0,
    max_consonance=5000
):
    """
    Generate all unused 9-limit ratios:
      - value > min_value
      - value <= max_value
      - sorted by consonance (n * d)
      - excludes all currently assigned ratios
    """
    ratios = generate_by_prime_limit(
        limit=9,
        count=None,
        max_consonance=max_consonance,
        polarity=None,
        exclude_ratios=HARMONIC_MAP.used_ratios
    )

    # Filter to range (>1 and within limits)
    ratios = [
        r for r in ratios
        if r.value() > min_value and r.value() <= max_value
    ]

    # Sort by consonance, then value
    ratios.sort(key=lambda r: (r.consonance(), r.value()))

    return ratios


# === RUN ===
available_9lim = generate_9limit_unused(
    min_value=1.0,
    max_value=16.0,
    max_consonance=3000
)

print(f"Available unused 9-limit ratios: {len(available_9lim)}\n")

# Preview top candidates
for r in available_9lim[:40]:
    print(
        f"{str(r):<7} "
        f"value={r.value():<6.3f} "
        f"n×d={r.consonance():<5} "
        f"pol={r.polarity():+.2f} "
        f"{r.polarity_label()}"
    )

Available unused 9-limit ratios: 210

14/1    value=14.000 n×d=14    pol=+1.00 harmonic
6/5     value=1.200  n×d=30    pol=-0.13 neutral
10/3    value=3.333  n×d=30    pol=+0.13 neutral
7/5     value=1.400  n×d=35    pol=+0.09 neutral
8/5     value=1.600  n×d=40    pol=-0.50 subharmonic
9/5     value=1.800  n×d=45    pol=+0.15 neutral
25/2    value=12.500 n×d=50    pol=+0.71 harmonic
8/7     value=1.143  n×d=56    pol=-0.50 subharmonic
12/5    value=2.400  n×d=60    pol=-0.11 neutral
20/3    value=6.667  n×d=60    pol=+0.11 neutral
9/7     value=1.286  n×d=63    pol=+0.06 neutral
10/7    value=1.429  n×d=70    pol=-0.07 neutral
14/5    value=2.800  n×d=70    pol=+0.07 neutral
25/3    value=8.333  n×d=75    pol=+0.49 harmonic
16/5    value=3.200  n×d=80    pol=-0.45 subharmonic
12/7    value=1.714  n×d=84    pol=-0.16 neutral
10/9    value=1.111  n×d=90    pol=-0.11 neutral
18/5    value=3.600  n×d=90    pol=+0.11 neutral
25/4    value=6.250  n×d=100   pol=+0.58 harmonic
15/7    value=2

In [30]:
# ------------------------------------------------------------
# METHIONINE / SAM — Methylation System
# ------------------------------------------------------------
# 35 = 7 × 5
#   7 → amino‑acid identity
#   5 → embodied chemical modification
#
# Denominator controls degree of circulation (3‑power).
#
# 35/3  = methionine I (canonical, high throughput)
# 35/6  = methionine III (regulated / alternate)
# 35/9  = SAM (universal methyl donor; full circulation)
# ------------------------------------------------------------

METHIONINE_ASSIGNMENTS = {
    'HOMOSER-METSYN-PWY': (35, 3, 'methionine I — sulfhydrylation (35 = 7×5)'),
    'HSERMETANA-PWY':     (35, 6, 'methionine III — transsulfuration (35 = 7×5)'),
    'MET-SAM-PWY':        (35, 9, 'SAM — methylation currency (35 = 7×5)'),
}

for pid, (n, d, deriv) in METHIONINE_ASSIGNMENTS.items():
    HARMONIC_MAP.assign(
        pid,
        Ratio(n, d),
        {
            'subcategory': 'Amino Acids',
            'category': 'biosynthesis',
            'derivation': deriv,
        }
    )
    print(f"✓ {pid} → {n}/{d}")

HARMONIC_MAP.save()

✓ HOMOSER-METSYN-PWY → 35/3
✓ HSERMETANA-PWY → 35/6
✓ MET-SAM-PWY → 35/9
✓ Saved 51 assignments to harmonic_map.json


In [31]:
# ------------------------------------------------------------
# Threonine, Serine, Glycine: branch points and one-carbon donors
#
# 10/3 = (2×5) / 3 → threonine (branch point, feeds Ile/Gly/Met)
# 25/3 = (5×5) / 3 → serine/glycine (one-carbon metabolism)
# ------------------------------------------------------------

SMALL_POLAR_ASSIGNMENTS = {
    'THRESYN-PWY':    (10, 3, 'threonine — branch point'),
    'SER-GLYSYN-PWY': (25, 3, 'serine/glycine — one-carbon supply'),
}

for pid, (n, d, deriv) in SMALL_POLAR_ASSIGNMENTS.items():
    HARMONIC_MAP.assign(pid, Ratio(n, d), {
        'subcategory': 'Amino Acids', 'category': 'biosynthesis', 'derivation': deriv,
    })
    print(f"✓ {pid} → {n}/{d}")
HARMONIC_MAP.save()

✓ THRESYN-PWY → 10/3
✓ SER-GLYSYN-PWY → 25/3
✓ Saved 53 assignments to harmonic_map.json


In [32]:
# ------------------------------------------------------------
# AROMATIC AMINO ACIDS — RESOLVED IDENTITY (POST-CHORISMATE)
# ------------------------------------------------------------
# role: fixed aromatic amino-acid identities
#
# 7/5     → phenylalanine (base aromatic identity)
# 21/10   → tyrosine (functionalized aromatic identity)
# 49/25   → tryptophan (deep, high-cost aromatic identity)
# ------------------------------------------------------------

AROMATIC_ASSIGNMENTS = {
    'PWY-6628': (
        7, 5,
        'phenylalanine — base aromatic identity'
    ),
    'PWY-6630': (
        21, 10,
        'tyrosine — functionalized aromatic identity'
    ),
    'TRPSYN-PWY': (
        21, 20,
        'tryptophan — deep, high-cost aromatic identity'
    ),
}

for pid, (n, d, note) in AROMATIC_ASSIGNMENTS.items():
    HARMONIC_MAP.assign(
        pid,
        Ratio(n, d),
        {
            'category': 'biosynthesis',
            'subcategory': 'Amino Acids',
            'derivation': note,
        }
    )
    print(f"✓ {pid} → {n}/{d}")

HARMONIC_MAP.save()

✓ PWY-6628 → 7/5
✓ PWY-6630 → 21/10
✓ TRPSYN-PWY → 21/20
✓ Saved 56 assignments to harmonic_map.json


In [33]:
# ------------------------------------------------------------
# ACIDIC — Metabolic Currency
# ------------------------------------------------------------
# Glutamate/Aspartate: nitrogen carriers, TCA connections
#
# 9/7  = (3²) / 7 → glutamate/glutamine (nitrogen hub)
# 18/7 = (2×9) / 7 → aspartate/asparagine (TCA-linked)
# ------------------------------------------------------------

ACIDIC_ASSIGNMENTS = {
    'PWY-5505':  (9, 7, 'glutamate/glutamine — nitrogen currency'),
    'ASPASN-PWY':(18, 7, 'aspartate/asparagine — TCA currency'),
}

for pid, (n, d, deriv) in ACIDIC_ASSIGNMENTS.items():
    HARMONIC_MAP.assign(pid, Ratio(n, d), {
        'subcategory': 'Amino Acids', 'category': 'biosynthesis', 'derivation': deriv,
    })
    print(f"✓ {pid} → {n}/{d}")
HARMONIC_MAP.save()

✓ PWY-5505 → 9/7
✓ ASPASN-PWY → 18/7
✓ Saved 58 assignments to harmonic_map.json


In [34]:
# ------------------------------------------------------------
# PROLINE — Structural Curvature  
#
# 15/7 = (3×5) / 7 → structural constraint on identity
# ------------------------------------------------------------

HARMONIC_MAP.assign('PWY-4981', Ratio(15, 7), {
    'pathway_name': 'L-proline biosynthesis II (from arginine)',
    'subcategory': 'Amino Acids',
    'category': 'biosynthesis',
    'derivation': 'proline — structural curvature',
})
print("✓ PWY-4981 → 15/7")
HARMONIC_MAP.save()

✓ PWY-4981 → 15/7
✓ Saved 59 assignments to harmonic_map.json


In [35]:
# ------------------------------------------------------------
# SULFUR / SELENIUM — Redox Chemistry (CORRECTED)
# ------------------------------------------------------------
# Cysteine:
#   • Disulfide bond formation
#   • Glutathione backbone
#   • Iron–sulfur cluster chemistry
#
# Selenocysteine:
#   • Rare cysteine analogue with enhanced redox kinetics
#   • Incorporated conditionally into selenoproteins
#   • Same biological kind, higher catalytic flow, lower availability
#
# Harmonic logic:
#   45/7   = (3² × 5) / 7
#          → embodied amino‑acid identity with strong redox flow
#
#   135/14 = (3³ × 5) / (2 × 7)
#          → same identity class, intensified kinetics,
#            availability‑constrained (selenium)
# ------------------------------------------------------------

SULFUR_ASSIGNMENTS = {
    'SULFATE-CYS-PWY': (
        45, 7,
        'cysteine — embodied redox identity (disulfides, glutathione, Fe–S)'
    ),
    'PWY-6936': (
        135, 14,
        'seleno-amino acids — rare, constrained high-kinetic redox identity'
    ),
}

for pid, (n, d, deriv) in SULFUR_ASSIGNMENTS.items():
    HARMONIC_MAP.assign(
        pid,
        Ratio(n, d),
        {
            'category': 'biosynthesis',
            'subcategory': 'Amino Acids',
            'derivation': deriv,
        }
    )
    print(f"✓ {pid} → {n}/{d}")

HARMONIC_MAP.save()

✓ SULFATE-CYS-PWY → 45/7
✓ PWY-6936 → 135/14
✓ Saved 61 assignments to harmonic_map.json


In [36]:
# ------------------------------------------------------------
# ALANINE — Nitrogen Shuttle
# ------------------------------------------------------------
# Simplest amino acid. Pyruvate ↔ alanine interconversion.
# Carries nitrogen between muscle and liver.
#
# 14/1 = (2×7) / 1 → identity in transport
# ------------------------------------------------------------

HARMONIC_MAP.assign('PWY0-1061', Ratio(14, 1), {
    'pathway_name': 'superpathway of L-alanine biosynthesis',
    'subcategory': 'Amino Acids',
    'category': 'biosynthesis',
    'derivation': 'alanine — nitrogen shuttle',
})
print("✓ PWY0-1061 → 14/1")
HARMONIC_MAP.save()



✓ PWY0-1061 → 14/1
✓ Saved 62 assignments to harmonic_map.json


##### Amino Acid Summary

In [37]:
# ============================================================
# AMINO ACIDS — COMPLETE SUMMARY
# ============================================================

print("=" * 70)
print("🧬 AMINO ACID BIOSYNTHESIS — COMPLETE ASSIGNMENT SUMMARY")
print("=" * 70)

# Get all amino acid pathways
aa_df = (
    biosynthesis_df[
        biosynthesis_df["subcategory"]
        .astype(str)
        .str.lower()
        .str.contains("amino")
    ]
    .copy()
)

# Check assignment status
aa_df["assigned"] = aa_df["pathway_id"].apply(HARMONIC_MAP.is_assigned)
aa_df["ratio"] = aa_df["pathway_id"].apply(
    lambda pid: str(HARMONIC_MAP.get_ratio(pid)) if HARMONIC_MAP.is_assigned(pid) else None
)

assigned_count = aa_df["assigned"].sum()
total_count = len(aa_df)

print(f"\nTotal amino acid pathways: {total_count}")
print(f"Assigned: {assigned_count}")
print(f"Remaining: {total_count - assigned_count}")

if assigned_count == total_count:
    print("\n✅ ALL AMINO ACID PATHWAYS ASSIGNED!")
else:
    print(f"\n⚠ {total_count - assigned_count} pathways still unassigned")
    unassigned = aa_df[~aa_df["assigned"]]
    for _, row in unassigned.iterrows():
        print(f"  • {row['pathway_id']}: {row['pathway_name'][:50]}")

# Display complete assignment table
print(f"\n{'='*70}")
print("COMPLETE AMINO ACID HARMONIC MAP")
print("="*70)

assigned_df = (
    aa_df[aa_df["assigned"]]
    .sort_values("prevalence", ascending=False)
    .reset_index(drop=True)
)

# Add harmonic properties
def get_props(pid):
    r = HARMONIC_MAP.get_ratio(pid)
    if r:
        return {
            "consonance": r.consonance(),
            "prime_limit": r.prime_limit(),
            "polarity": r.polarity_label(),
        }
    return {}

for col in ["consonance", "prime_limit", "polarity"]:
    assigned_df[col] = assigned_df["pathway_id"].apply(
        lambda pid: get_props(pid).get(col)
    )

display(
    assigned_df[[
        "pathway_id", "pathway_name", "ratio", 
        "consonance", "prime_limit", "prevalence"
    ]]
    
)

# Prime limit distribution
print(f"\n{'='*70}")
print("PRIME LIMIT DISTRIBUTION")
print("="*70)

prime_counts = assigned_df.groupby("prime_limit").size()
for prime, count in prime_counts.items():
    pct = count / len(assigned_df) * 100
    bar = "█" * int(pct / 5)
    print(f"  {prime:>2}-limit: {count:>3} ({pct:>5.1f}%) {bar}")

print(f"\n{'='*70}")
print("✅ AMINO ACID BIOSYNTHESIS COMPLETE")
print("="*70)


🧬 AMINO ACID BIOSYNTHESIS — COMPLETE ASSIGNMENT SUMMARY

Total amino acid pathways: 28
Assigned: 28
Remaining: 0

✅ ALL AMINO ACID PATHWAYS ASSIGNED!

COMPLETE AMINO ACID HARMONIC MAP


,pathway_id,pathway_name,ratio,consonance,prime_limit,prevalence
0,VALSYN-PWY,L-valine biosynthesis,7/2,14,7,0.999178
1,ILEUSYN-PWY,L-isoleucine biosynthesis I (from threonine),7/4,28,7,0.998972
2,BRANCHED-CHAIN-AA-SYN-PWY,superpathway of branched amino acid biosynthesis,7/1,7,7,0.998972
3,PWY-5097,L-lysine biosynthesis VI,7/3,21,7,0.998972
4,ARGSYNBSUB-PWY,L-arginine biosynthesis II (acetyl cycle),21/4,84,7,0.998972
5,PWY-5103,L-isoleucine biosynthesis III,21/8,168,7,0.998972
6,HISTSYN-PWY,L-histidine biosynthesis,5/3,15,5,0.998972
7,ARGSYN-PWY,L-arginine biosynthesis I (via L-ornithine),21/2,42,7,0.998767
8,PWY-2942,L-lysine biosynthesis III,7/6,42,7,0.998767
9,THRESYN-PWY,superpathway of L-threonine biosynthesis,10/3,30,5,0.998767



PRIME LIMIT DISTRIBUTION
   5-limit:   3 ( 10.7%) ██
   7-limit:  25 ( 89.3%) █████████████████

✅ AMINO ACID BIOSYNTHESIS COMPLETE


#### Lipids & Membranes

In [38]:
# ============================================================
# LIPIDS & MEMBRANES — CURATED OVERVIEW (FINAL, COLUMN-SAFE)
# ============================================================

import pandas as pd

# --- Safety checks ---
required_cols = {"pathway_id", "pathway_name", "subcategory", "prevalence"}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"Missing required columns: {missing}")

# --- Optional MS columns ---
MS_STATUS_COL = "ms_status" if "ms_status" in df.columns else None
MS_LDA_COL = "ms_lda_score" if "ms_lda_score" in df.columns else None

# --- Lipid keyword matcher (biological intent) ---
lipid_keywords = [
    "lipid",
    "membrane",
    "fatty",
    "phospho",
    "lps",
    "lipopolysaccharide",
]

pattern = "|".join(lipid_keywords)

lipid_df = (
    df[
        df["subcategory"]
        .astype(str)
        .str.lower()
        .str.contains(pattern)
    ]
    .copy()
)

if lipid_df.empty:
    raise ValueError("No lipid-related pathways found — check subcategory naming.")

# --- Define biological lipid families ---
LIPID_FAMILIES = {
    "Core phospholipids": [
        "PHOSLIPSYN-PWY",
        "PWY-5667",
        "PWY4FS-8",
    ],
    "Fatty acid backbone (FAS II)": [
        "FASYN-INITIAL-PWY",
        "PWY-6284",
        "PWY-6285",
        "PWY0-881",
    ],
    "Saturated & monounsaturated fatty acids": [
        "PWY-5971",
        "PWY-5989",
        "PWY-6282",
        "PWY-5973",
        "PWY-7664",
        "PWY-7663",
        "PWY0-862",
        "PWY-5367",
    ],
    "LPS / outer membrane lipids": [
        "NAGLIPASYN-PWY",
        "KDO-NAGLIPASYN-PWY",
    ],
}

# --- Assign lipid family ---
def assign_lipid_family(pid):
    for family, members in LIPID_FAMILIES.items():
        if pid in members:
            return family
    return "Other / supporting lipids"

lipid_df["lipid_family"] = lipid_df["pathway_id"].apply(assign_lipid_family)

# --- Sort for readability ---
lipid_df = lipid_df.sort_values(
    ["lipid_family", "prevalence"],
    ascending=[True, False]
)

# --- Build display columns dynamically ---
display_cols = [
    "pathway_id",
    "pathway_name",
    "subcategory",
    "lipid_family",
    "prevalence",
    "mean_abundance",
]

if MS_STATUS_COL:
    display_cols.append(MS_STATUS_COL)
if MS_LDA_COL:
    display_cols.append(MS_LDA_COL)

# --- Display ---
display(lipid_df[display_cols])

print()
print(f"🧪 Total lipid & membrane pathways (curated): {len(lipid_df)}")
print("───────────────────────────────────────────")
display(
    lipid_df.groupby("lipid_family")
    .size()
    .rename("pathway_count")
)

,pathway_id,pathway_name,subcategory,lipid_family,prevalence,mean_abundance,ms_status,ms_lda_score
55,PWY-5667,CDP-diacylglycerol biosynthesis I,Fatty Acids/Lipids,Core phospholipids,0.998150,0.000397,depleted_in_MS,0.6500
96,PWY4FS-8,phosphatidylglycerol biosynthesis II (non-plas...,Fatty Acids/Lipids,Core phospholipids,0.995889,0.000125,NaN,NaN
102,PHOSLIPSYN-PWY,superpathway of phospholipid biosynthesis I (b...,Fatty Acids/Lipids,Core phospholipids,0.995067,0.000162,NaN,NaN
152,FASYN-INITIAL-PWY,superpathway of fatty acid biosynthesis initia...,Fatty Acids/Lipids,Fatty acid backbone (FAS II),0.976773,0.000067,NaN,NaN
242,PWY-6284,superpathway of unsaturated fatty acids biosyn...,Fatty Acids/Lipids,Fatty acid backbone (FAS II),0.818911,0.000021,NaN,NaN
316,PWY-6285,superpathway of fatty acids biosynthesis (E. c...,Fatty Acids/Lipids,Fatty acid backbone (FAS II),0.367934,0.000014,NaN,NaN
318,PWY0-881,superpathway of fatty acid biosynthesis I (E. ...,Fatty Acids/Lipids,Fatty acid backbone (FAS II),0.365879,0.000006,NaN,NaN
195,NAGLIPASYN-PWY,lipid IVA biosynthesis,Fatty Acids/Lipids,LPS / outer membrane lipids,0.946763,0.000048,depleted_in_MS,0.8583
348,KDO-NAGLIPASYN-PWY,superpathway of (Kdo)2-lipid A biosynthesis,Fatty Acids/Lipids,LPS / outer membrane lipids,0.296197,0.000002,NaN,NaN
62,PWY-5973,cis-vaccenate biosynthesis,Fatty Acids/Lipids,Saturated & monounsaturated fatty acids,0.998150,0.000243,NaN,NaN



🧪 Total lipid & membrane pathways (curated): 17
───────────────────────────────────────────


lipid_family
Core phospholipids                         3
Fatty acid backbone (FAS II)               4
LPS / outer membrane lipids                2
Saturated & monounsaturated fatty acids    8
Name: pathway_count, dtype: int64

In [39]:
# ------------------------------------------------------------
# PHOSPHOLIPIDS — MEMBRANE MATERIAL (POST-FAS)
# ------------------------------------------------------------
# role: capture and expression of fatty material into membrane boundary
#
# 45/49   (3²×5 / 7²) → entry of fatty material into boundary context
# 90/49   (2×3²×5 / 7²) → bulk membrane phospholipid synthesis
# 135/49  (3³×5 / 7²) → high-activity / adaptive membrane synthesis
# ------------------------------------------------------------

PHOSPHOLIPID_ASSIGNMENTS = {
    'PWY-5667':        (45, 49,  'CDP-diacylglycerol — boundary capture of fatty material'),
    'PHOSLIPSYN-PWY':  (90, 49,  'phospholipid superpathway — bulk membrane synthesis'),
    'PWY4FS-8':        (135, 49, 'phosphatidylglycerol — active membrane expression'),
}

for pid, (n, d, note) in PHOSPHOLIPID_ASSIGNMENTS.items():
    HARMONIC_MAP.assign(
        pid,
        Ratio(n, d),
        {
            'category': 'biosynthesis',
            'subcategory': 'Lipids & Membranes',
            'derivation': note,
        }
    )
    print(f"✓ {pid} → {n}/{d}")

HARMONIC_MAP.save()

✓ PWY-5667 → 45/49
✓ PHOSLIPSYN-PWY → 90/49
✓ PWY4FS-8 → 135/49
✓ Saved 65 assignments to harmonic_map.json


In [40]:
# ============================================================
# BIOSYNTHESIS TOOL — 7-limit consonant ratios (unused)
# Above unison, within range
# ============================================================

def generate_7limit_consonant_unused(
    min_value=1.0,
    max_value=16.0,
    max_consonance=3000
):
    """
    Generate unused 7-limit ratios:
      - value > min_value
      - value <= max_value
      - sorted by consonance (n * d)
      - excludes all currently assigned ratios
    """
    ratios = generate_by_prime_limit(
        limit=7,
        count=None,
        max_consonance=max_consonance,
        polarity=None,
        exclude_ratios=HARMONIC_MAP.used_ratios
    )

    # Filter to range
    ratios = [
        r for r in ratios
        if r.value() > min_value and r.value() <= max_value
    ]

    # Sort by consonance, then by value
    ratios.sort(key=lambda r: (r.consonance(), r.value()))

    return ratios


# === RUN ===
aa_candidate_pool = generate_7limit_consonant_unused(
    min_value=1.0,
    max_value=16.0,
    max_consonance=2500
)

# Preview top candidates
print(f"Total available 7-limit ratios: {len(aa_candidate_pool)}\n")

for r in aa_candidate_pool[:30]:
    print(
        f"{str(r):<6} "
        f"value={r.value():<6.3f} "
        f"n×d={r.consonance():<4} "
        f"pol={r.polarity():+.2f} "
        f"{r.polarity_label()}"
    )

Total available 7-limit ratios: 178

6/5    value=1.200  n×d=30   pol=-0.13 neutral
8/5    value=1.600  n×d=40   pol=-0.50 subharmonic
9/5    value=1.800  n×d=45   pol=+0.15 neutral
25/2   value=12.500 n×d=50   pol=+0.71 harmonic
8/7    value=1.143  n×d=56   pol=-0.50 subharmonic
12/5   value=2.400  n×d=60   pol=-0.11 neutral
20/3   value=6.667  n×d=60   pol=+0.11 neutral
10/7   value=1.429  n×d=70   pol=-0.07 neutral
14/5   value=2.800  n×d=70   pol=+0.07 neutral
16/5   value=3.200  n×d=80   pol=-0.45 subharmonic
12/7   value=1.714  n×d=84   pol=-0.16 neutral
10/9   value=1.111  n×d=90   pol=-0.11 neutral
18/5   value=3.600  n×d=90   pol=+0.11 neutral
25/4   value=6.250  n×d=100  pol=+0.58 harmonic
21/5   value=4.200  n×d=105  pol=+0.31 harmonic
16/7   value=2.286  n×d=112  pol=-0.45 subharmonic
15/8   value=1.875  n×d=120  pol=+0.50 harmonic
24/5   value=4.800  n×d=120  pol=-0.09 neutral
40/3   value=13.333 n×d=120  pol=+0.09 neutral
14/9   value=1.556  n×d=126  pol=-0.04 neutral
27/

In [41]:
# ------------------------------------------------------------
# FATTY ACID SYNTHESIS (FAS II) — Structural Engine (105 Realm)
# ------------------------------------------------------------
# Iterative, constrained machinery for membrane material
#
# 35/4     → Initiation (commitment threshold)
# 105/16   → Elongation (full prime-chain engine)
# 315/62   → Unsaturation (adaptation under constraint)
# 105/32   → Superpathway aggregation (taxon-specific echo)
# ------------------------------------------------------------

FAS_ASSIGNMENTS = {
    'FASYN-INITIAL-PWY': (35, 4,   'FAS initiation — commitment threshold'),
    'PWY-6285':          (105, 16, 'FAS elongation — full 105-realm engine'),
    'PWY-6284':          (175, 32, 'FAS unsaturation — constrained adaptation'),
    'PWY0-881':          (35, 8, 'FAS superpathway — taxon-specific aggregation'),
}

for pid, (n, d, deriv) in FAS_ASSIGNMENTS.items():
    HARMONIC_MAP.assign(pid, Ratio(n, d), {
        'subcategory': 'Fatty Acid Synthesis',
        'category': 'biosynthesis',
        'derivation': deriv,
    })
    print(f"✓ {pid} → {n}/{d}")

HARMONIC_MAP.save()

✓ FASYN-INITIAL-PWY → 35/4
✓ PWY-6285 → 105/16
✓ PWY-6284 → 175/32
✓ PWY0-881 → 35/8
✓ Saved 69 assignments to harmonic_map.json


In [42]:
# ------------------------------------------------------------
# FATTY ACID END‑PRODUCTS & LPS — FINAL ASSIGNMENT
# Material-space (prime‑5 denominators)
# Outer Membrane update: simplified the factorization to represet a clear boundary.
# 35/16 35/32
# ------------------------------------------------------------

LIPID_MATERIAL_ASSIGNMENTS = {

    # Light membrane material (7² base)
    'PWY-7664': (49, 8,   'oleate IV — light monounsaturated membrane'),
    'PWY-6282': (49, 9,  'palmitoleate — viscous monounsaturated variant'),

    # Flexible / curved membrane material (3²×7 base)
    'PWY-5973': (49, 5,   'cis-vaccenate — flexible membrane curvature'),
    'PWY0-862': (49, 25,  'dodecenoate — short-chain flexible lipid'),

    # Dense / rigid membrane material (3×7² base)
    'PWY-7663': (63, 5,  'gondoate — dense long-chain lipid'),
    'PWY-5367': (63, 25, 'petroselinate — heavy monounsaturated lipid'),

    # Saturated fatty acids (highest rigidity)
    'PWY-5989': (147, 40, 'stearate — saturated rigidity anchor'),
    'PWY-5971': (147, 50,  'palmitate — saturated membrane anchor'),

    # Outer membrane / LPS (structural boundary)
    'NAGLIPASYN-PWY':     (35, 16,   'lipid IVA — outer membrane scaffold'),
    'KDO-NAGLIPASYN-PWY': (35, 32, 'Kdo2-lipid A — immune boundary stress'),
}

for pid, (n, d, deriv) in LIPID_MATERIAL_ASSIGNMENTS.items():
    HARMONIC_MAP.assign(
        pid,
        Ratio(n, d),
        {
            'subcategory': 'Fatty Acids / Lipids',
            'category': 'biosynthesis',
            'derivation': deriv,
        }
    )
    print(f"✓ {pid} → {n}/{d}")

HARMONIC_MAP.save()

✓ PWY-7664 → 49/8
✓ PWY-6282 → 49/9
✓ PWY-5973 → 49/5
✓ PWY0-862 → 49/25
✓ PWY-7663 → 63/5
✓ PWY-5367 → 63/25
✓ PWY-5989 → 147/40
✓ PWY-5971 → 147/50
✓ NAGLIPASYN-PWY → 35/16
✓ KDO-NAGLIPASYN-PWY → 35/32
✓ Saved 79 assignments to harmonic_map.json


In [43]:
# ============================================================
# LIPIDS — COMPLETE SUMMARY
# ============================================================

print("=" * 70)
print("🧫 LIPID METABOLISM — COMPLETE ASSIGNMENT SUMMARY")
print("=" * 70)

# Get all lipid-related pathways
lipid_df = (
    biosynthesis_df[
        biosynthesis_df["subcategory"]
        .astype(str)
        .str.lower()
        .str.contains("lipid|fatty")
    ]
    .copy()
)

# Check assignment status
lipid_df["assigned"] = lipid_df["pathway_id"].apply(HARMONIC_MAP.is_assigned)
lipid_df["ratio"] = lipid_df["pathway_id"].apply(
    lambda pid: str(HARMONIC_MAP.get_ratio(pid)) if HARMONIC_MAP.is_assigned(pid) else None
)

assigned_count = lipid_df["assigned"].sum()
total_count = len(lipid_df)

print(f"\nTotal lipid pathways: {total_count}")
print(f"Assigned: {assigned_count}")
print(f"Remaining: {total_count - assigned_count}")

if assigned_count == total_count:
    print("\n✅ ALL LIPID PATHWAYS ASSIGNED!")
else:
    print(f"\n⚠ {total_count - assigned_count} pathways still unassigned")
    unassigned = lipid_df[~lipid_df["assigned"]]
    for _, row in unassigned.iterrows():
        print(f"  • {row['pathway_id']}: {row['pathway_name'][:50]}")

# ------------------------------------------------------------
# COMPLETE LIPID HARMONIC MAP
# ------------------------------------------------------------
print(f"\n{'='*70}")
print("COMPLETE LIPID HARMONIC MAP")
print("="*70)

assigned_df = (
    lipid_df[lipid_df["assigned"]]
    .sort_values("prevalence", ascending=False)
    .reset_index(drop=True)
)

# Add harmonic properties
def get_props(pid):
    r = HARMONIC_MAP.get_ratio(pid)
    if r:
        return {
            "consonance": r.consonance(),
            "prime_limit": r.prime_limit(),
            "polarity": r.polarity_label(),
        }
    return {}

for col in ["consonance", "prime_limit", "polarity"]:
    assigned_df[col] = assigned_df["pathway_id"].apply(
        lambda pid: get_props(pid).get(col)
    )

display(
    assigned_df[[
        "pathway_id",
        "pathway_name",
        "subcategory",
        "ratio",
        "consonance",
        "prime_limit",
        "prevalence"
    ]]
)

# ------------------------------------------------------------
# PRIME LIMIT DISTRIBUTION (LIPIDS)
# ------------------------------------------------------------
print(f"\n{'='*70}")
print("LIPID PRIME LIMIT DISTRIBUTION")
print("="*70)

prime_counts = assigned_df.groupby("prime_limit").size()
for prime, count in prime_counts.items():
    pct = count / len(assigned_df) * 100
    bar = "█" * int(pct / 5)
    print(f"  {prime:>2}-limit: {count:>3} ({pct:>5.1f}%) {bar}")

print(f"\n{'='*70}")
print("✅ LIPID METABOLISM COMPLETE")
print("="*70)

🧫 LIPID METABOLISM — COMPLETE ASSIGNMENT SUMMARY

Total lipid pathways: 17
Assigned: 17
Remaining: 0

✅ ALL LIPID PATHWAYS ASSIGNED!

COMPLETE LIPID HARMONIC MAP


,pathway_id,pathway_name,subcategory,ratio,consonance,prime_limit,prevalence
0,PWY-5667,CDP-diacylglycerol biosynthesis I,Fatty Acids/Lipids,45/49,2205,7,0.998150
1,PWY-5973,cis-vaccenate biosynthesis,Fatty Acids/Lipids,49/5,245,7,0.998150
2,PWY-7663,gondoate biosynthesis (anaerobic),Fatty Acids/Lipids,63/5,315,7,0.998150
3,PWY4FS-8,phosphatidylglycerol biosynthesis II (non-plas...,Fatty Acids/Lipids,135/49,6615,7,0.995889
4,PHOSLIPSYN-PWY,superpathway of phospholipid biosynthesis I (b...,Fatty Acids/Lipids,90/49,4410,7,0.995067
5,PWY-5989,stearate biosynthesis II (bacteria and plants),Fatty Acids/Lipids,147/40,5880,7,0.982323
6,PWY0-862,(5Z)-dodec-5-enoate biosynthesis,Fatty Acids/Lipids,49/25,1225,7,0.977184
7,FASYN-INITIAL-PWY,superpathway of fatty acid biosynthesis initia...,Fatty Acids/Lipids,35/4,140,7,0.976773
8,PWY-7664,oleate biosynthesis IV (anaerobic),Fatty Acids/Lipids,49/8,392,7,0.976567
9,PWY-6282,palmitoleate biosynthesis I (from (5Z)-dodec-5...,Fatty Acids/Lipids,49/9,441,7,0.976567



LIPID PRIME LIMIT DISTRIBUTION
   7-limit:  17 (100.0%) ████████████████████

✅ LIPID METABOLISM COMPLETE


#### Cofactors/Vitamins

In [44]:
# ============================================================
# BIOSYNTHESIS — REMAINING WORK OVERVIEW
# ============================================================

print("=" * 80)
print("🧬 BIOSYNTHESIS — REMAINING ASSIGNMENT OVERVIEW")
print("=" * 80)

# Work only within biosynthesis
bio_df = biosynthesis_df.copy()

# Assignment status
bio_df["assigned"] = bio_df["pathway_id"].apply(HARMONIC_MAP.is_assigned)
bio_df["ratio"] = bio_df["pathway_id"].apply(
    lambda pid: str(HARMONIC_MAP.get_ratio(pid)) if HARMONIC_MAP.is_assigned(pid) else None
)

total_count = len(bio_df)
assigned_count = bio_df["assigned"].sum()
remaining_count = total_count - assigned_count

print(f"\nTotal biosynthesis pathways: {total_count}")
print(f"Assigned: {assigned_count}")
print(f"Remaining: {remaining_count}")

# ------------------------------------------------------------
# SUMMARY BY SUBCATEGORY
# ------------------------------------------------------------
print(f"\n{'='*80}")
print("REMAINING BIOSYNTHESIS PATHWAYS BY SUBCATEGORY")
print("="*80)

remaining_df = bio_df[~bio_df["assigned"]]

if remaining_df.empty:
    print("\n✅ ALL BIOSYNTHESIS PATHWAYS ASSIGNED!")
else:
    subcat_counts = (
        remaining_df
        .groupby("subcategory")
        .agg(
            remaining=("pathway_id", "size"),
            mean_prevalence=("prevalence", "mean"),
        )
        .sort_values("remaining", ascending=False)
    )

    for subcat, row in subcat_counts.iterrows():
        print(
            f"• {subcat:<30} "
            f"{int(row['remaining']):>3} remaining "
            f"(mean prev: {row['mean_prevalence']:.3f})"
        )

# ------------------------------------------------------------
# DETAILED LIST (TOP BY PREVALENCE)
# ------------------------------------------------------------
print(f"\n{'='*80}")
print("TOP UNASSIGNED BIOSYNTHESIS PATHWAYS (BY PREVALENCE)")
print("="*80)

if not remaining_df.empty:
    display(
        remaining_df
        .sort_values("prevalence", ascending=False)
        .reset_index(drop=True)[[
            "pathway_id",
            "pathway_name",
            "subcategory",
            "prevalence",
            "mean_abundance",
        ]]
    )

print(f"\n{'='*80}")
print("🔎 BIOSYNTHESIS REMAINING — READY FOR NEXT COMPOSITION PASS")
print("="*80)

🧬 BIOSYNTHESIS — REMAINING ASSIGNMENT OVERVIEW

Total biosynthesis pathways: 143
Assigned: 52
Remaining: 91

REMAINING BIOSYNTHESIS PATHWAYS BY SUBCATEGORY
• Cofactors/Vitamins              28 remaining (mean prev: 0.710)
• Cell Envelope                   24 remaining (mean prev: 0.692)
• Specialized                     14 remaining (mean prev: 0.431)
• Other                           12 remaining (mean prev: 0.492)
• Isoprenoids                      7 remaining (mean prev: 0.673)
• Polyamines                       6 remaining (mean prev: 0.502)

TOP UNASSIGNED BIOSYNTHESIS PATHWAYS (BY PREVALENCE)


,pathway_id,pathway_name,subcategory,prevalence,mean_abundance
0,DTDPRHAMSYN-PWY,dTDP-L-rhamnose biosynthesis I,Cell Envelope,0.999383,5.795910e-04
1,PWY-6386,UDP-N-acetylmuramoyl-pentapeptide biosynthesis...,Cell Envelope,0.999178,5.476870e-04
2,PWY-6700,queuosine biosynthesis,Other,0.999178,4.618380e-04
3,COA-PWY,coenzyme A biosynthesis I,Cofactors/Vitamins,0.999178,4.141540e-04
4,PWY-6387,UDP-N-acetylmuramoyl-pentapeptide biosynthesis...,Cell Envelope,0.998972,5.439090e-04
...,...,...,...,...,...
86,PWY-6351,"D-myo-inositol (1,4,5)-trisphosphate biosynthesis",Other,0.000822,4.740000e-10
87,PWY-7255,ergothioneine biosynthesis I (bacteria),Specialized,0.000822,1.020000e-09
88,PWY-5508,adenosylcobalamin biosynthesis from cobyrinate...,Cofactors/Vitamins,0.000411,2.300000e-09
89,PWY-5507,adenosylcobalamin biosynthesis I (early cobalt...,Cofactors/Vitamins,0.000206,2.950000e-10



🔎 BIOSYNTHESIS REMAINING — READY FOR NEXT COMPOSITION PASS


In [45]:
#### Cofactors/Vitamins

In [46]:
# ============================================================
# BIOSYNTHESIS — COFACTORS / VITAMINS : FULL SITUATION ASSESSMENT
# ============================================================

print("=" * 90)
print("🧬 BIOSYNTHESIS — COFACTORS / VITAMINS : FULL DATA OVERVIEW")
print("=" * 90)

# ------------------------------------------------------------
# Select cofactor / vitamin pathways
# ------------------------------------------------------------
cofactor_df = biosynthesis_df[
    biosynthesis_df["subcategory"]
    .astype(str)
    .str.lower()
    .str.contains("cofactor|vitamin")
].copy()

# Assignment status
cofactor_df["assigned"] = cofactor_df["pathway_id"].apply(HARMONIC_MAP.is_assigned)
cofactor_df["ratio"] = cofactor_df["pathway_id"].apply(
    lambda pid: str(HARMONIC_MAP.get_ratio(pid)) if HARMONIC_MAP.is_assigned(pid) else None
)

total = len(cofactor_df)
assigned = cofactor_df["assigned"].sum()
remaining = total - assigned

print(f"\nTotal Cofactors/Vitamins pathways: {total}")
print(f"Assigned: {assigned}")
print(f"Remaining: {remaining}")

# ------------------------------------------------------------
# Summary by SUBCATEGORY (as labeled in your data)
# ------------------------------------------------------------
print(f"\n{'='*90}")
print("SUBCATEGORY BREAKDOWN")
print("="*90)

subcat_summary = (
    cofactor_df
    .groupby("subcategory")
    .agg(
        total=("pathway_id", "size"),
        assigned=("assigned", "sum"),
        remaining=("assigned", lambda x: (~x).sum()),
        mean_prev=("prevalence", "mean"),
        median_prev=("prevalence", "median"),
    )
    .sort_values("remaining", ascending=False)
)

display(subcat_summary)

# ------------------------------------------------------------
# Prevalence-ordered list (THIS is your priority signal)
# ------------------------------------------------------------
print(f"\n{'='*90}")
print("COFACTORS / VITAMINS — SORTED BY PREVALENCE")
print("="*90)

display(
    cofactor_df
    .sort_values("prevalence", ascending=False)
    .reset_index(drop=True)[[
        "pathway_id",
        "pathway_name",
        "subcategory",
        "prevalence",
        "mean_abundance",
        "assigned",
        "ratio",
    ]]
)

# ------------------------------------------------------------
# Assigned vs Unassigned (explicit lists)
# ------------------------------------------------------------
print(f"\n{'='*90}")
print("UNASSIGNED COFACTORS / VITAMINS (ACTION REQUIRED)")
print("="*90)

unassigned_df = cofactor_df[~cofactor_df["assigned"]]

if unassigned_df.empty:
    print("\n✅ ALL COFACTORS / VITAMINS ASSIGNED!")
else:
    display(
        unassigned_df
        .sort_values("prevalence", ascending=False)
        .reset_index(drop=True)[[
            "pathway_id",
            "pathway_name",
            "subcategory",
            "prevalence",
            "mean_abundance",
        ]]
    )

print(f"\n{'='*90}")
print("✅ COFACTORS / VITAMINS — DATA FULLY ACCOUNTED FOR")
print("="*90)

🧬 BIOSYNTHESIS — COFACTORS / VITAMINS : FULL DATA OVERVIEW

Total Cofactors/Vitamins pathways: 28
Assigned: 0
Remaining: 28

SUBCATEGORY BREAKDOWN


,total,assigned,remaining,mean_prev,median_prev
subcategory,,,,,
Cofactors/Vitamins,28,0,28,0.710351,0.960021



COFACTORS / VITAMINS — SORTED BY PREVALENCE


,pathway_id,pathway_name,subcategory,prevalence,mean_abundance,assigned,ratio
0,COA-PWY,coenzyme A biosynthesis I,Cofactors/Vitamins,0.999178,4.141540e-04,False,None
1,PWY-4242,pantothenate and coenzyme A biosynthesis III,Cofactors/Vitamins,0.998972,4.117360e-04,False,None
2,1CMET2-PWY,N10-formyl-tetrahydrofolate biosynthesis,Cofactors/Vitamins,0.998767,4.067540e-04,False,None
3,PYRIDNUCSYN-PWY,NAD biosynthesis I (from aspartate),Cofactors/Vitamins,0.998561,2.933960e-04,False,None
4,RIBOSYN2-PWY,flavin biosynthesis I (bacteria and plants),Cofactors/Vitamins,0.997739,3.645790e-04,False,None
5,PANTOSYN-PWY,pantothenate and coenzyme A biosynthesis I,Cofactors/Vitamins,0.997533,3.448380e-04,False,None
6,PWY-6892,thiazole biosynthesis I (E. coli),Cofactors/Vitamins,0.995067,1.795550e-04,False,None
7,PWY-6147,6-hydroxymethyl-dihydropterin diphosphate bios...,Cofactors/Vitamins,0.994861,1.710280e-04,False,None
8,PWY-5188,tetrapyrrole biosynthesis I (from glutamate),Cofactors/Vitamins,0.994656,1.319380e-04,False,None
9,THISYN-PWY,superpathway of thiamin diphosphate biosynthes...,Cofactors/Vitamins,0.994039,1.962300e-04,False,None



UNASSIGNED COFACTORS / VITAMINS (ACTION REQUIRED)


,pathway_id,pathway_name,subcategory,prevalence,mean_abundance
0,COA-PWY,coenzyme A biosynthesis I,Cofactors/Vitamins,0.999178,4.141540e-04
1,PWY-4242,pantothenate and coenzyme A biosynthesis III,Cofactors/Vitamins,0.998972,4.117360e-04
2,1CMET2-PWY,N10-formyl-tetrahydrofolate biosynthesis,Cofactors/Vitamins,0.998767,4.067540e-04
3,PYRIDNUCSYN-PWY,NAD biosynthesis I (from aspartate),Cofactors/Vitamins,0.998561,2.933960e-04
4,RIBOSYN2-PWY,flavin biosynthesis I (bacteria and plants),Cofactors/Vitamins,0.997739,3.645790e-04
5,PANTOSYN-PWY,pantothenate and coenzyme A biosynthesis I,Cofactors/Vitamins,0.997533,3.448380e-04
6,PWY-6892,thiazole biosynthesis I (E. coli),Cofactors/Vitamins,0.995067,1.795550e-04
7,PWY-6147,6-hydroxymethyl-dihydropterin diphosphate bios...,Cofactors/Vitamins,0.994861,1.710280e-04
8,PWY-5188,tetrapyrrole biosynthesis I (from glutamate),Cofactors/Vitamins,0.994656,1.319380e-04
9,THISYN-PWY,superpathway of thiamin diphosphate biosynthes...,Cofactors/Vitamins,0.994039,1.962300e-04



✅ COFACTORS / VITAMINS — DATA FULLY ACCOUNTED FOR


In [47]:
# ------------------------------------------------------------
# NAD / REDOX CURRENCY (11-limit introduction)
# ------------------------------------------------------------
# 11/1   → Main NAD pathway (foundational)
# 121(11x11)/8  → Rare NAD variant (derived, complex)
# ------------------------------------------------------------

NAD_ASSIGNMENTS = {
    'PYRIDNUCSYN-PWY': (11, 1,   'NAD biosynthesis I — 11-limit foundation'),
    'NADSYN-PWY':      (121, 8,  'NAD biosynthesis II — 11-limit variant'),
}

for pid, (n, d, deriv) in NAD_ASSIGNMENTS.items():
    HARMONIC_MAP.assign(
        pid,
        Ratio(n, d),
        {
            'subcategory': 'Cofactors/Vitamins',
            'category': 'biosynthesis',
            'derivation': deriv,
        }
    )
    print(f"✓ {pid} → {n}/{d}")

HARMONIC_MAP.save()

✓ PYRIDNUCSYN-PWY → 11/1
✓ NADSYN-PWY → 121/8
✓ Saved 81 assignments to harmonic_map.json


In [48]:
# ------------------------------------------------------------
# COENZYME A / PANTOTHENATE (11-limit octave series)
# ------------------------------------------------------------
# Extends the 11/1 NAD identity into functional registers
# for acyl transfer.
# ------------------------------------------------------------

COA_ASSIGNMENTS = {
    'COA-PWY':      (11, 2, 'CoA biosynthesis I — 11-limit octave 1'),
    'PWY-4242':     (11, 4, 'CoA biosynthesis III — 11-limit octave 2'),
    'PANTOSYN-PWY': (11, 8, 'CoA biosynthesis I — 11-limit octave 3'),
}

for pid, (n, d, deriv) in COA_ASSIGNMENTS.items():
    HARMONIC_MAP.assign(
        pid,
        Ratio(n, d),
        {
            'subcategory': 'Cofactors/Vitamins',
            'category': 'biosynthesis',
            'derivation': deriv,
        }
    )
    print(f"✓ {pid} → {n}/{d}")

HARMONIC_MAP.save()

✓ COA-PWY → 11/2
✓ PWY-4242 → 11/4
✓ PANTOSYN-PWY → 11/8
✓ Saved 84 assignments to harmonic_map.json


In [49]:
# ------------------------------------------------------------
# THIAMINE / B1 (11-limit over prime 3)
# ------------------------------------------------------------
# The "pyruvate gateway" cofactors, with a sense of flow.
# Superpathways are octave-paired; precursors are octave-paired.
# ------------------------------------------------------------

THIAMINE_ASSIGNMENTS = {
    'THISYN-PWY': (11, 3, 'Thiamine main superpathway — 11/3 anchor'),
    'PWY-6895':   (11, 6, 'Thiamine variant superpathway — octave echo'),
    'PWY-6892':   (33, 4, 'Thiazole precursor I — building block'),
    'PWY-6891':   (33, 8, 'Thiazole precursor II — building block echo'),
}

for pid, (n, d, deriv) in THIAMINE_ASSIGNMENTS.items():
    HARMONIC_MAP.assign(
        pid,
        Ratio(n, d),
        {
            'subcategory': 'Cofactors/Vitamins',
            'category': 'biosynthesis',
            'derivation': deriv,
        }
    )
    print(f"✓ {pid} → {n}/{d}")

HARMONIC_MAP.save()

✓ THISYN-PWY → 11/3
✓ PWY-6895 → 11/6
✓ PWY-6892 → 33/4
✓ PWY-6891 → 33/8
✓ Saved 88 assignments to harmonic_map.json


In [50]:
# ------------------------------------------------------------
# ONE-CARBON / FOLATE (11-limit over prime 5)
# ------------------------------------------------------------
# The regulatory backbone for purine/DNA synthesis.
# Mapped as a clean octave pair.
# ------------------------------------------------------------

FOLATE_ASSIGNMENTS = {
    '1CMET2-PWY': (11, 5,  'Active folate form — 11/5 anchor'),
    'PWY-6147':   (11, 10, 'Folate precursor — octave echo'),
}

for pid, (n, d, deriv) in FOLATE_ASSIGNMENTS.items():
    HARMONIC_MAP.assign(
        pid,
        Ratio(n, d),
        {
            'subcategory': 'Cofactors/Vitamins',
            'category': 'biosynthesis',
            'derivation': deriv,
        }
    )
    print(f"✓ {pid} → {n}/{d}")

HARMONIC_MAP.save()

✓ 1CMET2-PWY → 11/5
✓ PWY-6147 → 11/10
✓ Saved 90 assignments to harmonic_map.json


In [51]:
# ============================================================
# STATUS CHECK — COFACTORS / VITAMINS
# Schema-correct, defensive, read-only
# ============================================================

SECTION = "Cofactors/Vitamins"

print(f"=== STATUS: {SECTION} ===")

# ------------------------------------------------------------
# Subset dataframe
# ------------------------------------------------------------
cofactor_df = df[df["subcategory"] == SECTION].copy()

print(f"Total pathways in section: {len(cofactor_df)}")
print()

# ------------------------------------------------------------
# Assignment state
# ------------------------------------------------------------
assigned = []
unassigned = []

for pid in cofactor_df["pathway_id"]:
    if HARMONIC_MAP.is_assigned(pid):
        assigned.append(pid)
    else:
        unassigned.append(pid)

print(f"Assigned pathways:   {len(assigned)}")
print(f"Unassigned pathways: {len(unassigned)}")
print()

# ------------------------------------------------------------
# Helper: safe column selection
# ------------------------------------------------------------
def safe_cols(df, cols):
    return [c for c in cols if c in df.columns]

# ------------------------------------------------------------
# ASSIGNED pathways with harmonic metadata
# ------------------------------------------------------------
print("=== ASSIGNED COFACTORS / VITAMINS ===")

if assigned:
    rows = []
    for pid in assigned:
        r = HARMONIC_MAP.get_ratio(pid)
        rows.append({
            "pathway_id": pid,
            "ratio": str(r),
            "polarity": round(r.polarity(), 3),
            "consonance (n×d)": r.consonance(),
            "prime_limit": r.prime_limit(),
        })

    display(
        cofactor_df[cofactor_df["pathway_id"].isin(assigned)][
            safe_cols(
                cofactor_df,
                ["pathway_id", "pathway_name", "prevalence", "mean_abundance",
                 "ms_status", "ms_lda_score"]
            )
        ]
        .merge(pd.DataFrame(rows), on="pathway_id")
        .sort_values(["prime_limit", "consonance (n×d)"])
    )
else:
    print("None assigned yet.")
print()

# ------------------------------------------------------------
# FOLATE / ONE-CARBON SPOTLIGHT
# ------------------------------------------------------------
print("=== FOLATE / ONE-CARBON PATHWAYS ===")

folate_ids = [
    "1CMET2-PWY",   # N10-formyl-THF (one-carbon metabolism)
    "PWY-6147",     # HMDP diphosphate (folate precursor)
]

folate_df = cofactor_df[cofactor_df["pathway_id"].isin(folate_ids)]

if len(folate_df) == 0:
    print("No folate-related pathways found (unexpected).")
else:
    rows = []
    for pid in folate_df["pathway_id"]:
        if HARMONIC_MAP.is_assigned(pid):
            r = HARMONIC_MAP.get_ratio(pid)
            rows.append({
                "pathway_id": pid,
                "ratio": str(r),
                "polarity": round(r.polarity(), 3),
                "consonance (n×d)": r.consonance(),
                "prime_limit": r.prime_limit(),
            })
        else:
            rows.append({
                "pathway_id": pid,
                "ratio": None,
                "polarity": None,
                "consonance (n×d)": None,
                "prime_limit": None,
            })

    display(
        folate_df[
            safe_cols(
                folate_df,
                ["pathway_id", "pathway_name", "prevalence", "mean_abundance",
                 "ms_status", "ms_lda_score"]
            )
        ].merge(pd.DataFrame(rows), on="pathway_id", how="left")
    )
print()

# ------------------------------------------------------------
# UNASSIGNED pathways (planning view)
# ------------------------------------------------------------
print("=== UNASSIGNED COFACTORS / VITAMINS (PLANNING) ===")

if unassigned:
    display(
        cofactor_df[cofactor_df["pathway_id"].isin(unassigned)]
        .sort_values("prevalence", ascending=False)
    )
else:
    print("All cofactors/vitamins are assigned.")
print()

# ------------------------------------------------------------
# Harmonic footprint summary
# ------------------------------------------------------------
print("=== HARMONIC FOOTPRINT — COFACTORS / VITAMINS ===")

for pid in assigned:
    r = HARMONIC_MAP.get_ratio(pid)
    print(
        f"{pid:15s}  {str(r):>8s}  "
        f"pol={r.polarity():+.3f}  "
        f"n×d={r.consonance():4d}  "
        f"limit={r.prime_limit()}"
    )

=== STATUS: Cofactors/Vitamins ===
Total pathways in section: 28

Assigned pathways:   11
Unassigned pathways: 17

=== ASSIGNED COFACTORS / VITAMINS ===


,pathway_id,pathway_name,prevalence,mean_abundance,ms_status,ms_lda_score,ratio,polarity,consonance (n×d),prime_limit
3,PYRIDNUCSYN-PWY,NAD biosynthesis I (from aspartate),0.998561,2.933960e-04,depleted_in_MS,0.6500,11/1,1.000,11,11
0,COA-PWY,coenzyme A biosynthesis I,0.999178,4.141540e-04,elevated_in_MS,0.5000,11/2,0.707,22,11
7,THISYN-PWY,superpathway of thiamin diphosphate biosynthes...,0.994039,1.962300e-04,NaN,NaN,11/3,0.372,33,11
1,PWY-4242,pantothenate and coenzyme A biosynthesis III,0.998972,4.117360e-04,NaN,NaN,11/4,0.577,44,11
2,1CMET2-PWY,N10-formyl-tetrahydrofolate biosynthesis,0.998767,4.067540e-04,NaN,NaN,11/5,0.197,55,11
9,PWY-6895,superpathway of thiamin diphosphate biosynthes...,0.960329,9.220000e-05,depleted_in_MS,1.5538,11/6,0.263,66,11
4,PANTOSYN-PWY,pantothenate and coenzyme A biosynthesis I,0.997533,3.448380e-04,elevated_in_MS,0.6000,11/8,0.500,88,11
6,PWY-6147,6-hydroxymethyl-dihydropterin diphosphate bios...,0.994861,1.710280e-04,NaN,NaN,11/10,0.139,110,11
5,PWY-6892,thiazole biosynthesis I (E. coli),0.995067,1.795550e-04,NaN,NaN,33/4,0.577,132,11
8,PWY-6891,thiazole biosynthesis II (Bacillus),0.963001,4.050000e-05,NaN,NaN,33/8,0.500,264,11



=== FOLATE / ONE-CARBON PATHWAYS ===


,pathway_id,pathway_name,prevalence,mean_abundance,ms_status,ms_lda_score,ratio,polarity,consonance (n×d),prime_limit
0,1CMET2-PWY,N10-formyl-tetrahydrofolate biosynthesis,0.998767,0.000407,NaN,NaN,11/5,0.197,55,11
1,PWY-6147,6-hydroxymethyl-dihydropterin diphosphate bios...,0.994861,0.000171,NaN,NaN,11/10,0.139,110,11



=== UNASSIGNED COFACTORS / VITAMINS (PLANNING) ===


,rank,pathway_id,pathway_name,category,subcategory,prevalence,prevalence_pct,tier,median_abundance,mean_abundance,ms_status,ms_lda_score
67,68,RIBOSYN2-PWY,flavin biosynthesis I (bacteria and plants),biosynthesis,Cofactors/Vitamins,0.997739,99.77%,UNIVERSAL,0.000346,3.645790e-04,NaN,NaN
108,109,PWY-5188,tetrapyrrole biosynthesis I (from glutamate),biosynthesis,Cofactors/Vitamins,0.994656,99.47%,UNIVERSAL,0.000129,1.319380e-04,NaN,NaN
168,169,PWY-6519,8-amino-7-oxononanoate biosynthesis I,biosynthesis,Cofactors/Vitamins,0.967523,96.75%,VERY_COMMON,0.000051,6.400000e-05,NaN,NaN
170,171,BIOTIN-BIOSYNTHESIS-PWY,biotin biosynthesis I,biosynthesis,Cofactors/Vitamins,0.967112,96.71%,VERY_COMMON,0.000057,6.930000e-05,NaN,NaN
184,185,PYRIDOXSYN-PWY,pyridoxal 5'-phosphate biosynthesis I,biosynthesis,Cofactors/Vitamins,0.959712,95.97%,VERY_COMMON,0.000055,7.740000e-05,NaN,NaN
198,199,HEMESYN2-PWY,heme biosynthesis II (anaerobic),biosynthesis,Cofactors/Vitamins,0.941418,94.14%,COMMON,0.000011,1.460000e-05,NaN,NaN
211,212,PWY-5005,biotin biosynthesis II,biosynthesis,Cofactors/Vitamins,0.914080,91.41%,COMMON,0.000016,2.040000e-05,NaN,NaN
237,238,HEME-BIOSYNTHESIS-II,heme biosynthesis I (aerobic),biosynthesis,Cofactors/Vitamins,0.829599,82.96%,MODERATE,0.000005,9.180000e-06,NaN,NaN
238,239,PWY-5189,tetrapyrrole biosynthesis II (from glycine),biosynthesis,Cofactors/Vitamins,0.828777,82.88%,MODERATE,0.000005,1.120000e-05,NaN,NaN
253,254,PWY-5918,superpathay of heme biosynthesis from glutamate,biosynthesis,Cofactors/Vitamins,0.730319,73.03%,MODERATE,0.000005,1.090000e-05,NaN,NaN



=== HARMONIC FOOTPRINT — COFACTORS / VITAMINS ===
COA-PWY              11/2  pol=+0.707  n×d=  22  limit=11
PWY-4242             11/4  pol=+0.577  n×d=  44  limit=11
1CMET2-PWY           11/5  pol=+0.197  n×d=  55  limit=11
PYRIDNUCSYN-PWY      11/1  pol=+1.000  n×d=  11  limit=11
PANTOSYN-PWY         11/8  pol=+0.500  n×d=  88  limit=11
PWY-6892             33/4  pol=+0.577  n×d= 132  limit=11
PWY-6147            11/10  pol=+0.139  n×d= 110  limit=11
THISYN-PWY           11/3  pol=+0.372  n×d=  33  limit=11
PWY-6891             33/8  pol=+0.500  n×d= 264  limit=11
PWY-6895             11/6  pol=+0.263  n×d=  66  limit=11
NADSYN-PWY          121/8  pol=+0.500  n×d= 968  limit=11


In [52]:
# ------------------------------------------------------------
# FLAVIN (RIBOFLAVIN → FMN / FAD)
# ------------------------------------------------------------
# Structural redox cofactor.
# Enzyme‑bound, not free currency.
# Adjacent to NAD in redox space.
#
# 11/9 → Undecimal functional plane
#        Denominator 9 = 3×3 (attenuated process)
# ------------------------------------------------------------

FLAVIN_ASSIGNMENTS = {
    'RIBOSYN2-PWY': (11, 9, 'Flavin biosynthesis — structural redox cofactor adjacent to NAD (11/9 = 3×3)'),
}

for pid, (n, d, deriv) in FLAVIN_ASSIGNMENTS.items():
    HARMONIC_MAP.assign(
        pid,
        Ratio(n, d),
        {
            'subcategory': 'Cofactors/Vitamins',
            'category': 'biosynthesis',
            'derivation': deriv,
        }
    )
    print(f"✓ {pid} → {n}/{d}")

HARMONIC_MAP.save()

✓ RIBOSYN2-PWY → 11/9
✓ Saved 91 assignments to harmonic_map.json


In [53]:
# ------------------------------------------------------------
# TETRAPYRROLE → HEME (STRUCTURAL → FUNCTIONAL TRANSITION)
# ------------------------------------------------------------
# Tetrapyrrole = macrocyclic scaffold
# Heme = metal‑inserted, functionalized cofactor
#
# Two entry chemistries:
#   Glutamate → broader, infrastructural (33‑family)
#   Glycine   → tighter, recursive (99‑family)
#
# Note: 99‑ratios are octave‑adjusted for human hearing,
#       starting at /10 rather than /5.
# ------------------------------------------------------------

HEME_ASSIGNMENTS = {
    # --- Tetrapyrrole scaffold (pre‑heme) ---
    'PWY-5188': (33, 5,   'Tetrapyrrole scaffold (from glutamate) — structural macrocycle'),
    'PWY-5189': (99, 10,  'Tetrapyrrole scaffold (from glycine) — recursive structural macrocycle'),

    # --- Heme (functionalized cofactor) ---
    'HEMESYN2-PWY':          (33, 10, 'Heme biosynthesis (anaerobic) — functionalized cofactor'),
    'HEME-BIOSYNTHESIS-II':  (99, 20, 'Heme biosynthesis (aerobic) — functionalized cofactor'),

    # --- Superpathways (Type A packaging / octave echoes) ---
    'PWY-5918': (33, 20, 'Heme superpathway (from glutamate) — packaged / aggregate'),
    'PWY-5920': (99, 40, 'Heme superpathway (from glycine) — packaged / aggregate'),
}

for pid, (n, d, deriv) in HEME_ASSIGNMENTS.items():
    HARMONIC_MAP.assign(
        pid,
        Ratio(n, d),
        {
            'subcategory': 'Cofactors/Vitamins',
            'category': 'biosynthesis',
            'derivation': deriv,
        }
    )
    print(f"✓ {pid} → {n}/{d}")

HARMONIC_MAP.save()

✓ PWY-5188 → 33/5
✓ PWY-5189 → 99/10
✓ HEMESYN2-PWY → 33/10
✓ HEME-BIOSYNTHESIS-II → 99/20
✓ PWY-5918 → 33/20
✓ PWY-5920 → 99/40
✓ Saved 97 assignments to harmonic_map.json


In [54]:
# ------------------------------------------------------------
# VITAMIN B6 (PYRIDOXAL 5'-PHOSPHATE)
# ------------------------------------------------------------
# Aminotransferase cofactor.
# Embedded in amino acid metabolism.
# Functional (11‑limit) under identity constraint (7‑limit).
# ------------------------------------------------------------

B6_ASSIGNMENTS = {
    'PYRIDOXSYN-PWY': (11, 7, 'Vitamin B6 (PLP) biosynthesis — aminotransferase cofactor (11/7)'),
}

for pid, (n, d, deriv) in B6_ASSIGNMENTS.items():
    HARMONIC_MAP.assign(
        pid,
        Ratio(n, d),
        {
            'subcategory': 'Cofactors/Vitamins',
            'category': 'biosynthesis',
            'derivation': deriv,
        }
    )
    print(f"✓ {pid} → {n}/{d}")

HARMONIC_MAP.save()

✓ PYRIDOXSYN-PWY → 11/7
✓ Saved 98 assignments to harmonic_map.json


In [55]:
# ------------------------------------------------------------
# BIOTIN (VITAMIN B7) — SUBHARMONIC PERMISSIVE COFACTOR
# ------------------------------------------------------------
# Biotin enables carboxylation reactions and material expansion,
# particularly fatty‑acid synthesis.
#
# Ontology:
# - Subharmonic (gives way, enables)
# - Undecimal (cofactor plane)
# - Septimal context (lipid / membrane adjacency)
#
# Represented as a septimal undecimal family with increasing
# structural commitment.
# ------------------------------------------------------------

BIOTIN_ASSIGNMENTS = {
    # Canonical biotin biosynthesis — flexible material enabler
    'BIOTIN-BIOSYNTHESIS-PWY': (21, 11, 'Biotin biosynthesis — permissive lipid‑adjacent cofactor'),

    # Alternate route — increased material capacity
    'PWY-5005': (35, 11, 'Biotin biosynthesis (alternate) — increased material capacity'),

    # Precursor — dense structural commitment
    'PWY-6519': (49, 11, 'Biotin precursor biosynthesis — dense septimal scaffold'),
}

for pid, (n, d, deriv) in BIOTIN_ASSIGNMENTS.items():
    HARMONIC_MAP.assign(
        pid,
        Ratio(n, d),
        {
            'subcategory': 'Cofactors/Vitamins',
            'category': 'biosynthesis',
            'derivation': deriv,
        }
    )
    print(f"✓ {pid} → {n}/{d}")

HARMONIC_MAP.save()

✓ BIOTIN-BIOSYNTHESIS-PWY → 21/11
✓ PWY-5005 → 35/11
✓ PWY-6519 → 49/11
✓ Saved 101 assignments to harmonic_map.json


In [56]:
# ------------------------------------------------------------
# COBALAMIN (VITAMIN B12) — DENSE, RARE FUNCTIONAL COFACTOR
# ------------------------------------------------------------
# Prevalence-aware assignment:
#   Most prevalent → most consonant (lowest n×d)
# ------------------------------------------------------------

B12_ASSIGNMENTS = {
    # Salvage / community metabolism (most prevalent)
    'PWY-5509': (77, 8,  'Cobalamin (B12) biosynthesis — salvage / community route'),

    # De novo, late cobalt insertion
    'PWY-5508': (77, 16, 'Cobalamin (B12) biosynthesis — de novo (late cobalt insertion)'),

    # De novo, early cobalt insertion (rarest)
    'PWY-5507': (77, 32, 'Cobalamin (B12) biosynthesis — de novo (early cobalt insertion)'),
}

for pid, (n, d, deriv) in B12_ASSIGNMENTS.items():
    HARMONIC_MAP.assign(
        pid,
        Ratio(n, d),
        {
            'subcategory': 'Cofactors/Vitamins',
            'category': 'biosynthesis',
            'derivation': deriv,
        }
    )
    print(f"✓ {pid} → {n}/{d}")

HARMONIC_MAP.save()

✓ PWY-5509 → 77/8
✓ PWY-5508 → 77/16
✓ PWY-5507 → 77/32
✓ Saved 104 assignments to harmonic_map.json


In [57]:
# ------------------------------------------------------------
# MOLYBDENUM COFACTOR (MoCo) — HARD FUNCTIONAL GATE
# ------------------------------------------------------------
# Enables specific, high‑energy boundary redox reactions
# (e.g., nitrate, sulfite, formate).
#
# Ontology:
# - Derived undecimal (11²): second‑order functional abstraction
# - Acts on material at a boundary (5 × 7)
# - Binary: reaction either occurs or does not
#
# Single, unique harmonic identity.
# ------------------------------------------------------------

MOCO_ASSIGNMENTS = {
    'PWY-6823': (121, 35, 'Molybdenum cofactor biosynthesis — boundary redox gate'),
}

for pid, (n, d, deriv) in MOCO_ASSIGNMENTS.items():
    HARMONIC_MAP.assign(
        pid,
        Ratio(n, d),
        {
            'subcategory': 'Cofactors/Vitamins',
            'category': 'biosynthesis',
            'derivation': deriv,
        }
    )
    print(f"✓ {pid} → {n}/{d}")

HARMONIC_MAP.save()

✓ PWY-6823 → 121/35
✓ Saved 105 assignments to harmonic_map.json


In [58]:
# ------------------------------------------------------------
# METHANOGEN COFACTORS — TERMINAL METABOLIC SINK
# ------------------------------------------------------------
# Coenzyme M (CoM) and Coenzyme B (CoB) mediate the
# final, irreversible step of methanogenesis.
#
# Ontology:
# - Material‑functional chemistry (5 × 11)
# - Deep septimal constraint (terminal boundary)
# - Not permissive, not regulatory — end‑point chemistry
#
# Represented as a constrained 55‑family.
# ------------------------------------------------------------

METHANOGEN_ASSIGNMENTS = {
    # Primary terminal actor
    'P261-PWY': (55, 7,  'Coenzyme M biosynthesis — terminal methyl carrier'),

    # Deeply constrained partner
    'P241-PWY': (55, 49, 'Coenzyme B biosynthesis — terminal reductant partner'),
}

for pid, (n, d, deriv) in METHANOGEN_ASSIGNMENTS.items():
    HARMONIC_MAP.assign(
        pid,
        Ratio(n, d),
        {
            'subcategory': 'Cofactors/Vitamins',
            'category': 'biosynthesis',
            'derivation': deriv,
        }
    )
    print(f"✓ {pid} → {n}/{d}")

HARMONIC_MAP.save()

✓ P261-PWY → 55/7
✓ P241-PWY → 55/49
✓ Saved 107 assignments to harmonic_map.json


#### Progress Check

In [59]:
# ============================================================
# BIOSYNTHESIS STATUS — REMAINING UNASSIGNED PATHWAYS
# ============================================================
# Purpose:
# - Show what biosynthesis pathways are still unassigned
# - Group by subcategory for planning
# - No assignments, no side effects
# ============================================================

print("=== BIOSYNTHESIS STATUS CHECK ===\n")

# ------------------------------------------------------------
# Subset to biosynthesis category
# ------------------------------------------------------------
biosyn_df = df[df["category"] == "biosynthesis"].copy()

print(f"Total biosynthesis pathways (curated): {len(biosyn_df)}")

# ------------------------------------------------------------
# Assignment state
# ------------------------------------------------------------
biosyn_df["assigned"] = biosyn_df["pathway_id"].apply(
    lambda pid: HARMONIC_MAP.is_assigned(pid)
)

assigned_count = biosyn_df["assigned"].sum()
unassigned_count = len(biosyn_df) - assigned_count

print(f"Assigned:   {assigned_count}")
print(f"Unassigned: {unassigned_count}\n")

# ------------------------------------------------------------
# Breakdown by subcategory
# ------------------------------------------------------------
print("=== UNASSIGNED BIOSYNTHESIS BY SUBCATEGORY ===")

unassigned = biosyn_df[~biosyn_df["assigned"]]

display(
    unassigned
    .groupby("subcategory")
    .size()
    .sort_values(ascending=False)
    .to_frame("unassigned_count")
)

# ------------------------------------------------------------
# Detailed table for planning
# ------------------------------------------------------------
print("\n=== DETAILED UNASSIGNED BIOSYNTHESIS PATHWAYS ===")

display(
    unassigned
    .sort_values(["subcategory", "prevalence"], ascending=[True, False])[
        [
            "pathway_id",
            "pathway_name",
            "subcategory",
            "prevalence",
            "mean_abundance",
            "tier",
            "ms_status",
            "ms_lda_score",
        ]
    ]
)

# ------------------------------------------------------------
# Optional: quick sanity check on coverage
# ------------------------------------------------------------
print("\n=== BIOSYNTHESIS COVERAGE SUMMARY ===")

display(
    biosyn_df
    .groupby(["subcategory", "assigned"])
    .size()
    .unstack(fill_value=0)
    .rename(columns={True: "assigned", False: "unassigned"})
    .sort_values("unassigned", ascending=False)
)

=== BIOSYNTHESIS STATUS CHECK ===

Total biosynthesis pathways (curated): 143
Assigned:   80
Unassigned: 63

=== UNASSIGNED BIOSYNTHESIS BY SUBCATEGORY ===


,unassigned_count
subcategory,
Cell Envelope,24
Specialized,14
Other,12
Isoprenoids,7
Polyamines,6



=== DETAILED UNASSIGNED BIOSYNTHESIS PATHWAYS ===


,pathway_id,pathway_name,subcategory,prevalence,mean_abundance,tier,ms_status,ms_lda_score
0,DTDPRHAMSYN-PWY,dTDP-L-rhamnose biosynthesis I,Cell Envelope,0.999383,5.795910e-04,UNIVERSAL,NaN,NaN
4,PWY-6386,UDP-N-acetylmuramoyl-pentapeptide biosynthesis...,Cell Envelope,0.999178,5.476870e-04,UNIVERSAL,NaN,NaN
14,PWY-6387,UDP-N-acetylmuramoyl-pentapeptide biosynthesis...,Cell Envelope,0.998972,5.439090e-04,UNIVERSAL,NaN,NaN
15,PEPTIDOGLYCANSYN-PWY,peptidoglycan biosynthesis I (meso-diaminopime...,Cell Envelope,0.998972,5.376550e-04,UNIVERSAL,NaN,NaN
16,PWY-6385,peptidoglycan biosynthesis III (mycobacteria),Cell Envelope,0.998972,5.230880e-04,UNIVERSAL,NaN,NaN
...,...,...,...,...,...,...,...,...
397,AEROBACTINSYN-PWY,aerobactin biosynthesis,Specialized,0.069065,2.730000e-07,RARE,NaN,NaN
412,URSIN-PWY,ureide biosynthesis,Specialized,0.053649,3.500000e-07,RARE,NaN,NaN
430,PWY-5514,UDP-N-acetyl-D-galactosamine biosynthesis II,Specialized,0.025283,3.520000e-08,RARE,NaN,NaN
489,PWY-7090,"UDP-2,3-diacetamido-2,3-dideoxy-&alpha;-D-mann...",Specialized,0.000822,1.400000e-09,RARE,NaN,NaN



=== BIOSYNTHESIS COVERAGE SUMMARY ===


assigned,unassigned,assigned
subcategory,,
Cell Envelope,24,0
Specialized,14,0
Other,12,0
Isoprenoids,7,0
Polyamines,6,0
Amino Acids,0,28
Cofactors/Vitamins,0,28
Fatty Acids/Lipids,0,17
Nucleotides,0,7


In [60]:
# ============================================================
# TOP 10 MOST CRUCIAL UNASSIGNED BIOSYNTHESIS PATHWAYS
# ============================================================
# Criteria:
# 1. High prevalence
# 2. High mean abundance
# 3. Biosynthetic role (already filtered)
# ============================================================

biosyn_df = df[df["category"] == "biosynthesis"].copy()

biosyn_df["assigned"] = biosyn_df["pathway_id"].apply(
    lambda pid: HARMONIC_MAP.is_assigned(pid)
)

unassigned = biosyn_df[~biosyn_df["assigned"]].copy()

print(f"Total unassigned biosynthesis pathways: {len(unassigned)}\n")

# Sort by importance: prevalence → abundance → tier (if available)
unassigned_sorted = unassigned.sort_values(
    by=["prevalence", "mean_abundance"],
    ascending=[False, False]
)

print("=== TOP 10 UNASSIGNED BIOSYNTHESIS PATHWAYS ===\n")

display(
    unassigned_sorted.head(10)[
        [
            "pathway_id",
            "pathway_name",
            "subcategory",
            "prevalence",
            "mean_abundance",
            "tier",
        ]
    ]
)

Total unassigned biosynthesis pathways: 63

=== TOP 10 UNASSIGNED BIOSYNTHESIS PATHWAYS ===



,pathway_id,pathway_name,subcategory,prevalence,mean_abundance,tier
0,DTDPRHAMSYN-PWY,dTDP-L-rhamnose biosynthesis I,Cell Envelope,0.999383,0.000580,UNIVERSAL
4,PWY-6386,UDP-N-acetylmuramoyl-pentapeptide biosynthesis...,Cell Envelope,0.999178,0.000548,UNIVERSAL
8,PWY-6700,queuosine biosynthesis,Other,0.999178,0.000462,UNIVERSAL
14,PWY-6387,UDP-N-acetylmuramoyl-pentapeptide biosynthesis...,Cell Envelope,0.998972,0.000544,UNIVERSAL
15,PEPTIDOGLYCANSYN-PWY,peptidoglycan biosynthesis I (meso-diaminopime...,Cell Envelope,0.998972,0.000538,UNIVERSAL
16,PWY-6385,peptidoglycan biosynthesis III (mycobacteria),Cell Envelope,0.998972,0.000523,UNIVERSAL
24,GLUTORN-PWY,L-ornithine biosynthesis,Specialized,0.998972,0.000359,UNIVERSAL
46,PWY-6163,chorismate biosynthesis from 3-dehydroquinate,Other,0.998356,0.000487,UNIVERSAL
47,ARO-PWY,chorismate biosynthesis I,Other,0.998356,0.000460,UNIVERSAL
51,GLYCOGENSYNTH-PWY,glycogen biosynthesis I (from ADP-D-Glucose),Other,0.998356,0.000304,UNIVERSAL


#### Cell Envelope

In [61]:
# ------------------------------------------------------------
# L-RHAMNOSE — External Identity Marker (Cell Envelope)
# ------------------------------------------------------------
# 1/7 = subharmonic identity
#
# 7  → identity
# Inverted polarity → identity asserted outward, read by others
#
# Rhamnose is:
# - built only for the cell surface
# - not metabolized internally
# - used for strain/species recognition (phage, immune, ecology)
# ------------------------------------------------------------

HARMONIC_MAP.assign(
    'DTDPRHAMSYN-PWY',
    Ratio(1, 7),
    {
        'subcategory': 'Cell Envelope',
        'category': 'biosynthesis',
        'derivation': 'L-rhamnose — external identity marker (1/7)',
    }
)

print("✓ DTDPRHAMSYN-PWY → 1/7")

HARMONIC_MAP.save()

✓ DTDPRHAMSYN-PWY → 1/7
✓ Saved 108 assignments to harmonic_map.json


In [62]:
# ------------------------------------------------------------
# PEPTIDOGLYCAN BIOSYNTHESIS — CELL ENVELOPE (SUBHARMONIC)
# ------------------------------------------------------------
# role: external load‑bearing boundary
#
# 8/35   (7×5)   → canonical peptidoglycan
# 16/35  (7×5)   → thickened / variant peptidoglycan
# 16/49  (7²)    → reinforced boundary
# 32/49  (7²)    → extreme envelope
# 32/63  (3²×7)  → adaptive remodeling
# ------------------------------------------------------------

PG_ASSIGNMENTS = {
    'PEPTIDOGLYCANSYN-PWY': (8,  35, 'canonical peptidoglycan'),
    'PWY-5265':            (16, 35, 'thickened peptidoglycan'),
    'PWY-6470':            (16, 49, 'reinforced peptidoglycan'),
    'PWY-6385':            (32, 49, 'extreme peptidoglycan envelope'),
    'PWY-6471':            (32, 63, 'adaptive peptidoglycan remodeling'),
}

for pid, (n, d, note) in PG_ASSIGNMENTS.items():
    HARMONIC_MAP.assign(
        pid,
        Ratio(n, d),
        {
            'category': 'biosynthesis',
            'subcategory': 'Cell Envelope',
            'derivation': note,
        }
    )
    print(f"✓ {pid} → {n}/{d}")

HARMONIC_MAP.save()

✓ PEPTIDOGLYCANSYN-PWY → 8/35
✓ PWY-5265 → 16/35
✓ PWY-6470 → 16/49
✓ PWY-6385 → 32/49
✓ PWY-6471 → 32/63
✓ Saved 113 assignments to harmonic_map.json


In [63]:
# ------------------------------------------------------------
# UDP-ACTIVATED SUGAR PRECURSORS — INFRASTRUCTURE LAYER
# ------------------------------------------------------------
# role: activated material feeding envelope construction
#
# 105/32  (3×5×7) → most abundant UDP sugar (GlcNAc)
# 105/64  (3×5×7) → PG-specific UDP-MurNAc precursor
# 105/128 (3×5×7) → variant / secondary MurNAc activation
# ------------------------------------------------------------

UDP_ASSIGNMENTS = {
    'UDPNAGSYN-PWY': (105, 32,  'UDP-GlcNAc — highest availability'),
    'PWY-6386':      (105, 64,  'UDP-MurNAc — PG precursor'),
    'PWY-6387':      (105, 128, 'UDP-MurNAc variant'),
}

for pid, (n, d, note) in UDP_ASSIGNMENTS.items():
    HARMONIC_MAP.assign(
        pid,
        Ratio(n, d),
        {
            'subcategory': 'Cell Envelope',
            'category': 'biosynthesis',
            'derivation': note,
        }
    )
    print(f"✓ {pid} → {n}/{d}")

HARMONIC_MAP.save()

✓ UDPNAGSYN-PWY → 105/32
✓ PWY-6386 → 105/64
✓ PWY-6387 → 105/128
✓ Saved 116 assignments to harmonic_map.json


In [64]:
# ------------------------------------------------------------
# GDP-ACTIVATED SUGAR PRECURSORS — OUTER ENVELOPE ACTIVATION
# ------------------------------------------------------------
# role: activated material for outer membrane / capsule synthesis
#
# 256/105 (3×5×7) → prevalent GDP sugar (mannose)
# 512/105 (3×5×7) → specialized GDP sugar (heptose)
# ------------------------------------------------------------

GDP_ASSIGNMENTS = {
    'PWY-5659': (256, 105, 'GDP-mannose — prevalent outer-envelope precursor'),
    'PWY-6478': (512, 105, 'GDP-heptose — specialized outer-core precursor'),
}

for pid, (n, d, note) in GDP_ASSIGNMENTS.items():
    HARMONIC_MAP.assign(
        pid,
        Ratio(n, d),
        {
            'subcategory': 'Cell Envelope',
            'category': 'biosynthesis',
            'derivation': note,
        }
    )
    print(f"✓ {pid} → {n}/{d}")

HARMONIC_MAP.save()

✓ PWY-5659 → 256/105
✓ PWY-6478 → 512/105
✓ Saved 118 assignments to harmonic_map.json


In [65]:
# ------------------------------------------------------------
# COLANIC ACID BIOSYNTHESIS — CAPSULE / SOFT BOUNDARY
# ------------------------------------------------------------
# role: adaptive extracellular capsule
#
# 32/147 (7²×3) → soft, stress-responsive boundary layer
# ------------------------------------------------------------

HARMONIC_MAP.assign(
    'COLANSYN-PWY',
    Ratio(32, 147),
    {
        'subcategory': 'Cell Envelope',
        'category': 'biosynthesis',
        'derivation': 'colanic acid capsule — adaptive septimal boundary (7²×3)',
    }
)

print("✓ COLANSYN-PWY → 32/147")

HARMONIC_MAP.save()

✓ COLANSYN-PWY → 32/147
✓ Saved 119 assignments to harmonic_map.json


In [66]:
# ------------------------------------------------------------
# ENTEROBACTERIAL COMMON ANTIGEN — SURFACE COHERENCE LAYER
# ------------------------------------------------------------
# role: shared outer-surface antigen (group identity)
#
# 147/25 (7²×3 / 5²) → coherent, adaptive surface recognition layer
# ------------------------------------------------------------

HARMONIC_MAP.assign(
    'ECASYN-PWY',
    Ratio(147, 25),
    {
        'subcategory': 'Cell Envelope',
        'category': 'biosynthesis',
        'derivation': 'enterobacterial common antigen — group-level surface identity',
    }
)

print("✓ ECASYN-PWY → 147/25")

HARMONIC_MAP.save()

✓ ECASYN-PWY → 147/25
✓ Saved 120 assignments to harmonic_map.json


In [67]:
# ------------------------------------------------------------
# TEICHOIC ACID BIOSYNTHESIS — PG-ANCHORED MODULATION LAYER
# ------------------------------------------------------------
# role: Gram+ wall-embedded polymer modulating charge and flow
#
# 4/21 (3×7) → boundary-anchored, flow-modulating constraint
# ------------------------------------------------------------

HARMONIC_MAP.assign(
    'TEICHOICACID-PWY',
    Ratio(4, 21),
    {
        'subcategory': 'Cell Envelope',
        'category': 'biosynthesis',
        'derivation': 'teichoic acid — PG-anchored boundary modulation (3×7)',
    }
)

print("✓ TEICHOICACID-PWY → 4/21")

HARMONIC_MAP.save()

✓ TEICHOICACID-PWY → 4/21
✓ Saved 121 assignments to harmonic_map.json


In [68]:
# ------------------------------------------------------------
# LIPOPOLYSACCHARIDE (LPS) BIOSYNTHESIS — OUTER MEMBRANE ANCHOR
# ------------------------------------------------------------
# role: dominant outer membrane architecture (Gram−)
#
# 147/64 (7²×3) → identity-saturated, adaptive outer shell
# ------------------------------------------------------------

HARMONIC_MAP.assign(
    'LPSSYN-PWY',
    Ratio(147, 64),
    {
        'subcategory': 'Cell Envelope',
        'category': 'biosynthesis',
        'derivation': 'lipopolysaccharide — dominant septimal outer membrane (7²×3)',
    }
)

print("✓ LPSSYN-PWY → 147/64")

HARMONIC_MAP.save()

✓ LPSSYN-PWY → 147/64
✓ Saved 122 assignments to harmonic_map.json


In [69]:
# ------------------------------------------------------------
# ADP-HEPTOSE BIOSYNTHESIS — LPS INNER-CORE BRIDGE
# ------------------------------------------------------------
# role: activated precursor for LPS inner core assembly
#
# 147/128 (7²×3) → deep infrastructure enabling outer membrane
# ------------------------------------------------------------

HARMONIC_MAP.assign(
    'PWY0-1241',
    Ratio(147, 128),
    {
        'subcategory': 'Cell Envelope',
        'category': 'biosynthesis',
        'derivation': 'ADP-heptose — LPS inner-core precursor (7²×3)',
    }
)

print("✓ PWY0-1241 → 147/128")

HARMONIC_MAP.save()

✓ PWY0-1241 → 147/128
✓ Saved 123 assignments to harmonic_map.json


In [70]:
# ------------------------------------------------------------
# CMP-ACTIVATED SUGARS — TERMINAL OUTER-MEMBRANE PUNCTUATION
# ------------------------------------------------------------
# role: terminal, embodied surface decorations
#
# 245/64  (7²×5) → prevalent CMP sugar (KDO)
# 245/32  (7²×5) → specialized terminal decoration
# 245/16  (7²×5) → rare terminal decoration
# ------------------------------------------------------------

CMP_ASSIGNMENTS = {
    'PWY-1269': (245, 64, 'CMP-KDO — prevalent LPS core anchor'),
    'PWY-6749': (245, 32, 'CMP-legionaminate — specialized terminal sugar'),
    'PWY-6143': (245, 16, 'CMP-pseudaminate — rare terminal decoration'),
}

for pid, (n, d, note) in CMP_ASSIGNMENTS.items():
    HARMONIC_MAP.assign(
        pid,
        Ratio(n, d),
        {
            'category': 'biosynthesis',
            'subcategory': 'Cell Envelope',
            'derivation': note,
        }
    )
    print(f"✓ {pid} → {n}/{d}")

HARMONIC_MAP.save()

✓ PWY-1269 → 245/64
✓ PWY-6749 → 245/32
✓ PWY-6143 → 245/16
✓ Saved 126 assignments to harmonic_map.json


In [71]:
# ------------------------------------------------------------
# dTDP-ACTIVATED SUGARS — RARE SURFACE IDENTITY (EMBODIED)
# ------------------------------------------------------------
# role: strain-level, embodied surface identity markers
#
# 16/245  (7²×5) → most prevalent rare identity sugar
# 32/245  (7²×5) → moderately rare identity
# 64/245  (7²×5) → rare identity
# 128/245 (7²×5) → very rare identity
# 256/245 (7²×5) → extreme / exotic identity
# ------------------------------------------------------------

DTDP_ASSIGNMENTS = {
    'PWY-7315': (16,  245, 'dTDP-N-acetylthomosamine — common rare identity (7²×5)'),
    'PWY-7312': (32,  245, 'dTDP-β-D-fucofuranose — moderate identity marker (7²×5)'),
    'PWY-7316': (64,  245, 'dTDP-N-acetylviosamine — rare surface identity (7²×5)'),
    'PWY-6953': (128, 245, 'dTDP-3-acetamido-3,6-dideoxy-galactose — very rare identity (7²×5)'),
    'PWY-7413': (256, 245, 'dTDP-6-deoxy-α-D-allose — extreme / exotic identity (7²×5)'),
}

for pid, (n, d, note) in DTDP_ASSIGNMENTS.items():
    HARMONIC_MAP.assign(
        pid,
        Ratio(n, d),
        {
            'subcategory': 'Cell Envelope',
            'category': 'biosynthesis',
            'derivation': note,
        }
    )
    print(f"✓ {pid} → {n}/{d}")

HARMONIC_MAP.save()

✓ PWY-7315 → 16/245
✓ PWY-7312 → 32/245
✓ PWY-7316 → 64/245
✓ PWY-6953 → 128/245
✓ PWY-7413 → 256/245
✓ Saved 131 assignments to harmonic_map.json


#### Chorismate 13/1

In [72]:
# ------------------------------------------------------------
# CHORISMATE BIOSYNTHESIS — UNIVERSAL DECISION (ORDERED)
# ------------------------------------------------------------
# role: irreversible biosynthetic decision into incompatible
#       aromatic futures
#
# Prime 13 → decision / contingency
# Register (2-limit) → prevalence / dominance
#
# 13/1 → primary, dominant chorismate implementation
# 13/2 → secondary, alternative implementation
# ------------------------------------------------------------

import pandas as pd

CHORISMATE_PWYS = [
    'PWY-6163',  # chorismate biosynthesis from 3-dehydroquinate
    'ARO-PWY',   # chorismate biosynthesis I
]

# ------------------------------------------------------------------
# 1️⃣ Extract prevalence / abundance data
# ------------------------------------------------------------------

chor_df = df[df['pathway_id'].isin(CHORISMATE_PWYS)].copy()

display(
    chor_df[
        [
            'pathway_id',
            'pathway_name',
            'prevalence',
            'prevalence_pct',
            'median_abundance',
            'mean_abundance',
        ]
    ]
)

# ------------------------------------------------------------------
# 2️⃣ Determine ordering
# ------------------------------------------------------------------
# Primary sort: prevalence
# Secondary sort: median abundance
# (mean abundance used only if needed)

chor_df = chor_df.sort_values(
    by=['prevalence', 'median_abundance', 'mean_abundance'],
    ascending=False
)

primary_pid   = chor_df.iloc[0]['pathway_id']
secondary_pid = chor_df.iloc[1]['pathway_id']

print(f"Primary chorismate pathway   → {primary_pid} (13/1)")
print(f"Secondary chorismate pathway → {secondary_pid} (13/2)")

# ------------------------------------------------------------------
# 3️⃣ Assign ratios
# ------------------------------------------------------------------

HARMONIC_MAP.assign(
    primary_pid,
    Ratio(13, 1),
    {
        'category': 'biosynthesis',
        'subcategory': 'Other',
        'derivation': (
            'chorismate — dominant irreversible decision point '
            'into aromatic biosynthetic futures (Prime 13)'
        ),
    }
)

HARMONIC_MAP.assign(
    secondary_pid,
    Ratio(13, 2),
    {
        'category': 'biosynthesis',
        'subcategory': 'Other',
        'derivation': (
            'chorismate — alternative implementation of the '
            'aromatic decision point (Prime 13, secondary register)'
        ),
    }
)

print(f"✓ {primary_pid} → 13/1")
print(f"✓ {secondary_pid} → 13/2")

HARMONIC_MAP.save()

,pathway_id,pathway_name,prevalence,prevalence_pct,median_abundance,mean_abundance
46,PWY-6163,chorismate biosynthesis from 3-dehydroquinate,0.998356,99.84%,0.000491,0.000487
47,ARO-PWY,chorismate biosynthesis I,0.998356,99.84%,0.000468,0.000460


Primary chorismate pathway   → PWY-6163 (13/1)
Secondary chorismate pathway → ARO-PWY (13/2)
✓ PWY-6163 → 13/1
✓ ARO-PWY → 13/2
✓ Saved 133 assignments to harmonic_map.json


#### Isoprenoids

In [73]:
# ------------------------------------------------------------
# ISOPRENOIDS — ACTIVE SCAFFOLD → BACKBONE → EXTENSION
# ------------------------------------------------------------
# ordering principle:
#   abundance + activity → lower n·d (more consonant)
#
# 33-space → active scaffold management / hubs
# 55-space → backbone provisioning
# 77-space → extended / specialized scaffolds
# ------------------------------------------------------------

ISOPRENOID_ASSIGNMENTS = {
    # Active scaffold management (most abundant, most dynamic)
    'POLYISOPRENSYN-PWY': (55, 8, 'polyisoprenoid biosynthesis — active scaffold hub'),
    'PWY-6270':           (55, 9, 'isoprene biosynthesis — dynamic short-chain pool'),

    # Backbone provisioning
    'PWY-6859': (77, 5, 'all-trans-farnesol biosynthesis — flexible backbone supply'),
    'PWY-5837': (77, 9, 'dihydroxynaphthoate biosynthesis — stable backbone supply'),

    # Extended / specialized scaffolds
    'PWY-5838': (77, 10,  'menaquinol-8 superpathway — adaptive extended scaffold'),
    'PWY-6708': (77, 15, 'ubiquinol-8 biosynthesis — heavy extended scaffold'),

    # Structural reserve / boundary-constrained enablement
    'PWY-5910': (33, 7, 'GGPP superpathway — boundary-constrained scaffold reserve'),
}

for pid, (n, d, note) in ISOPRENOID_ASSIGNMENTS.items():
    HARMONIC_MAP.assign(
        pid,
        Ratio(n, d),
        {
            'category': 'biosynthesis',
            'subcategory': 'Isoprenoids',
            'derivation': note,
        }
    )
    print(f"✓ {pid} → {n}/{d}")

HARMONIC_MAP.save()

✓ POLYISOPRENSYN-PWY → 55/8
✓ PWY-6270 → 55/9
✓ PWY-6859 → 77/5
✓ PWY-5837 → 77/9
✓ PWY-5838 → 77/10
✓ PWY-6708 → 77/15
✓ PWY-5910 → 33/7
✓ Saved 140 assignments to harmonic_map.json


#### Polyamines

In [74]:
# ------------------------------------------------------------
# POLYAMINES — MATERIAL DENSITY BUFFER (LATE BIOSYNTHESIS)
# ------------------------------------------------------------
# role: high-abundance polycation pools for electrostatic buffering
#
# single semantic class (25 = 5²)
# variation by register only
#
# abundance-ordered:
# 25/2  → highest abundance / baseline pool
# 25/4
# 25/8
# 25/12
# 25/16
# 25/21 → lowest abundance / geometry-specific
# ------------------------------------------------------------

POLYAMINE_ASSIGNMENTS = {
    'PWY-6305':        (25, 2,  'putrescine biosynthesis — baseline polyamine pool'),
    'POLYAMSYN-PWY':   (25, 4,  'polyamine superpathway I — expanded buffering'),
    'POLYAMINSYN3-PWY':(25, 8,  'polyamine superpathway II — extended buffering'),
    'PWY-6562':        (25, 12, 'norspermidine biosynthesis — geometric modulation'),
    'PWY-6565':        (25, 16, 'polyamine superpathway III — low-abundance extension'),
    'PWY-6834':        (25, 21, 'spermidine biosynthesis III — rare / specialized pool'),
}

for pid, (n, d, note) in POLYAMINE_ASSIGNMENTS.items():
    HARMONIC_MAP.assign(
        pid,
        Ratio(n, d),
        {
            'category': 'biosynthesis',
            'subcategory': 'Polyamines',
            'derivation': note,
        }
    )
    print(f"✓ {pid} → {n}/{d}")

HARMONIC_MAP.save()

✓ PWY-6305 → 25/2
✓ POLYAMSYN-PWY → 25/4
✓ POLYAMINSYN3-PWY → 25/8
✓ PWY-6562 → 25/12
✓ PWY-6565 → 25/16
✓ PWY-6834 → 25/21
✓ Saved 146 assignments to harmonic_map.json


#### Specialized/Other

In [75]:
# ------------------------------------------------------------
# tRNA MODIFICATIONS — TRANSLATIONAL GEOMETRY TUNING
# ------------------------------------------------------------
# role: material incorporation that modulates codon–anticodon
#       interaction geometry without catalytic or regulatory function
#
# 75 = 3 × 5²  → interaction-modifying dense material
#
# 75/16 → queuosine (high-prevalence tRNA modification)
# 75/32 → preQ₀ precursor (metabolic antecedent)
# 75/49 → wyosine derivatives (identity-anchored, specialized)
# ------------------------------------------------------------

TRNA_MOD_ASSIGNMENTS = {
    'PWY-6700': (75, 16, 'queuosine — tRNA wobble-base modification'),
    'PWY-6703': (75, 32, 'preQ0 — metabolic precursor to queuosine'),
    'PWY-7286': (75, 49, 'wyosine derivative — identity-anchored tRNA modification'),
}

for pid, (n, d, note) in TRNA_MOD_ASSIGNMENTS.items():
    HARMONIC_MAP.assign(
        pid,
        Ratio(n, d),
        {
            'category': 'biosynthesis',
            'subcategory': 'tRNA Modifications',
            'derivation': note,
        }
    )
    print(f"✓ {pid} → {n}/{d}")

HARMONIC_MAP.save()

✓ PWY-6700 → 75/16
✓ PWY-6703 → 75/32
✓ PWY-7286 → 75/49
✓ Saved 149 assignments to harmonic_map.json


In [76]:
# ------------------------------------------------------------
# GLYCOGEN — CARBON STORAGE RESERVE
# ------------------------------------------------------------
# role: inert, reversible polymeric carbon storage
#
# 1/8 → deep, low-salience material reserve
# ------------------------------------------------------------

HARMONIC_MAP.assign(
    'GLYCOGENSYNTH-PWY',
    Ratio(1, 8),
    {
        'category': 'biosynthesis',
        'subcategory': 'Carbon Storage',
        'derivation': 'glycogen biosynthesis — inert polymeric carbon reserve',
    }
)

print("✓ GLYCOGENSYNTH-PWY → 1/8")

HARMONIC_MAP.save()

✓ GLYCOGENSYNTH-PWY → 1/8
✓ Saved 150 assignments to harmonic_map.json


In [77]:
# ------------------------------------------------------------
# ppGpp — GLOBAL STRESS / GROWTH DECISION
# ------------------------------------------------------------
# role: guanosine-derived alarmone enforcing mutually exclusive
#       transcriptional states
#
# 13/5 → decision signal embodied in nucleotide chemistry
# ------------------------------------------------------------

HARMONIC_MAP.assign(
    'PPGPPMET-PWY',
    Ratio(13, 5),
    {
        'category': 'biosynthesis',
        'subcategory': 'Stress / Decision',
        'derivation': 'ppGpp biosynthesis — guanosine-based global decision signal',
    }
)

print("✓ PPGPPMET-PWY → 13/5")

HARMONIC_MAP.save()

✓ PPGPPMET-PWY → 13/5
✓ Saved 151 assignments to harmonic_map.json


In [78]:
# ============================================================
# BIOSYNTHESIS COMPLETION — Final 19 Pathways
# ============================================================
#
# This cell completes the biosynthesis category with mathematically
# coherent assignments based on the established prime architecture:
#
#   2, 3    → Energy (glycolysis, TCA, fermentation)
#   5       → Information (nucleotides, polyamines)  
#   7       → Structural identity (amino acids, membranes)
#   11      → Catalytic (NAD, CoA, B12, thiamin)
#   13      → Aromatic gate (chorismate, siderophores)
#
# Composite primes encode functional relationships:
#   27 = 3³     → Deep TCA / nitrogen metabolism
#   35 = 5×7   → Biosynthetic structure
#   45 = 9×5   → TCA² × biosynthetic
#   55 = 5×11  → Biosynthetic × cofactor
#   77 = 7×11  → Structural × cofactor
#   91 = 7×13  → Structural × aromatic
#   143 = 11×13 → Cofactor × aromatic
#   231 = 3×7×11 → TCA × structural × cofactor
#
# ============================================================

# ─────────────────────────────────────────────────────────────
# NITROGEN CYCLE — 27-series (3³)
# ─────────────────────────────────────────────────────────────
# Ornithine and citrulline are the urea cycle intermediates.
# 27 = 3³ connects them to deep TCA/energy metabolism.
# These feed into arginine biosynthesis (21-series) and polyamines (25-series).

HARMONIC_MAP.assign("GLUTORN-PWY", Ratio(27, 5), {
    "pathway_name": "L-ornithine biosynthesis",
    "subcategory": "Specialized",
    "category": "biosynthesis",
    "prevalence": 0.999,
    "derivation": "ornithine — nitrogen pool for arginine/polyamines (3³/5)"
})

HARMONIC_MAP.assign("CITRULBIO-PWY", Ratio(27, 10), {
    "pathway_name": "L-citrulline biosynthesis",
    "subcategory": "Specialized",
    "category": "biosynthesis",
    "prevalence": 0.981,
    "derivation": "citrulline — urea cycle shuttle (3³/2×5)"
})

# ─────────────────────────────────────────────────────────────
# PURINE ENDPOINT — 25/9 (5²/3²)
# ─────────────────────────────────────────────────────────────
# Urate is where purines go to die. 
# 5² (information squared) divided by 3² (TCA squared) = 
# purine catabolism channeled through energy metabolism.

HARMONIC_MAP.assign("PWY-5695", Ratio(25, 9), {
    "pathway_name": "urate biosynthesis/inosine 5'-phosphate degradation",
    "subcategory": "Specialized",
    "category": "biosynthesis",
    "prevalence": 0.998,
    "derivation": "urate — purine endpoint, nitrogen excretion (5²/3²)"
})

# ─────────────────────────────────────────────────────────────
# NITROGEN TRANSPORT — 45-series (9×5)
# ─────────────────────────────────────────────────────────────
# Ureides transport nitrogen in plants/bacteria.
# 45 = 9×5 = TCA² × biosynthetic

HARMONIC_MAP.assign("URSIN-PWY", Ratio(45, 8), {
    "pathway_name": "ureide biosynthesis",
    "subcategory": "Specialized",
    "category": "biosynthesis",
    "prevalence": 0.054,
    "derivation": "ureide — nitrogen transport compound (9×5/2³)"
})

# ─────────────────────────────────────────────────────────────
# CENTRAL METABOLISM
# ─────────────────────────────────────────────────────────────

# Acetyl-CoA superpathway — the universal 2-carbon donor
# 33 = 3×11 connects TCA (3) to CoA cofactor chemistry (11)
HARMONIC_MAP.assign("PWY-5173", Ratio(33, 16), {
    "pathway_name": "superpathway of acetyl-CoA biosynthesis",
    "subcategory": "Specialized",
    "category": "biosynthesis",
    "prevalence": 0.714,
    "derivation": "acetyl-CoA — universal 2C donor (3×11/2⁴)"
})

# Butanediol — fermentation product, stays in 27 family
HARMONIC_MAP.assign("P125-PWY", Ratio(27, 20), {
    "pathway_name": "superpathway of (R,R)-butanediol biosynthesis",
    "subcategory": "Specialized",
    "category": "biosynthesis",
    "prevalence": 0.467,
    "derivation": "butanediol — fermentation product (3³/4×5)"
})

# Mannosylglycerate — osmolyte (stress protection)
# Simple 9/5 = TCA energy for biosynthetic stress response
HARMONIC_MAP.assign("PWY-5656", Ratio(9, 5), {
    "pathway_name": "mannosylglycerate biosynthesis I",
    "subcategory": "Specialized",
    "category": "biosynthesis",
    "prevalence": 0.364,
    "derivation": "mannosylglycerate — osmolyte, stress protection (3²/5)"
})

# ─────────────────────────────────────────────────────────────
# SIDEROPHORES — 91-series (7×13)
# ─────────────────────────────────────────────────────────────
# Iron-scavenging molecules. They are:
#   - Derived from chorismate (aromatic, prime 13)
#   - Secreted identity markers (structural, prime 7)
#   - Enable metalloenzyme catalysis (functional bridge)
#
# 91 = 7×13 perfectly captures this dual nature.
# Enterobactin is the major siderophore; aerobactin is a variant.

HARMONIC_MAP.assign("ENTBACSYN-PWY", Ratio(91, 8), {
    "pathway_name": "enterobactin biosynthesis",
    "subcategory": "Specialized",
    "category": "biosynthesis",
    "prevalence": 0.307,
    "derivation": "enterobactin — primary siderophore, iron scavenger (7×13/2³)"
})

HARMONIC_MAP.assign("AEROBACTINSYN-PWY", Ratio(91, 16), {
    "pathway_name": "aerobactin biosynthesis",
    "subcategory": "Specialized",
    "category": "biosynthesis",
    "prevalence": 0.069,
    "derivation": "aerobactin — hydroxamate siderophore variant (7×13/2⁴)"
})

# ─────────────────────────────────────────────────────────────
# ANTIOXIDANT — 55-series (5×11)
# ─────────────────────────────────────────────────────────────
# Mycothiol is the mycobacterial equivalent of glutathione.
# 55 = 5×11 = biosynthetic × cofactor (protective chemistry)
# This extends the 55-series: CoM (55/7), CoB (55/49), now mycothiol.

HARMONIC_MAP.assign("PWY1G-0", Ratio(55, 16), {
    "pathway_name": "mycothiol biosynthesis",
    "subcategory": "Specialized",
    "category": "biosynthesis",
    "prevalence": 0.094,
    "derivation": "mycothiol — antioxidant, mycobacterial glutathione (5×11/2⁴)"
})

# ─────────────────────────────────────────────────────────────
# METHANOGEN / ARCHAEAL — 55 and 143 series
# ─────────────────────────────────────────────────────────────
# Factor 420 (F420) is a methanogen-specific electron carrier.
# Extends the 55-series with CoM and CoB (already assigned).

HARMONIC_MAP.assign("PWY-5198", Ratio(55, 4), {
    "pathway_name": "factor 420 biosynthesis",
    "subcategory": "Other",
    "category": "biosynthesis",
    "prevalence": 0.297,
    "derivation": "factor F420 — methanogen deazaflavin cofactor (5×11/2²)"
})

# CDP-archaeol is the archaeal membrane lipid precursor.
# 143 = 11×13 = cofactor × aromatic (unique archaeal chemistry)
HARMONIC_MAP.assign("PWY-6349", Ratio(143, 16), {
    "pathway_name": "CDP-archaeol biosynthesis",
    "subcategory": "Other",
    "category": "biosynthesis",
    "prevalence": 0.282,
    "derivation": "CDP-archaeol — archaeal membrane precursor (11×13/2⁴)"
})

# ─────────────────────────────────────────────────────────────
# MENAQUINONE PRECURSOR — 77-series (7×11)
# ─────────────────────────────────────────────────────────────
# 1,4-dihydroxy-6-naphthoate feeds into menaquinone (vitamin K2).
# 77 = 7×11 = structural × cofactor (membrane-embedded electron carrier)
# This connects to the existing 77-series (B12 at 77/8, 77/16, 77/32).

HARMONIC_MAP.assign("PWY-7374", Ratio(77, 6), {
    "pathway_name": "1,4-dihydroxy-6-naphthoate biosynthesis I",
    "subcategory": "Other",
    "category": "biosynthesis",
    "prevalence": 0.030,
    "derivation": "naphthoate — menaquinone precursor (7×11/2×3)"
})

# ─────────────────────────────────────────────────────────────
# REMAINING LOW-PREVALENCE PATHWAYS
# ─────────────────────────────────────────────────────────────

# Tetrahydromethanopterin — methanogen C1 carrier (like folate)
# Extends 143-series (11×13) with CDP-archaeol
HARMONIC_MAP.assign("PWY-6148", Ratio(143, 32), {
    "pathway_name": "tetrahydromethanopterin biosynthesis",
    "subcategory": "Other",
    "category": "biosynthesis",
    "prevalence": 0.0008,
    "derivation": "tetrahydromethanopterin — methanogen C1 carrier (11×13/2⁵)"
})

# Inositol-1,4,5-trisphosphate — signaling molecule
# 15/8 = 3×5/2³ — simple, information-adjacent
HARMONIC_MAP.assign("PWY-6351", Ratio(15, 8), {
    "pathway_name": "D-myo-inositol (1,4,5)-trisphosphate biosynthesis",
    "subcategory": "Other",
    "category": "biosynthesis",
    "prevalence": 0.0008,
    "derivation": "inositol-1,4,5-P3 — signaling molecule (3×5/2³)"
})

# Cob(II)yrinate a,c-diamide — B12 precursor
# 231 = 3×7×11 connects TCA (3), structural (7), cofactor (11)
HARMONIC_MAP.assign("PWY-7377", Ratio(231, 16), {
    "pathway_name": "cob(II)yrinate a,c-diamide biosynthesis I (early cobalt insertion)",
    "subcategory": "Other",
    "category": "biosynthesis",
    "prevalence": 0.0002,
    "derivation": "cob(II)yrinate — B12 precursor (3×7×11/2⁴)"
})

# UDP-N-acetyl-D-galactosamine — surface sugar
# Extends 245-series (7²×5) with other cell envelope sugars
HARMONIC_MAP.assign("PWY-5514", Ratio(245, 128), {
    "pathway_name": "UDP-N-acetyl-D-galactosamine biosynthesis II",
    "subcategory": "Specialized",
    "category": "biosynthesis",
    "prevalence": 0.025,
    "derivation": "UDP-GalNAc — surface sugar (7²×5/2⁷)"
})

# UDP-ManNAc derivative — rare surface sugar
# 245/256 < 1 gives it subharmonic character (external presentation)
HARMONIC_MAP.assign("PWY-7090", Ratio(245, 256), {
    "pathway_name": "UDP-2,3-diacetamido-2,3-dideoxy-α-D-mannuronate biosynthesis",
    "subcategory": "Specialized",
    "category": "biosynthesis",
    "prevalence": 0.0008,
    "derivation": "UDP-ManNAc — rare surface identity (7²×5/2⁸)"
})

# Ergothioneine — histidine-derived antioxidant
# Extends 55-series (5×11) with mycothiol
HARMONIC_MAP.assign("PWY-7255", Ratio(55, 32), {
    "pathway_name": "ergothioneine biosynthesis I (bacteria)",
    "subcategory": "Specialized",
    "category": "biosynthesis",
    "prevalence": 0.0008,
    "derivation": "ergothioneine — histidine-derived antioxidant (5×11/2⁵)"
})

# ─────────────────────────────────────────────────────────────
# VERIFICATION
# ─────────────────────────────────────────────────────────────

print("=" * 70)
print("BIOSYNTHESIS COMPLETION — 19 Pathways Added")
print("=" * 70)

new_assignments = [
    # Nitrogen cycle (27-series)
    ("GLUTORN-PWY", 27, 5, "ornithine"),
    ("CITRULBIO-PWY", 27, 10, "citrulline"),
    ("P125-PWY", 27, 20, "butanediol"),
    # Purine/nitrogen
    ("PWY-5695", 25, 9, "urate"),
    ("URSIN-PWY", 45, 8, "ureide"),
    # Central metabolism
    ("PWY-5173", 33, 16, "acetyl-CoA"),
    ("PWY-5656", 9, 5, "mannosylglycerate"),
    # Siderophores (91-series)
    ("ENTBACSYN-PWY", 91, 8, "enterobactin"),
    ("AEROBACTINSYN-PWY", 91, 16, "aerobactin"),
    # Antioxidants (55-series)
    ("PWY1G-0", 55, 16, "mycothiol"),
    ("PWY-7255", 55, 32, "ergothioneine"),
    # Methanogen/Archaeal
    ("PWY-5198", 55, 4, "factor 420"),
    ("PWY-6349", 143, 16, "CDP-archaeol"),
    ("PWY-6148", 143, 32, "tetrahydromethanopterin"),
    # Quinone precursor
    ("PWY-7374", 77, 6, "naphthoate"),
    # B12 precursor
    ("PWY-7377", 231, 16, "cob(II)yrinate"),
    # Signaling
    ("PWY-6351", 15, 8, "inositol-P3"),
    # Surface sugars (245-series)
    ("PWY-5514", 245, 128, "UDP-GalNAc"),
    ("PWY-7090", 245, 256, "UDP-ManNAc"),
]

print(f"\n{'Pathway':<25} {'Ratio':<10} {'Value':>8} {'n×d':>8} {'Prime':>6}  Name")
print("-" * 80)

for pid, n, d, name in new_assignments:
    ratio = HARMONIC_MAP.get_ratio(pid)
    if ratio:
        value = ratio.n / ratio.d
        consonance = ratio.n * ratio.d
        plimit = ratio.prime_limit()
        print(f"{pid:<25} {ratio!s:<10} {value:>8.3f} {consonance:>8} {plimit:>6}  {name}")
    else:
        print(f"{pid:<25} ✗ NOT ASSIGNED")

# Summary by prime family
print("\n" + "=" * 70)
print("PRIME FAMILIES USED")
print("=" * 70)
print("""
  27 (3³)     — Nitrogen cycle: ornithine, citrulline, butanediol
  25 (5²)     — Purine endpoint: urate
  45 (9×5)    — Nitrogen transport: ureide  
  33 (3×11)   — Central metabolism: acetyl-CoA
  9 (3²)      — Stress response: mannosylglycerate
  15 (3×5)    — Signaling: inositol-P3
  91 (7×13)   — Siderophores: enterobactin, aerobactin
  55 (5×11)   — Antioxidant/archaea: mycothiol, ergothioneine, factor 420
  77 (7×11)   — Quinone precursor: naphthoate
  143 (11×13) — Archaeal: CDP-archaeol, tetrahydromethanopterin
  231 (3×7×11)— B12 precursor: cob(II)yrinate
  245 (7²×5)  — Surface sugars: UDP-GalNAc, UDP-ManNAc
""")

# Total count
print("=" * 70)
print(f"TOTAL BIOSYNTHESIS ASSIGNMENTS: {sum(1 for pid, meta in HARMONIC_MAP.metadata.items() if meta.get('category') == 'biosynthesis')}")
print(f"TOTAL ALL ASSIGNMENTS: {len(HARMONIC_MAP.assignments)}")
print("=" * 70)

# Save
HARMONIC_MAP.save()
print("\n✓ Saved to harmonic_map.json")

BIOSYNTHESIS COMPLETION — 19 Pathways Added

Pathway                   Ratio         Value      n×d  Prime  Name
--------------------------------------------------------------------------------
GLUTORN-PWY               27/5          5.400      135      5  ornithine
CITRULBIO-PWY             27/10         2.700      270      5  citrulline
P125-PWY                  27/20         1.350      540      5  butanediol
PWY-5695                  25/9          2.778      225      5  urate
URSIN-PWY                 45/8          5.625      360      5  ureide
PWY-5173                  33/16         2.062      528     11  acetyl-CoA
PWY-5656                  9/5           1.800       45      5  mannosylglycerate
ENTBACSYN-PWY             91/8         11.375      728     13  enterobactin
AEROBACTINSYN-PWY         91/16         5.688     1456     13  aerobactin
PWY1G-0                   55/16         3.438      880     11  mycothiol
PWY-7255                  55/32         1.719     1760     11  ergot

In [79]:
# ============================================================
# BIOSYNTHESIS — REMAINING SUBCATEGORIES & PATHWAYS
# Orientation / triage cell (non-assigning)
# ============================================================

print("=" * 70)
print("🧬 BIOSYNTHESIS — REMAINING WORK OVERVIEW")
print("=" * 70)

# Work from biosynthesis slice
biosyn_df = df[df["category"] == "biosynthesis"].copy()

# Add assignment status
biosyn_df["assigned"] = biosyn_df["pathway_id"].apply(HARMONIC_MAP.is_assigned)

total_biosyn = len(biosyn_df)
assigned_biosyn = biosyn_df["assigned"].sum()

print(f"\nTotal biosynthesis pathways : {total_biosyn}")
print(f"Assigned                   : {assigned_biosyn}")
print(f"Remaining                  : {total_biosyn - assigned_biosyn}")
print(f"Completion                 : {assigned_biosyn / total_biosyn * 100:.1f}%")

# ------------------------------------------------------------
# 1️⃣ Subcategory-level completion status
# ------------------------------------------------------------

print(f"\n{'-'*70}")
print("SUBCATEGORY STATUS")
print("-"*70)

subcat_summary = (
    biosyn_df
    .groupby("subcategory")
    .agg(
        total=("pathway_id", "count"),
        assigned=("assigned", "sum"),
    )
    .sort_values("total", ascending=False)
)

subcat_summary["remaining"] = subcat_summary["total"] - subcat_summary["assigned"]
subcat_summary["pct_complete"] = (
    subcat_summary["assigned"] / subcat_summary["total"] * 100
).round(1)

display(subcat_summary)

# ------------------------------------------------------------
# 2️⃣ Focus: subcategories with remaining work
# ------------------------------------------------------------

print(f"\n{'-'*70}")
print("SUBCATEGORIES WITH REMAINING PATHWAYS")
print("-"*70)

open_subcats = subcat_summary[subcat_summary["remaining"] > 0]

if len(open_subcats) == 0:
    print("✅ All biosynthesis subcategories are complete!")
else:
    display(open_subcats)

# ------------------------------------------------------------
# 3️⃣ Detailed pathway view (remaining only)
# ------------------------------------------------------------

print(f"\n{'-'*70}")
print("REMAINING BIOSYNTHESIS PATHWAYS (PRIORITIZED)")
print("-"*70)

remaining_df = (
    biosyn_df[~biosyn_df["assigned"]]
    .sort_values(
        by=["subcategory", "prevalence", "median_abundance"],
        ascending=[True, False, False]
    )
)

if len(remaining_df) == 0:
    print("✅ No remaining biosynthesis pathways.")
else:
    display(
        remaining_df[
            [
                "pathway_id",
                "pathway_name",
                "subcategory",
                "prevalence",
                "prevalence_pct",
                "median_abundance",
                "mean_abundance",
            ]
        ]
    )

# ------------------------------------------------------------
# 4️⃣ High-impact remaining pathways (cross-subcategory)
# ------------------------------------------------------------

print(f"\n{'-'*70}")
print("HIGH-PREVALENCE REMAINING PATHWAYS (GLOBAL VIEW)")
print("-"*70)

high_impact = (
    biosyn_df[~biosyn_df["assigned"]]
    .sort_values(
        by=["prevalence", "median_abundance"],
        ascending=False
    )
    .head(20)
)

if len(high_impact) == 0:
    print("✅ No high-impact unassigned pathways.")
else:
    display(
        high_impact[
            [
                "pathway_id",
                "pathway_name",
                "subcategory",
                "prevalence",
                "prevalence_pct",
                "median_abundance",
            ]
        ]
    )

print("\n" + "=" * 70)
print("✅ BIOSYNTHESIS ORIENTATION COMPLETE")
print("=" * 70)

🧬 BIOSYNTHESIS — REMAINING WORK OVERVIEW

Total biosynthesis pathways : 143
Assigned                   : 143
Remaining                  : 0
Completion                 : 100.0%

----------------------------------------------------------------------
SUBCATEGORY STATUS
----------------------------------------------------------------------


,total,assigned,remaining,pct_complete
subcategory,,,,
Amino Acids,28,28,0,100.0
Cofactors/Vitamins,28,28,0,100.0
Cell Envelope,24,24,0,100.0
Fatty Acids/Lipids,17,17,0,100.0
Specialized,14,14,0,100.0
Other,12,12,0,100.0
Isoprenoids,7,7,0,100.0
Nucleotides,7,7,0,100.0
Polyamines,6,6,0,100.0



----------------------------------------------------------------------
SUBCATEGORIES WITH REMAINING PATHWAYS
----------------------------------------------------------------------
✅ All biosynthesis subcategories are complete!

----------------------------------------------------------------------
REMAINING BIOSYNTHESIS PATHWAYS (PRIORITIZED)
----------------------------------------------------------------------
✅ No remaining biosynthesis pathways.

----------------------------------------------------------------------
HIGH-PREVALENCE REMAINING PATHWAYS (GLOBAL VIEW)
----------------------------------------------------------------------
✅ No high-impact unassigned pathways.

✅ BIOSYNTHESIS ORIENTATION COMPLETE


---
### DEGRADATION

**Character:** Catabolic, breaking down molecules → **subharmonic ratios** (n < d)

**Prime limits:** Match corresponding biosynthesis pathways

**Key principle:** Use **inverse ratios** of paired biosynthesis pathways:
- If "L-lysine biosynthesis" = 7/4
- Then "L-lysine degradation" = 4/7
- Range: 1/8 (0.125) to 16/1 (16.0)
- **Asymmetric**: 3 octaves below, 4 octaves above
- Folding: values < 1/8 → double numerator; values > 16 → double denominator

In [80]:
# ============================================================
# DEGRADATION — Overview
# ============================================================

print("═" * 70)
print(" 🔻 DEGRADATION PATHWAYS")
print("═" * 70)

print(f"""
  Total pathways:  {len(degradation_df)}
  
  Subcategories:
""")

for subcat in degradation_df['subcategory'].unique():
    count = len(degradation_df[degradation_df['subcategory'] == subcat])
    print(f"    • {subcat:<25} {count}")

print()
print("  Strategy: Subharmonic ratios (n < d) as inverses of biosynthesis")


══════════════════════════════════════════════════════════════════════
 🔻 DEGRADATION PATHWAYS
══════════════════════════════════════════════════════════════════════

  Total pathways:  115
  
  Subcategories:

    • Carbohydrates             7
    • Nucleotides               9
    • Other                     82
    • Amino Acids               12
    • Aromatics                 5

  Strategy: Subharmonic ratios (n < d) as inverses of biosynthesis


In [81]:
# ============================================================
# DEGRADATION — Paired Pathways (Biosynthesis Inverses)
# ============================================================

print("═" * 70)
print(" PAIRED DEGRADATION — Inverse Ratios")
print("═" * 70)

# These pathways have clear biosynthesis counterparts
# Degradation gets the INVERSE ratio of its biosynthesis pair

PAIRED_DEG_MAPPING = {
    # ─────────────────────────────────────────────────────────
    # CARBOHYDRATES — Inverse of storage biosynthesis
    # ─────────────────────────────────────────────────────────
    "PWY-6737":     (1, 2),    # starch V deg ← inverse of glycogen bio (2/1)
    "GLYCOCAT-PWY": (3, 4),    # glycogen I deg ← inverse of starch bio (4/3)
    "PWY-5941":     (3, 8),    # glycogen II deg
    "PWY-6731":     (1, 4),    # starch III deg
    "PWY-6317":     (2, 3),    # galactose Leloir
    "LACTOSECAT-PWY": (5, 6),  # lactose/galactose
    "GLUCOSE1PMETAB-PWY": (5, 8), # glucose-1-P
    "PWY-6901":     (1, 3),    # glucose + xylose super
    "DHGLUCONATE-PYR-CAT-PWY": (2, 9), # gluconate oxidative
    
    # ─────────────────────────────────────────────────────────
    # AMINO ACIDS — Inverse of AA biosynthesis
    # ─────────────────────────────────────────────────────────
    "HISDEG-PWY":   (3, 5),    # histidine deg ← inverse of HISTSYN-PWY (5/3)
    "VALDEG-PWY":   (2, 7),    # valine deg ← inverse of VALSYN-PWY (7/2)
    "PWY-5651":     (20, 21),  # tryptophan XI ← inverse of TRPSYN-PWY (21/20)
    "TYRFUMCAT-PWY": (10, 21), # tyrosine I
    "PWY-5088":     (4, 7),    # glutamate VIII → propanoate
    "PWY-5028":     (6, 5),    # histidine II variant
    "PWY-5177":     (8, 7),    # glutaryl-CoA (Lys/Trp converge)
    "ARGDEG-PWY":   (8, 9),    # arginine super (GABA route)
    "P162-PWY":     (12, 5),   # glutamate V → TCA
    "AST-PWY":      (16, 21),  # arginine AST
    "ORNARGDEG-PWY": (14, 9),  # ornithine super
    "GLUDEG-II-PWY": (5, 7),   # glutamate VII → butanoate
    
    # ─────────────────────────────────────────────────────────
    # NUCLEOTIDES — Inverse of nucleotide biosynthesis (5-series)
    # ─────────────────────────────────────────────────────────
    "PWY0-1296":    (1, 5),    # purine ribonucleosides deg
    "PWY-6608":     (2, 5),    # guanosine deg
    "PWY-7209":     (4, 5),    # purine deoxyribonucleosides
    "P164-PWY":     (8, 5),    # purine anaerobic
    "PWY-5532":     (16, 5),   # adenosine IV Archaea
    "SALVADEHYPOX-PWY": (3, 10), # adenosine II
    "PWY-6353":     (6, 25),   # purine II aerobic
    "PWY0-1297":    (2, 15),   # purine deoxyribo super
    "PWY0-1298":    (4, 15),   # pyrimidine deoxyribo super
}

PAIRED_DEG_NOTES = {
    "PWY-6737":     "starch V — THE canonical breakdown (1/2)",
    "GLYCOCAT-PWY": "glycogen I bacterial (3/4)",
    "HISDEG-PWY":   "histidine deg ← inverse of HISTSYN (3/5)",
    "VALDEG-PWY":   "valine deg ← inverse of VALSYN (2/7)",
    "PWY-5651":     "tryptophan XI ← inverse of TRPSYN (20/21)",
    "PWY0-1296":    "purine ribonucleosides — 5-series (1/5)",
}

paired_count = 0
for pathway_id, (n, d) in PAIRED_DEG_MAPPING.items():
    if HARMONIC_MAP.is_assigned(pathway_id):
        continue
    if HARMONIC_MAP.is_ratio_used(n, d):
        print(f"  ⚠ {n}/{d} in use — skip {pathway_id}")
        continue
    
    row = degradation_df[degradation_df["pathway_id"] == pathway_id]
    if len(row) == 0:
        continue
    row = row.iloc[0]
    
    note = PAIRED_DEG_NOTES.get(pathway_id, "")
    if HARMONIC_MAP.assign(
        pathway_id,
        Ratio(n, d),
        {
            "pathway_name": row["pathway_name"],
            "subcategory": row["subcategory"],
            "category": "degradation",
            "prevalence": row["prevalence"],
            "derivation": note,
        }
    ):
        paired_count += 1
        print(f"  ✓ {pathway_id:<30} → {n}/{d}")

print(f"\nAssigned: {paired_count} paired degradation pathways")
HARMONIC_MAP.save()


══════════════════════════════════════════════════════════════════════
 PAIRED DEGRADATION — Inverse Ratios
══════════════════════════════════════════════════════════════════════
  ✓ PWY-6737                       → 1/2
  ✓ GLYCOCAT-PWY                   → 3/4
  ✓ PWY-5941                       → 3/8
  ✓ PWY-6731                       → 1/4
  ✓ PWY-6317                       → 2/3
  ✓ LACTOSECAT-PWY                 → 5/6
  ✓ GLUCOSE1PMETAB-PWY             → 5/8
  ✓ PWY-6901                       → 1/3
  ✓ DHGLUCONATE-PYR-CAT-PWY        → 2/9
  ✓ HISDEG-PWY                     → 3/5
  ✓ VALDEG-PWY                     → 2/7
  ✓ PWY-5651                       → 20/21
  ✓ TYRFUMCAT-PWY                  → 10/21
  ✓ PWY-5088                       → 4/7
  ✓ PWY-5028                       → 6/5
  ✓ PWY-5177                       → 8/7
  ✓ ARGDEG-PWY                     → 8/9
  ✓ P162-PWY                       → 12/5
  ✓ AST-PWY                        → 16/21
  ✓ ORNARGDEG-PWY                  

In [82]:
# ============================================================
# DEGRADATION — Fiber, Mucin & Dietary Substrates
# ============================================================

print("═" * 70)
print(" FIBER & DIETARY DEGRADATION")
print("═" * 70)

# High-prevalence pathways processing dietary fiber,
# oligosaccharides, and mucin components
# Using 7-limit and 9-limit subharmonics

FIBER_DEG_MAPPING = {
    # ─────────────────────────────────────────────────────────
    # OLIGOSACCHARIDES (universal)
    # ─────────────────────────────────────────────────────────
    "PWY-6527":   (1, 6),     # stachyose — dietary oligosaccharide
    "RHAMCAT-PWY": (1, 9),    # rhamnose — plant fiber
    "PWY-7242":   (5, 9),     # fructuronate — pectin
    
    # ─────────────────────────────────────────────────────────
    # MUCIN (host-microbe interface) — 7-series
    # ─────────────────────────────────────────────────────────
    "GLCMANNANAUT-PWY": (10, 9),  # GlcNAc/ManNAc/NeuAc — mucin core
    "P441-PWY":        (3, 7),    # N-acetylneuraminate — sialic acid
    "FUCCAT-PWY":      (12, 7),   # fucose — mucin/HMO
    
    # ─────────────────────────────────────────────────────────
    # PECTIN / URONATE (fiber) — 7-series
    # ─────────────────────────────────────────────────────────
    "GLUCUROCAT-PWY":        (10, 7),  # glucuronide
    "PWY-6507":              (6, 7),   # 4-deoxy-threo-hex
    "GALACTUROCAT-PWY":      (1, 14),  # galacturonate I
    "GALACT-GLUCUROCAT-PWY": (3, 14),  # hexuronide super
    
    # ─────────────────────────────────────────────────────────
    # SUCROSE (dietary sugar) — 9-series
    # ─────────────────────────────────────────────────────────
    "PWY-621":    (17, 2),    # sucrose III invertase
    "PWY-5384":   (9, 16),    # sucrose IV phosphorylase
    "PWY66-373":  (9, 32),    # sucrose V
    "PWY-3801":   (9, 64),    # sucrose II (rare)
    
    # ─────────────────────────────────────────────────────────
    # MANNAN & POLYOLS
    # ─────────────────────────────────────────────────────────
    "PWY-7456":           (7, 8),    # mannan — plant fiber
    "HEXITOLDEGSUPER-PWY": (3, 16),  # hexitol bacterial
    "PWY-2723":           (3, 32),   # trehalose — stress sugar
}

fiber_count = 0
for pathway_id, (n, d) in FIBER_DEG_MAPPING.items():
    if HARMONIC_MAP.is_assigned(pathway_id):
        continue
    if HARMONIC_MAP.is_ratio_used(n, d):
        print(f"  ⚠ {n}/{d} in use — skip {pathway_id}")
        continue
    
    row = degradation_df[degradation_df["pathway_id"] == pathway_id]
    if len(row) == 0:
        continue
    row = row.iloc[0]
    
    if HARMONIC_MAP.assign(
        pathway_id,
        Ratio(n, d),
        {
            "pathway_name": row["pathway_name"],
            "subcategory": row["subcategory"],
            "category": "degradation",
            "prevalence": row["prevalence"],
        }
    ):
        fiber_count += 1
        print(f"  ✓ {pathway_id:<30} → {n}/{d}")

print(f"\nAssigned: {fiber_count} fiber/dietary degradation pathways")
HARMONIC_MAP.save()


══════════════════════════════════════════════════════════════════════
 FIBER & DIETARY DEGRADATION
══════════════════════════════════════════════════════════════════════
  ✓ PWY-6527                       → 1/6
  ✓ RHAMCAT-PWY                    → 1/9
  ✓ PWY-7242                       → 5/9
  ✓ GLCMANNANAUT-PWY               → 10/9
  ✓ P441-PWY                       → 3/7
  ✓ FUCCAT-PWY                     → 12/7
  ✓ GLUCUROCAT-PWY                 → 10/7
  ✓ PWY-6507                       → 6/7
  ✓ GALACTUROCAT-PWY               → 1/14
  ✓ GALACT-GLUCUROCAT-PWY          → 3/14
  ✓ PWY-621                        → 17/2
  ✓ PWY-5384                       → 9/16
  ✓ PWY66-373                      → 9/32
  ✓ PWY-3801                       → 9/64
  ✓ PWY-7456                       → 7/8
  ✓ HEXITOLDEGSUPER-PWY            → 3/16
  ✓ PWY-2723                       → 3/32

Assigned: 17 fiber/dietary degradation pathways
✓ Saved 217 assignments to harmonic_map.json


In [83]:
# ============================================================
# DEGRADATION — Aromatics (11-limit)
# ============================================================

print("═" * 70)
print(" AROMATIC DEGRADATION — 11-limit")
print("═" * 70)

# Complex ring structures warrant 11-limit ratios

AROMATIC_DEG_MAPPING = {
    # ─────────────────────────────────────────────────────────
    # CORE AROMATICS — 11-series
    # ─────────────────────────────────────────────────────────
    "PWY-5431": (1, 11),     # β-ketoadipate — canonical
    "PWY-5415": (2, 11),     # catechol meta — ring cleavage
    "PWY-7431": (4, 11),     # aromatic biogenic amine
    "PWY-5178": (8, 11),     # toluene IV — methylbenzene
    "PWY-6215": (3, 11),     # 4-chlorobenzoate
    
    # ─────────────────────────────────────────────────────────
    # PHENYLPROPANOIDS
    # ─────────────────────────────────────────────────────────
    "HCAMHPDEG-PWY": (3, 22),  # phenylpropanoate I
    "PWY-6690":      (6, 11),  # cinnamate
    "PWY0-1277":     (5, 11),  # phenylpropanoate II
    "PWY0-321":      (7, 11),  # phenylacetate
    "PWY-7046":      (9, 11),  # coumarate
    
    # ─────────────────────────────────────────────────────────
    # TOLUENE SERIES
    # ─────────────────────────────────────────────────────────
    "PWY-5180":  (16, 11),   # toluene I
    "PWY-5181":  (32, 11),   # toluene III
    "PWY-5183":  (64, 11),   # toluene super
    
    # ─────────────────────────────────────────────────────────
    # SPECIALIZED AROMATICS
    # ─────────────────────────────────────────────────────────
    "PROTOCATECHUATE-ORTHO-CLEAVAGE-PWY": (13, 7),  # protocatechuate
    "3-HYDROXYPHENYLACETATE-DEGRADATION-PWY": (14, 11),  # hydroxyphenylacetate
    "PWY-6071":  (9, 13),    # phenylethylamine
    
    # ─────────────────────────────────────────────────────────
    # GALLATE (rare)
    # ─────────────────────────────────────────────────────────
    "METHYLGALLATE-DEGRADATION-PWY": (22, 5),   # methylgallate
    "GALLATE-DEGRADATION-I-PWY":     (15, 11),  # gallate II
    "GALLATE-DEGRADATION-II-PWY":    (11, 15),  # gallate I (inverse)
}

arom_count = 0
for pathway_id, (n, d) in AROMATIC_DEG_MAPPING.items():
    if HARMONIC_MAP.is_assigned(pathway_id):
        continue
    if HARMONIC_MAP.is_ratio_used(n, d):
        print(f"  ⚠ {n}/{d} in use — skip {pathway_id}")
        continue
    
    row = degradation_df[degradation_df["pathway_id"] == pathway_id]
    if len(row) == 0:
        continue
    row = row.iloc[0]
    
    if HARMONIC_MAP.assign(
        pathway_id,
        Ratio(n, d),
        {
            "pathway_name": row["pathway_name"],
            "subcategory": row["subcategory"],
            "category": "degradation",
            "prevalence": row["prevalence"],
        }
    ):
        arom_count += 1
        print(f"  ✓ {pathway_id:<45} → {n}/{d}")

print(f"\nAssigned: {arom_count} aromatic degradation pathways")
HARMONIC_MAP.save()


══════════════════════════════════════════════════════════════════════
 AROMATIC DEGRADATION — 11-limit
══════════════════════════════════════════════════════════════════════
  ✓ PWY-5431                                      → 1/11
  ✓ PWY-5415                                      → 2/11
  ✓ PWY-7431                                      → 4/11
  ✓ PWY-5178                                      → 8/11
  ✓ PWY-6215                                      → 3/11
  ✓ HCAMHPDEG-PWY                                 → 3/22
  ✓ PWY-6690                                      → 6/11
  ✓ PWY0-1277                                     → 5/11
  ✓ PWY0-321                                      → 7/11
  ✓ PWY-7046                                      → 9/11
  ✓ PWY-5180                                      → 16/11
  ✓ PWY-5181                                      → 32/11
  ✓ PWY-5183                                      → 64/11
  ✓ PROTOCATECHUATE-ORTHO-CLEAVAGE-PWY            → 13/7
  ✓ 3-HYDROXYPHENYLACETA

In [84]:
# ============================================================
# DEGRADATION — Other Substrates
# ============================================================

print("═" * 70)
print(" OTHER DEGRADATION")
print("═" * 70)

OTHER_DEG_MAPPING = {
    # ─────────────────────────────────────────────────────────
    # ALLANTOIN / URIC ACID — 13-series
    # ─────────────────────────────────────────────────────────
    "ALLANTOINDEG-PWY": (1, 13),  # allantoin
    "PWY0-41":          (2, 13),  # allantoin IV anaerobic
    
    # ─────────────────────────────────────────────────────────
    # PROPANEDIOL / GLYCEROL
    # ─────────────────────────────────────────────────────────
    "GOLPDLCAT-PWY":    (19, 2),  # glycerol → propanediol
    "PWY-7013":         (23, 2),  # propanediol
    "PWY-7003":         (17, 3),  # glycerol → butanol
    "FUC-RHAMCAT-PWY":  (29, 2),  # fucose+rhamnose
    
    # ─────────────────────────────────────────────────────────
    # SUGAR ACIDS
    # ─────────────────────────────────────────────────────────
    "GALACTARDEG-PWY":  (19, 3),  # galactarate
    "GLUCARDEG-PWY":    (23, 3),  # glucarate
    "GLUCARGALACTSUPER-PWY": (22, 9),  # glucar/galactar super
    
    # ─────────────────────────────────────────────────────────
    # NITROGEN COMPOUNDS — 21-series
    # ─────────────────────────────────────────────────────────
    "CRNFORCAT-PWY": (1, 21),   # creatinine I
    "PWY-4722":      (2, 21),   # creatinine II
    "ORNDEG-PWY":    (21, 5),   # ornithine
    "PWY-6344":      (8, 21),   # ornithine II Stickland
    
    # ─────────────────────────────────────────────────────────
    # TAURINE / SULFUR
    # ─────────────────────────────────────────────────────────
    "PWY-1541": (1, 42),        # taurine
    
    # ─────────────────────────────────────────────────────────
    # GLYCOL
    # ─────────────────────────────────────────────────────────
    "GLYCOL-GLYOXDEG-PWY": (13, 3),  # glycol
    "METHGLYUT-PWY":       (7, 9),   # methylglyoxal
    
    # ─────────────────────────────────────────────────────────
    # INOSITOL
    # ─────────────────────────────────────────────────────────
    "PWY-7237": (1, 18),        # inositol
    "P562-PWY": (1, 36),        # inositol I
    
    # ─────────────────────────────────────────────────────────
    # METHYLPHOSPHONATE
    # ─────────────────────────────────────────────────────────
    "PWY0-1533": (1, 26),       # methylphosphonate I
    "PWY-7399":  (1, 52),       # methylphosphonate II
    
    # ─────────────────────────────────────────────────────────
    # VITAMINS / COFACTORS — 5-series
    # ─────────────────────────────────────────────────────────
    "PWY-5499": (5, 14),        # vitamin B6
    "PWY-722":  (7, 15),        # nicotinate I
    "PWY-5055": (5, 42),        # nicotinate III
    
    # ─────────────────────────────────────────────────────────
    # NEUROTRANSMITTER-RELATED
    # ─────────────────────────────────────────────────────────
    "PWY-5022": (1, 22),        # GABA V
    
    # ─────────────────────────────────────────────────────────
    # XYLOSE/ARABINOSE — 5-series deep
    # ─────────────────────────────────────────────────────────
    "PWY-7294": (5, 18),        # xylose IV
    "PWY-6760": (5, 36),        # xylose III
    "PWY-7295": (5, 72),        # arabinose
    
    # ─────────────────────────────────────────────────────────
    # ANAEROBIC SUGARS — 27/31 series
    # ─────────────────────────────────────────────────────────
    "PWY-7345": (31, 2),        # anaerobic sucrose
    "PWY-6992": (31, 3),        # anhydrofructose
    
    # ─────────────────────────────────────────────────────────
    # TRYPTOPHAN DOWNSTREAM
    # ─────────────────────────────────────────────────────────
    "PWY-5654": (20, 11),       # amino-carboxymuconate
    
    # ─────────────────────────────────────────────────────────
    # BENZOATE/MANDELATE — 13-limit
    # ─────────────────────────────────────────────────────────
    "PWY-1361": (13, 8),        # benzoyl-CoA
    "PWY-1501": (13, 6),        # mandelate I
    "PWY-6957": (26, 11),       # mandelate II
    "PWY-6107": (13, 33),       # chlorosalicylate
    
    # ─────────────────────────────────────────────────────────
    # METHIONINE RELATED
    # ─────────────────────────────────────────────────────────
    "PWY-4361": (35, 12),       # methylthioribose
    "PWY-5328": (7, 15),        # methionine salvage/deg... wait this conflicts with PWY-722!
    "PWY-6641": (45, 14),       # sulfolactate
    
    # ─────────────────────────────────────────────────────────
    # SPECIALIZED
    # ─────────────────────────────────────────────────────────
    "PWY66-389":  (22, 7),      # phytol
    "PWY-6572":   (7, 10),      # chondroitin sulfate
    "PWY-6182":   (7, 22),      # salicylate
    "PWY-6948":   (11, 14),     # sitosterol
    "PWY-6906":   (7, 13),      # chitin derivatives
    "PWY-7118":   (11, 13),     # chitin → ethanol
    
    # ─────────────────────────────────────────────────────────
    # MALONATE & SYRINGATE
    # ─────────────────────────────────────────────────────────
    "PWY-6060": (4, 13),        # malonate
    "PWY-6339": (5, 13),        # syringate
    
    # ─────────────────────────────────────────────────────────
    # RARE
    # ─────────────────────────────────────────────────────────
    "PWY-6486": (7, 16),        # galacturonate II
    "P184-PWY": (13, 4),        # protocatechuate I
    "PWY-3661": (25, 7),        # glycine betaine
}

# Fix conflict: PWY-5328 and PWY-722 both had 7/15
# Change PWY-5328 to different ratio
OTHER_DEG_MAPPING["PWY-5328"] = (11, 21)  # methionine deg

other_count = 0
for pathway_id, (n, d) in OTHER_DEG_MAPPING.items():
    if HARMONIC_MAP.is_assigned(pathway_id):
        continue
    if HARMONIC_MAP.is_ratio_used(n, d):
        print(f"  ⚠ {n}/{d} in use — skip {pathway_id}")
        continue
    
    row = degradation_df[degradation_df["pathway_id"] == pathway_id]
    if len(row) == 0:
        continue
    row = row.iloc[0]
    
    if HARMONIC_MAP.assign(
        pathway_id,
        Ratio(n, d),
        {
            "pathway_name": row["pathway_name"],
            "subcategory": row["subcategory"],
            "category": "degradation",
            "prevalence": row["prevalence"],
        }
    ):
        other_count += 1
        print(f"  ✓ {pathway_id:<35} → {n}/{d}")

print(f"\nAssigned: {other_count} other degradation pathways")
HARMONIC_MAP.save()


══════════════════════════════════════════════════════════════════════
 OTHER DEGRADATION
══════════════════════════════════════════════════════════════════════
  ✓ ALLANTOINDEG-PWY                    → 1/13
  ✓ PWY0-41                             → 2/13
  ✓ GOLPDLCAT-PWY                       → 19/2
  ✓ PWY-7013                            → 23/2
  ✓ PWY-7003                            → 17/3
  ✓ FUC-RHAMCAT-PWY                     → 29/2
  ✓ GALACTARDEG-PWY                     → 19/3
  ✓ GLUCARDEG-PWY                       → 23/3
  ✓ GLUCARGALACTSUPER-PWY               → 22/9
  ✓ CRNFORCAT-PWY                       → 1/21
  ✓ PWY-4722                            → 2/21
  ✓ ORNDEG-PWY                          → 21/5
  ✓ PWY-6344                            → 8/21
  ✓ PWY-1541                            → 1/42
  ✓ GLYCOL-GLYOXDEG-PWY                 → 13/3
  ✓ METHGLYUT-PWY                       → 7/9
  ✓ PWY-7237                            → 1/18
  ✓ P562-PWY                            →

In [85]:
# ============================================================
# DEGRADATION — Summary
# ============================================================

print("═" * 70)
print(" DEGRADATION SUMMARY")
print("═" * 70)

total_deg = len(degradation_df)
assigned = [pid for pid in degradation_df["pathway_id"] if HARMONIC_MAP.is_assigned(pid)]
remaining = [pid for pid in degradation_df["pathway_id"] if not HARMONIC_MAP.is_assigned(pid)]

print(f"""
  Total degradation pathways:   {total_deg}
  Assigned:                     {len(assigned)}
  Remaining:                    {len(remaining)}
""")

# By subcategory
print("By subcategory:")
for subcat in degradation_df["subcategory"].unique():
    sub_ids = degradation_df[degradation_df["subcategory"] == subcat]["pathway_id"].tolist()
    sub_assigned = [pid for pid in sub_ids if HARMONIC_MAP.is_assigned(pid)]
    status = "✓" if len(sub_assigned) == len(sub_ids) else "○"
    print(f"  {status} {subcat:<35} {len(sub_assigned):>3}/{len(sub_ids)}")

if remaining:
    print(f"\nRemaining ({len(remaining)}):")
    rem_df = degradation_df[degradation_df["pathway_id"].isin(remaining)]
    rem_df = rem_df.sort_values("prevalence", ascending=False)
    for _, row in rem_df.iterrows():
        print(f"  {row['prevalence']:.4f}  {row['pathway_id']}")

HARMONIC_MAP.save()
print(f"\n✓ Map: {HARMONIC_MAP.status()}")


══════════════════════════════════════════════════════════════════════
 DEGRADATION SUMMARY
══════════════════════════════════════════════════════════════════════

  Total degradation pathways:   115
  Assigned:                     114
  Remaining:                    1

By subcategory:
  ✓ Carbohydrates                         7/7
  ✓ Nucleotides                           9/9
  ○ Other                                81/82
  ✓ Amino Acids                          12/12
  ✓ Aromatics                             5/5

Remaining (1):
  0.0018  LEU-DEG2-PWY
✓ Saved 284 assignments to harmonic_map.json

✓ Map: 284 pathways | 284 ratios


---
### OTHER / UNCLASSIFIED

**Character:** Cross-cutting processes that don't fit neatly into biosynthesis/degradation

**Strategy:** Use biologically meaningful ratios based on pathway function

In [86]:
# ============================================================
# OTHER — Unclassified Pathways
# ============================================================

print("═" * 70)
print(" OTHER / UNCLASSIFIED PATHWAYS")
print("═" * 70)

OTHER_MAPPING = {
    # ─────────────────────────────────────────────────────────
    # UNIVERSAL PROCESSES — 35-series (5×7 = methylation bridge)
    # ─────────────────────────────────────────────────────────
    'PWY-6151':          (4, 35),   # SAM cycle I — methylation
    'TRNA-CHARGING-PWY': (6, 35),   # tRNA charging — THE bridge
    'PWY0-1586':         (35, 64),  # peptidoglycan maturation
    'PWY-4041':          (9, 35),   # γ-glutamyl cycle
    'PWY0-1479':         (70, 9),   # tRNA processing
    'PWY0-1261':         (7, 32),   # anhydromuropeptides
    
    # ─────────────────────────────────────────────────────────
    # NUCLEOTIDE
    # ─────────────────────────────────────────────────────────
    'PWY-7197': (5, 12),            # pyrimidine deoxyNTP phosph
    
    # ─────────────────────────────────────────────────────────
    # FATTY ACID CYCLING (inverse pairs)
    # ─────────────────────────────────────────────────────────
    'FASYN-ELONG-PWY': (49, 6),     # FA elongation
    'FAO-PWY':         (6, 49),     # FA β-oxidation I — inverse!
    'PWY-5138':        (3, 49),     # unsaturated FA β-ox
    'PWY-6837':        (4, 49),     # FA β-ox V
    'PWY66-388':       (8, 49),     # FA α-ox III
    'PWY-6803':        (7, 48),     # phosphatidylcholine
    'PWY-7039':        (7, 64),     # phosphatidate signaling
    'LIPASYN-PWY':     (7, 81),     # phospholipases
    
    # ─────────────────────────────────────────────────────────
    # NITROGEN/UREA
    # ─────────────────────────────────────────────────────────
    'PWY-4984': (9, 14),            # urea cycle
    
    # ─────────────────────────────────────────────────────────
    # METHANOGENESIS (ancient! pure 3-limit)
    # ─────────────────────────────────────────────────────────
    'METH-ACETATE-PWY':   (81, 4),  # acetate → methane
    'METHANOGENESIS-PWY': (81, 16), # H2/CO2 → methane
    'PWY-6174':           (81, 32), # mevalonate II archaea
    
    # ─────────────────────────────────────────────────────────
    # C1 METABOLISM (Pythagorean 3-limit)
    # ─────────────────────────────────────────────────────────
    'PWY-1861':  (2, 27),           # formaldehyde RuMP
    'P185-PWY':  (16, 27),          # formaldehyde III
    'RUMP-PWY':  (4, 27),           # formaldehyde ox I
    'PWY-7616':  (8, 27),           # methanol ox
    'PWY-1622':  (1, 27),           # formaldehyde serine
    
    # ─────────────────────────────────────────────────────────
    # GABA SHUNT — Neurotransmitter! 11-limit
    # ─────────────────────────────────────────────────────────
    'GLUDEG-I-PWY': (11, 16),       # GABA shunt
    
    # ─────────────────────────────────────────────────────────
    # AMINO ACID INTERCONVERSION
    # ─────────────────────────────────────────────────────────
    'ARGORNPROST-PWY': (21, 19),    # Arg/Orn/Pro
    
    # ─────────────────────────────────────────────────────────
    # ISOPRENOID (MEP pathway)
    # ─────────────────────────────────────────────────────────
    'NONMEVIPP-PWY': (8, 15),       # MEP I
    'PWY-7560':      (20, 7),       # MEP II
    'PWY-922':       (15, 14),      # mevalonate I
    
    # ─────────────────────────────────────────────────────────
    # SULFUR
    # ─────────────────────────────────────────────────────────
    'SO4ASSIM-PWY': (45, 16),       # sulfate reduction I
    
    # ─────────────────────────────────────────────────────────
    # BIFIDOBACTERIUM — Probiotic!
    # ─────────────────────────────────────────────────────────
    'P124-PWY': (18, 5),            # Bifido shunt
    
    # ─────────────────────────────────────────────────────────
    # NAD PHOSPHORYLATION
    # ─────────────────────────────────────────────────────────
    'PWY-5083': (22, 3),            # NAD/NADH phosph
    
    # ─────────────────────────────────────────────────────────
    # MANNITOL
    # ─────────────────────────────────────────────────────────
    'PWY-6531': (7, 12),            # mannitol cycle
    
    # ─────────────────────────────────────────────────────────
    # NITRATE — 3-limit deep subharmonics
    # ─────────────────────────────────────────────────────────
    'PWY-5675':           (5, 27),  # nitrate V
    'PWY490-3':           (3, 20),  # nitrate VI
    'DENITRIFICATION-PWY': (3, 40), # denitrification
    'PWY-6748':           (3, 80),  # nitrate VII
    
    # ─────────────────────────────────────────────────────────
    # KETOGLUCONATE
    # ─────────────────────────────────────────────────────────
    'KETOGLUCONMET-PWY': (27, 7),   # ketogluconate
    
    # ─────────────────────────────────────────────────────────
    # RUBISCO
    # ─────────────────────────────────────────────────────────
    'PWY-5723': (24, 5),            # Rubisco shunt
    
    # ─────────────────────────────────────────────────────────
    # HYDROCARBON
    # ─────────────────────────────────────────────────────────
    'P221-PWY': (7, 27),            # octane ox
    
    # ─────────────────────────────────────────────────────────
    # POLYMYXIN RESISTANCE
    # ─────────────────────────────────────────────────────────
    'PWY0-1338': (28, 13),          # polymyxin resistance
    
    # ─────────────────────────────────────────────────────────
    # KETOGENESIS
    # ─────────────────────────────────────────────────────────
    'PWY66-367': (7, 18),           # ketogenesis
    
    # ─────────────────────────────────────────────────────────
    # HYDROGEN
    # ─────────────────────────────────────────────────────────
    'PWY-6785': (1, 16),            # H2 production
    
    # ─────────────────────────────────────────────────────────
    # N-GLYCOSYLATION
    # ─────────────────────────────────────────────────────────
    'PWY-7031': (7, 24),            # N-glycosylation bacterial
    
    # ─────────────────────────────────────────────────────────
    # ACETOGENESIS
    # ─────────────────────────────────────────────────────────
    'CODH-PWY': (1, 81),            # reductive acetyl-CoA
    
    # ─────────────────────────────────────────────────────────
    # FOLATE
    # ─────────────────────────────────────────────────────────
    'PWY-3841': (11, 18),           # folate II
    'PWY-2201': (18, 11),           # folate I — inverse
    
    # ─────────────────────────────────────────────────────────
    # METHYLASPARTATE
    # ─────────────────────────────────────────────────────────
    'PWY-6728': (13, 9),            # methylaspartate
    
    # ─────────────────────────────────────────────────────────
    # META CLEAVAGE
    # ─────────────────────────────────────────────────────────
    'PWY-5430': (13, 11),           # meta cleavage aromatic
    
    # ─────────────────────────────────────────────────────────
    # ARSENATE
    # ─────────────────────────────────────────────────────────
    'PWY-4202': (3, 13),            # arsenate detox
}

oth_count = 0
for pathway_id, (n, d) in OTHER_MAPPING.items():
    if HARMONIC_MAP.is_assigned(pathway_id):
        continue
    if HARMONIC_MAP.is_ratio_used(n, d):
        print(f"  ⚠ {n}/{d} in use — skip {pathway_id}")
        continue
    
    row = other_df[other_df["pathway_id"] == pathway_id]
    if len(row) == 0:
        row = df[df["pathway_id"] == pathway_id]
    if len(row) == 0:
        continue
    row = row.iloc[0]
    
    if HARMONIC_MAP.assign(
        pathway_id,
        Ratio(n, d),
        {
            "pathway_name": row["pathway_name"],
            "subcategory": "Unclassified",
            "category": "other",
            "prevalence": row["prevalence"],
        }
    ):
        oth_count += 1
        print(f"  ✓ {pathway_id:<30} → {n}/{d}")

print(f"\nAssigned: {oth_count} other pathways")
HARMONIC_MAP.save()
print(f"✓ Map: {HARMONIC_MAP.status()}")


══════════════════════════════════════════════════════════════════════
 OTHER / UNCLASSIFIED PATHWAYS
══════════════════════════════════════════════════════════════════════
  ✓ PWY-6151                       → 4/35
  ✓ TRNA-CHARGING-PWY              → 6/35
  ✓ PWY0-1586                      → 35/64
  ✓ PWY-4041                       → 9/35
  ✓ PWY0-1479                      → 70/9
  ✓ PWY0-1261                      → 7/32
  ✓ PWY-7197                       → 5/12
  ✓ FASYN-ELONG-PWY                → 49/6
  ✓ FAO-PWY                        → 6/49
  ✓ PWY-5138                       → 3/49
  ✓ PWY-6837                       → 4/49
  ✓ PWY66-388                      → 8/49
  ✓ PWY-6803                       → 7/48
  ✓ PWY-7039                       → 7/64
  ✓ LIPASYN-PWY                    → 7/81
  ✓ PWY-4984                       → 9/14
  ✓ METH-ACETATE-PWY               → 81/4
  ✓ METHANOGENESIS-PWY             → 81/16
  ✓ PWY-6174                       → 81/32
  ✓ PWY-1861              

---
### SALVAGE

**Character:** Recycling, conservation → **superparticular ratios** (n/n±1)

**Key principle:** Close to unison (1/1) — small adjustments, not big transformations

**Assigned LAST** to use most consonant remaining ratios resourcefully

In [87]:
# ============================================================
# SALVAGE — Recycling Pathways
# ============================================================

print("═" * 70)
print(" SALVAGE PATHWAYS")
print("═" * 70)

# Salvage = recycling = superparticular ratios close to unison
# These are assigned LAST to use remaining consonant ratios resourcefully

SALVAGE_MAPPING = {
    # ─────────────────────────────────────────────────────────
    # NUCLEOTIDE SALVAGE
    # ─────────────────────────────────────────────────────────
    'PWY-6609':    (10, 11),   # adenine salvage III — 99.9%
    'PWY-7199':    (9, 10),    # pyrimidine deoxyribo salvage
    'PWY-7208':    (16, 15),   # pyrimidine nucleobases super
    'PWY66-409':   (15, 16),   # purine nucleotide salvage
    'PWY-7196':    (13, 14),   # pyrimidine ribo salvage
    'PWY-7200':    (17, 18),   # pyrimidine deoxyribo super
    'PWY-7224':    (25, 24),   # purine deoxyribo salvage
    
    # ─────────────────────────────────────────────────────────
    # B12 SALVAGE — Ibn Sina's ratio!
    # ─────────────────────────────────────────────────────────
    'COBALSYN-PWY': (13, 12),  # B12 salvage I — THE ratio
    'PWY-6269':     (12, 13),  # B12 salvage II — inverse
    
    # ─────────────────────────────────────────────────────────
    # THIAMIN SALVAGE
    # ─────────────────────────────────────────────────────────
    'PWY-6897':    (22, 21),   # thiamin salvage II
    
    # ─────────────────────────────────────────────────────────
    # NAD SALVAGE
    # ─────────────────────────────────────────────────────────
    'NAD-BIOSYNTHESIS-II': (12, 11),  # NAD salvage II
    'PYRIDNUCSAL-PWY':     (11, 12),  # NAD salvage I — inverse
    
    # ─────────────────────────────────────────────────────────
    # FA SALVAGE
    # ─────────────────────────────────────────────────────────
    'PWY-7094':    (14, 13),   # FA salvage
    
    # ─────────────────────────────────────────────────────────
    # METHIONINE SALVAGE
    # ─────────────────────────────────────────────────────────
    'PWY-7527':    (36, 35),   # methionine salvage III
    'PWY-7528':    (35, 36),   # methionine salvage I
}

sal_count = 0
for pathway_id, (n, d) in SALVAGE_MAPPING.items():
    if HARMONIC_MAP.is_assigned(pathway_id):
        continue
    if HARMONIC_MAP.is_ratio_used(n, d):
        print(f"  ⚠ {n}/{d} in use — skip {pathway_id}")
        continue
    
    row = salvage_df[salvage_df["pathway_id"] == pathway_id]
    if len(row) == 0:
        row = df[df["pathway_id"] == pathway_id]
    if len(row) == 0:
        continue
    row = row.iloc[0]
    
    if HARMONIC_MAP.assign(
        pathway_id,
        Ratio(n, d),
        {
            "pathway_name": row["pathway_name"],
            "subcategory": "Salvage/Recycling",
            "category": "salvage",
            "prevalence": row["prevalence"],
        }
    ):
        sal_count += 1
        print(f"  ✓ {pathway_id:<30} → {n}/{d}")

print(f"\nAssigned: {sal_count} salvage pathways")
HARMONIC_MAP.save()
print(f"✓ Map: {HARMONIC_MAP.status()}")


══════════════════════════════════════════════════════════════════════
 SALVAGE PATHWAYS
══════════════════════════════════════════════════════════════════════
  ✓ PWY-6609                       → 10/11
  ✓ PWY-7199                       → 9/10
  ✓ PWY-7208                       → 16/15
  ✓ PWY66-409                      → 15/16
  ✓ PWY-7196                       → 13/14
  ✓ PWY-7200                       → 17/18
  ✓ PWY-7224                       → 25/24
  ✓ COBALSYN-PWY                   → 13/12
  ✓ PWY-6269                       → 12/13
  ✓ PWY-6897                       → 22/21
  ✓ NAD-BIOSYNTHESIS-II            → 12/11
  ✓ PYRIDNUCSAL-PWY                → 11/12
  ✓ PWY-7094                       → 14/13
  ✓ PWY-7527                       → 36/35
  ✓ PWY-7528                       → 35/36

Assigned: 15 salvage pathways
✓ Saved 349 assignments to harmonic_map.json
✓ Map: 349 pathways | 349 ratios


In [88]:
# ============================================================
# FINAL SUMMARY
# ============================================================

print("═" * 70)
print(" FINAL HARMONIC MAP SUMMARY")
print("═" * 70)

from collections import Counter
by_cat = Counter()
for pid, meta in HARMONIC_MAP.metadata.items():
    by_cat[meta.get('category', 'unknown')] += 1

print("\nAssigned by category:")
for cat in ['energy', 'biosynthesis', 'degradation', 'salvage', 'other']:
    target = len(df[df['category'] == cat])
    actual = by_cat.get(cat, 0)
    status = "✓" if actual == target else f"({actual - target:+d})"
    print(f"  {cat:<15} {actual:>3} / {target:<3} {status}")

print(f"\n  TOTAL: {len(HARMONIC_MAP.assignments)}")

# Prime limit distribution
print("\n" + "─" * 50)
print("PRIME LIMIT DISTRIBUTION:")

def prime_limit(n, d):
    primes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31]
    for val in [n, d]:
        for p in reversed(primes):
            if val % p == 0:
                return p
    return 1

limits = Counter()
for pid, (n, d) in HARMONIC_MAP.assignments.items():
    limits[prime_limit(n, d)] += 1

for lim in [1, 2, 3, 5, 7, 11, 13]:
    count = limits.get(lim, 0)
    pct = count / len(HARMONIC_MAP.assignments) * 100
    bar = "█" * int(pct / 3)
    print(f"  {lim:>2}-limit: {count:>3} ({pct:>5.1f}%) {bar}")

HARMONIC_MAP.save()
print(f"\n✓ Saved: {HARMONIC_MAP.status()}")


══════════════════════════════════════════════════════════════════════
 FINAL HARMONIC MAP SUMMARY
══════════════════════════════════════════════════════════════════════

Assigned by category:
  energy           27 / 27  ✓
  biosynthesis    143 / 143 ✓
  degradation     114 / 115 (-1)
  salvage          15 / 15  ✓
  other            50 / 50  ✓

  TOTAL: 349

──────────────────────────────────────────────────
PRIME LIMIT DISTRIBUTION:


TypeError: cannot unpack non-iterable Ratio object